In [ ]:
#!/usr/bin/env python
# -*- coding: utf8 -*-
# Импорт библиотек для поднятия spark-сессии
from mtspark import init_spark, kinit

kinit()
# Поднятие spark-сессии
spark = init_spark({"appName": "rkahmetov_analit_mob_all", # Меняем appName в соответствии с своей задачей + логин
                    "spark.sql.execution.arrow.enabled":"False",
                    'spark.jars.packages': 'ru.mts.bigdata.onetools:mtspark-scala-functions_2.12:1.0.4',
                    "spark.executor.memory": "4g",
                    "spark.driver.memory": "4g",
                    "spark.kryoserializer.buffer.max": "512"}, spark_version="3.2.0")

# Импорт библиотек для работы с spark

from pyspark import *
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import functions as F
from mtspark import save_as_table

import pandas as pd

In [ ]:
spark.stop()

In [ ]:
from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

In [ ]:
sch = 'analyticsb2b_sb.rkahmetov_schet_new2'
monthly_table = 'analyticsb2b_sb.rkahmetov_vr_mob'
monthly_table01 = 'analyticsb2b_sb.rkahmetov_vr_mob_202301'
monthly_table02 = 'analyticsb2b_sb.rkahmetov_vr_mob_202302'
monthly_table03 = 'analyticsb2b_sb.rkahmetov_vr_mob_202303'
source = 'analyticsb2b_sb.vas_iot_otchet'
source01 = 'analyticsb2b_sb.rkahmetov_correct_202301_2'
source02 = 'analyticsb2b_sb.rkahmetov_correct_202302_2'
source03 = 'analyticsb2b_sb.rkahmetov_correct_202303_2'
mnr01 = 'analyticsb2b_sb.rkahmetov_202301_netw_pl_call'
mnr02 = 'analyticsb2b_sb.rkahmetov_202302_netw_pl_call'
mnr03 = 'analyticsb2b_sb.rkahmetov_202303_netw_pl_call'

In [ ]:
%%time
spark.sql("""
select distinct 
    t1.regid
	, t1.app_n
from 
   (
	select 
		a.regid
		, a.app_n
		, b.product_pl_new
	from """+ source +""" as a 
	left join """ + sch + """ as b
		on round(a.group_id, 0) = round(b.group_id, 0) 
		and left(a.sub_acc_trans, 8) = b.sub_acc_trans
	) t1
where lower(t1.product_pl_new) = 'моб'
""").repartition(137).write.mode("overwrite").format("orc").saveAsTable(monthly_table+'_n')

In [ ]:
%%time
spark.sql("""
select distinct 
    t1.regid
	, t1.app_n
    , t1.con_typ1
	, date_format(to_date(t1.table_business_month), 'yyyyMM') as period
from """+ source +""" t1
where t1.regid||'-'||t1.app_n in
   (
    select distinct
        a.regid||'-'||a.app_n
    from """ + monthly_table + '_n' """ a
    )
""").repartition(2310).write.mode("overwrite").format("orc").saveAsTable(monthly_table+'_n1')

In [ ]:
%%time
spark.sql("""
select distinct 
    t1.regid
	, t1.app_n
    , t1.con_typ1
    , t2.segment as segment
	, t1.period
from """+ monthly_table + '_n1' """ t1
left join analyticsb2b_sb.rkahmetov_sprav_segment t2
	on lower(t1.con_typ1) = lower(t2.m_categ)
""").repartition(2310).write.mode("overwrite").format("orc").saveAsTable(monthly_table+'_n2')

In [ ]:
%%time
spark.sql("""
select distinct 
    t1.regid
	, t1.app_n
    , t1.con_typ1
    , t1.segment
	, t1.period
from """+ monthly_table + '_n2' """ t1
where COALESCE(t1.segment,0) not in ('B2C', 'Не в B2C')
""").repartition(365).write.mode("overwrite").format("orc").saveAsTable(monthly_table)

In [ ]:
month_table = spark.createDataFrame(["202101","202102","202103","202104","202105","202106","202107","202108","202109","202110","202111","202112",
                                     "202201","202202","202203","202204","202205","202206","202207","202208","202209", "202210", "202211", "202212", 
                                     "202301", "202302", "202303", "202304", "202305", "202306", "202307", "202308", "202309", "202310", "202311", 
                                    "202312", "202401", "202402", "202403", "202404"], "string").toDF("period")
month_table.registerTempTable('month_table_sql')                                     
spark.sql("""
SELECT 
    a1.*
    , a2.period
FROM (select distinct b.regid, b.app_n from """ + monthly_table + """ as b) as a1
CROSS JOIN month_table_sql as a2     
""").repartition(642).write.mode("overwrite").format("orc").saveAsTable("analyticsb2b_sb.rkahmetov_vr_mob_po_period")

In [ ]:
%%time
spark.sql("""
SELECT 
    t1.regid 
    , t1.app_n 
	, date_format(to_date(t1.table_business_month), 'yyyyMM') as period 
	, t2.product_pl_new
    , sum(t1.fclc_trans_part) as fclc_mob_total 
FROM """ + source + """ as t1
left join """ + sch + """ as t2
		on round(t1.group_id, 0) = round(t2.group_id, 0) 
		and left(t1.sub_acc_trans, 8) = t2.sub_acc_trans
WHERE t1.regid||t1.app_n IN (SELECT distinct a.regid||a.app_n FROM """ + monthly_table + """ a) 
    and lower(t1.service_id1) not like '%штраф%'
    and lower(t1.service_id1) not like '%дебет%коррект%'
	and lower(t2.product_pl_new) = 'моб'
    and t1.table_business_month not in ('2023-01-01', '2023-02-01', '2023-03-01')
group by
    t1.regid
    , t1.app_n
    , period
	, t2.product_pl_new
""").repartition(203).write.mode("overwrite").format("orc").saveAsTable(monthly_table+'_money')

In [ ]:
%%time
spark.sql("""
SELECT 
    t1.regid 
    , t1.app_n 
	, date_format(to_date(t1.table_business_month), 'yyyyMM') as period 
	, t2.product_pl_new
    , sum(t1.fclc_trans_part) as sum_fclc_no_vas 
FROM """ + source + """ as t1
left join """ + sch + """ as t2
		on round(t1.group_id, 0) = round(t2.group_id, 0) 
		and left(t1.sub_acc_trans, 8) = t2.sub_acc_trans
WHERE t1.regid||t1.app_n IN (SELECT distinct a.regid||a.app_n FROM """ + monthly_table + """ a) 
    and lower(t1.service_id1) not like '%штраф%'
    and lower(t1.service_id1) not like '%дебет%коррект%'
	and lower(t2.product_pl_new) = 'моб'
	and lower(t2.product_pl_det2) <> 'прочие vas'
    and lower(t1.service_id1) not like '%mobile%сервис%'
    and lower(t1.service_id1) not like 'b2b%оборудование%обслуживание%новая%цена%'
    and t1.table_business_month not in ('2023-01-01', '2023-02-01', '2023-03-01')
group by
    t1.regid
    , t1.app_n
    , period
	, t2.product_pl_new
""").repartition(203).write.mode("overwrite").format("orc").saveAsTable(monthly_table+'_money_no_vas')

In [ ]:
%%time
spark.sql("""
SELECT 
    t1.regid 
    , t1.app_n 
	, date_format(to_date(t1.table_business_month), 'yyyyMM') as period 
	, t2.product_pl_new
    , sum(t1.fclc_trans_part) as prochie_mob 
FROM """ + source + """ as t1
left join """ + sch + """ as t2
		on round(t1.group_id, 0) = round(t2.group_id, 0) 
		and left(t1.sub_acc_trans, 8) = t2.sub_acc_trans
WHERE t1.regid||t1.app_n IN (SELECT distinct a.regid||a.app_n FROM """ + monthly_table + """ a) 
    and lower(t1.service_id1) not like '%штраф%'
    and lower(t1.service_id1) not like '%дебет%коррект%'
	and lower(t2.product_pl_new) = 'моб'
	and lower(t2.product_pl_det2) = 'прочие моб'
    and lower(t1.service_id1) not like '%mobile%сервис%'
    and lower(t1.service_id1) not like 'b2b%оборудование%обслуживание%новая%цена%'
    and t1.table_business_month not in ('2023-01-01', '2023-02-01', '2023-03-01')
group by
    t1.regid
    , t1.app_n
    , period
	, t2.product_pl_new
""").repartition(100).write.mode("overwrite").format("orc").saveAsTable(monthly_table+'_money_prochm')

In [ ]:
%%time
spark.sql("""
SELECT 
    t1.regid 
    , t1.app_n 
	, date_format(to_date(t1.table_business_month), 'yyyyMM') as period 
	, t2.product_pl_new
    , sum(t1.fclc_trans_part) as prochie_vas 
FROM """ + source + """ as t1
left join """ + sch + """ as t2
		on round(t1.group_id, 0) = round(t2.group_id, 0) 
		and left(t1.sub_acc_trans, 8) = t2.sub_acc_trans
WHERE t1.regid||t1.app_n IN (SELECT distinct a.regid||a.app_n FROM """ + monthly_table + """ a) 
    and lower(t1.service_id1) not like '%штраф%'
    and lower(t1.service_id1) not like '%дебет%коррект%'
	and lower(t2.product_pl_new) = 'моб'
	and (lower(t2.product_pl_det2) = 'прочие vas'
    or lower(t1.service_id1) like '%mobile%сервис%'
    or lower(t1.service_id1) like 'b2b%оборудование%обслуживание%новая%цена%')
    and t1.table_business_month not in ('2023-01-01', '2023-02-01', '2023-03-01')
group by
    t1.regid
    , t1.app_n
    , period
	, t2.product_pl_new
""").repartition(1).write.mode("overwrite").format("orc").saveAsTable(monthly_table+'_money_prochv')

In [ ]:
%%time
spark.sql("""
SELECT 
    t1.regid 
    , t1.app_n 
	, date_format(to_date(t1.table_business_month), 'yyyyMM') as period 
	, t2.product_pl_new
    , sum(t1.fclc_trans_part) as gprs 
FROM """ + source + """ as t1
left join """ + sch + """ as t2
		on round(t1.group_id, 0) = round(t2.group_id, 0) 
		and left(t1.sub_acc_trans, 8) = t2.sub_acc_trans
WHERE t1.regid||t1.app_n IN (SELECT distinct a.regid||a.app_n FROM """ + monthly_table + """ a) 
    and lower(t1.service_id1) not like '%штраф%'
    and lower(t1.service_id1) not like '%дебет%коррект%'
	and lower(t2.product_pl_new) = 'моб'
	and lower(t2.product_pl_det2) = 'gprs'
    and lower(t1.service_id1) not like 'b2b%оборудование%обслуживание%новая%цена%'
    and t1.table_business_month not in ('2023-01-01', '2023-02-01', '2023-03-01')
group by
    t1.regid
    , t1.app_n
    , period
	, t2.product_pl_new
""").repartition(55).write.mode("overwrite").format("orc").saveAsTable(monthly_table+'_money_gprs')

In [ ]:
%%time
spark.sql("""
SELECT 
    t1.regid 
    , t1.app_n 
	, date_format(to_date(t1.table_business_month), 'yyyyMM') as period 
	, t2.product_pl_new
    , sum(t1.fclc_trans_part) as golos 
FROM """ + source + """ as t1
left join """ + sch + """ as t2
		on round(t1.group_id, 0) = round(t2.group_id, 0) 
		and left(t1.sub_acc_trans, 8) = t2.sub_acc_trans
WHERE t1.regid||t1.app_n IN (SELECT distinct a.regid||a.app_n FROM """ + monthly_table + """ a) 
    and lower(t1.service_id1) not like '%штраф%'
    and lower(t1.service_id1) not like '%дебет%коррект%'
	and lower(t2.product_pl_new) = 'моб'
	and lower(t2.product_pl_det2) = 'голос'
    and t1.table_business_month not in ('2023-01-01', '2023-02-01', '2023-03-01')
group by
    t1.regid
    , t1.app_n
    , period
	, t2.product_pl_new
""").repartition(1).write.mode("overwrite").format("orc").saveAsTable(monthly_table+'_money_golos')

In [ ]:
%%time
spark.sql("""
SELECT 
    t1.regid 
    , t1.app_n 
	, date_format(to_date(t1.table_business_month), 'yyyyMM') as period 
	, t2.product_pl_new
    , sum(t1.fclc_trans_part) as sms 
FROM """ + source + """ as t1
left join """ + sch + """ as t2
		on round(t1.group_id, 0) = round(t2.group_id, 0) 
		and left(t1.sub_acc_trans, 8) = t2.sub_acc_trans
WHERE t1.regid||t1.app_n IN (SELECT distinct a.regid||a.app_n FROM """ + monthly_table + """ a) 
    and lower(t1.service_id1) not like '%штраф%'
    and lower(t1.service_id1) not like '%дебет%коррект%'
	and lower(t2.product_pl_new) = 'моб'
	and lower(t2.product_pl_det2) = 'sms'
    and t1.table_business_month not in ('2023-01-01', '2023-02-01', '2023-03-01')
group by
    t1.regid
    , t1.app_n
    , period
	, t2.product_pl_new
""").repartition(40).write.mode("overwrite").format("orc").saveAsTable(monthly_table+'_money_sms')

In [ ]:
%%time
spark.sql("""
SELECT 
    t1.regid 
    , t1.app_n 
	, date_format(to_date(t1.table_business_month), 'yyyyMM') as period 
	, t2.product_pl_new
    , sum(t1.fclc_trans_part) as vsr 
FROM """ + source + """ as t1
left join """ + sch + """ as t2
		on round(t1.group_id, 0) = round(t2.group_id, 0) 
		and left(t1.sub_acc_trans, 8) = t2.sub_acc_trans
WHERE t1.regid||t1.app_n IN (SELECT distinct a.regid||a.app_n FROM """ + monthly_table + """ a) 
    and lower(t1.service_id1) not like '%штраф%'
    and lower(t1.service_id1) not like '%дебет%коррект%'
	and lower(t2.product_pl_new) = 'моб'
	and lower(t2.product_pl_det2) = 'вср'
    and t1.table_business_month not in ('2023-01-01', '2023-02-01', '2023-03-01')
group by
    t1.regid
    , t1.app_n
    , period
	, t2.product_pl_new
""").repartition(12).write.mode("overwrite").format("orc").saveAsTable(monthly_table+'_money_vsr')

In [ ]:
%%time
spark.sql("""
SELECT 
    t1.regid 
    , t1.app_n 
	, date_format(to_date(t1.table_business_month), 'yyyyMM') as period 
	, t2.product_pl_new
    , sum(t1.fclc_trans_part) as vz_mg_mn 
FROM """ + source + """ as t1
left join """ + sch + """ as t2
		on round(t1.group_id, 0) = round(t2.group_id, 0) 
		and left(t1.sub_acc_trans, 8) = t2.sub_acc_trans
WHERE t1.regid||t1.app_n IN (SELECT distinct a.regid||a.app_n FROM """ + monthly_table + """ a) 
    and lower(t1.service_id1) not like '%штраф%'
    and lower(t1.service_id1) not like '%дебет%коррект%'
	and lower(t2.product_pl_new) = 'моб'
	and lower(t2.product_pl_det2) = 'вз-мг-мн'
    and t1.table_business_month not in ('2023-01-01', '2023-02-01', '2023-03-01')
group by
    t1.regid
    , t1.app_n
    , period
	, t2.product_pl_new
""").repartition(52).write.mode("overwrite").format("orc").saveAsTable(monthly_table+'_money_vz')

In [ ]:
%%time
spark.sql("""
SELECT 
    t1.regid 
    , t1.app_n 
	, date_format(to_date(t1.table_business_month), 'yyyyMM') as period 
	, t2.product_pl_new
    , sum(t1.fclc_trans_part) as abon_plata 
FROM """ + source + """ as t1
left join """ + sch + """ as t2
		on round(t1.group_id, 0) = round(t2.group_id, 0) 
		and left(t1.sub_acc_trans, 8) = t2.sub_acc_trans
WHERE t1.regid||t1.app_n IN (SELECT distinct a.regid||a.app_n FROM """ + monthly_table + """ a) 
    and lower(t1.service_id1) not like '%штраф%'
    and lower(t1.service_id1) not like '%дебет%коррект%'
	and lower(t2.product_pl_new) = 'моб'
	and lower(t2.product_pl_det2) = 'абон.плата'
    and t1.table_business_month not in ('2023-01-01', '2023-02-01', '2023-03-01')
group by
    t1.regid
    , t1.app_n
    , period
	, t2.product_pl_new
""").repartition(122).write.mode("overwrite").format("orc").saveAsTable(monthly_table+'_money_abon')

In [ ]:
%%time
spark.sql("""
SELECT 
    t1.regid 
    , t1.app_n 
	, date_format(to_date(t1.table_business_month), 'yyyyMM') as period 
	, sum(t1.fclc_trans_part) as fclc_total 
FROM """ + source + """ as t1
WHERE t1.regid||t1.app_n IN (SELECT distinct a.regid||a.app_n FROM """ + monthly_table + """ a)
and t1.table_business_month not in ('2023-01-01', '2023-02-01', '2023-03-01')
group by
    t1.regid
    , t1.app_n
    , period
""").repartition(344).write.mode("overwrite").format("orc").saveAsTable(monthly_table+'_money_total')

In [ ]:
%%time
spark.sql("""
SELECT 
    t1.regid 
    , t1.app_n 
	, date_format(to_date(t1.table_business_month), 'yyyyMM') as period 
	, sum(t1.fclc_trans_part) as fclc_aff_serv 
FROM """ + source + """ as t1
WHERE t1.regid||t1.app_n IN (SELECT distinct a.regid||a.app_n FROM """ + monthly_table + """ a)
	and (lower(t1.service_id1) like '%добровольная%блокировка%'
        or lower(t1.service_id1) like '%ежесуточная%плата%по%тарифу%180%дней%' 
        or lower(t1.service_id1) like '%ежесуточная%плата%по%тарифу%90%дней%')
    and t1.table_business_month not in ('2023-01-01', '2023-02-01', '2023-03-01')
group by
    t1.regid
    , t1.app_n
    , period
""").repartition(13).write.mode("overwrite").format("orc").saveAsTable(monthly_table+'_money_aff_serv')

In [ ]:
#202301

In [ ]:
%%time
spark.sql("""
select
	t1.table_business_month
	, t1.regid
	, t1.app_n
	, t1.network_operator_id
	, t1.sub_acc_trans
	, t1.fclc_trans_part
	, t1.service_id1
	, t1.group_id
from """ + source + """ t1
where t1.table_business_month = '2023-01-01'
 and left(t1.sub_acc_trans, 8) IN 
	   (
		select distinct b.sub_acc_trans from """ + sch + """ as b where b.product_pl_new = "Моб"
		)
""").repartition(272).write.mode("overwrite").format("orc").saveAsTable("analyticsb2b_sb.rkahmetov_correct_202301")

In [ ]:
%%time
spark.sql("""
select
	t1.table_business_month
	, t1.regid
	, t1.app_n
	, t1.network_operator_id
	, t1.sub_acc_trans
	, t1.fclc_trans_part
	, t1.service_id1
	, t1.group_id
	, t2.place_of_call as place_of_call
from analyticsb2b_sb.rkahmetov_correct_202301 t1
left join """ + mnr01 + """ t2
    on round(t1.network_operator_id, 3) = round(t2.network_operator_id, 3)
""").repartition(272).write.mode("overwrite").format("orc").saveAsTable("analyticsb2b_sb.rkahmetov_correct_202301_1")

In [ ]:
%%time
spark.sql("""
select
	t1.table_business_month
	, t1.regid
	, t1.app_n
	, t1.network_operator_id
	, t1.sub_acc_trans
	, t1.fclc_trans_part
	, t1.service_id1
	, t1.group_id
	, t1.place_of_call
from analyticsb2b_sb.rkahmetov_correct_202301_1 t1
where t1.place_of_call <> 4 or t1.place_of_call is null
""").repartition(272).write.mode("overwrite").format("orc").saveAsTable("analyticsb2b_sb.rkahmetov_correct_202301_2")

In [ ]:
%%time
spark.sql("""
SELECT 
    t1.regid 
    , t1.app_n 
	, date_format(to_date(t1.table_business_month), 'yyyyMM') as period 
	, t2.product_pl_new
    , sum(t1.fclc_trans_part) as fclc_mob_total 
FROM """ + source01 + """ as t1
left join """ + sch + """ as t2
		on round(t1.group_id, 0) = round(t2.group_id, 0) 
		and left(t1.sub_acc_trans, 8) = t2.sub_acc_trans
WHERE t1.regid||t1.app_n IN (SELECT distinct a.regid||a.app_n FROM """ + monthly_table + """ a) 
    and lower(t1.service_id1) not like '%штраф%'
    and lower(t1.service_id1) not like '%дебет%коррект%'
	and lower(t2.product_pl_new) = 'моб'
group by
    t1.regid
    , t1.app_n
    , period
	, t2.product_pl_new
""").repartition(5).write.mode("overwrite").format("orc").saveAsTable(monthly_table01+'_money')

In [ ]:
%%time
spark.sql("""
SELECT 
    t1.regid 
    , t1.app_n 
	, date_format(to_date(t1.table_business_month), 'yyyyMM') as period 
	, t2.product_pl_new
    , sum(t1.fclc_trans_part) as sum_fclc_no_vas 
FROM """ + source01 + """ as t1
left join """ + sch + """ as t2
		on round(t1.group_id, 0) = round(t2.group_id, 0) 
		and left(t1.sub_acc_trans, 8) = t2.sub_acc_trans
WHERE t1.regid||t1.app_n IN (SELECT distinct a.regid||a.app_n FROM """ + monthly_table + """ a) 
    and lower(t1.service_id1) not like '%штраф%'
    and lower(t1.service_id1) not like '%дебет%коррект%'
	and lower(t2.product_pl_new) = 'моб'
	and lower(t2.product_pl_det2) <> 'прочие vas'
    and lower(t1.service_id1) not like '%mobile%сервис%'
    and lower(t1.service_id1) not like 'b2b%оборудование%обслуживание%новая%цена%'
group by
    t1.regid
    , t1.app_n
    , period
	, t2.product_pl_new
""").repartition(5).write.mode("overwrite").format("orc").saveAsTable(monthly_table01+'_money_no_vas')

In [ ]:
%%time
spark.sql("""
SELECT 
    t1.regid 
    , t1.app_n 
	, date_format(to_date(t1.table_business_month), 'yyyyMM') as period 
	, t2.product_pl_new
    , sum(t1.fclc_trans_part) as prochie_mob 
FROM """ + source01 + """ as t1
left join """ + sch + """ as t2
		on round(t1.group_id, 0) = round(t2.group_id, 0) 
		and left(t1.sub_acc_trans, 8) = t2.sub_acc_trans
WHERE t1.regid||t1.app_n IN (SELECT distinct a.regid||a.app_n FROM """ + monthly_table + """ a) 
    and lower(t1.service_id1) not like '%штраф%'
    and lower(t1.service_id1) not like '%дебет%коррект%'
	and lower(t2.product_pl_new) = 'моб'
	and lower(t2.product_pl_det2) = 'прочие моб'
    and lower(t1.service_id1) not like '%mobile%сервис%'
    and lower(t1.service_id1) not like 'b2b%оборудование%обслуживание%новая%цена%'
group by
    t1.regid
    , t1.app_n
    , period
	, t2.product_pl_new
""").repartition(4).write.mode("overwrite").format("orc").saveAsTable(monthly_table01+'_money_prochm')

In [ ]:
%%time
spark.sql("""
SELECT 
    t1.regid 
    , t1.app_n 
	, date_format(to_date(t1.table_business_month), 'yyyyMM') as period 
	, t2.product_pl_new
    , sum(t1.fclc_trans_part) as prochie_vas 
FROM """ + source01 + """ as t1
left join """ + sch + """ as t2
		on round(t1.group_id, 0) = round(t2.group_id, 0) 
		and left(t1.sub_acc_trans, 8) = t2.sub_acc_trans
WHERE t1.regid||t1.app_n IN (SELECT distinct a.regid||a.app_n FROM """ + monthly_table + """ a) 
    and lower(t1.service_id1) not like '%штраф%'
    and lower(t1.service_id1) not like '%дебет%коррект%'
	and lower(t2.product_pl_new) = 'моб'
	and (lower(t2.product_pl_det2) = 'прочие vas'
    or lower(t1.service_id1) like '%mobile%сервис%'
    or lower(t1.service_id1) like 'b2b%оборудование%обслуживание%новая%цена%')
group by
    t1.regid
    , t1.app_n
    , period
	, t2.product_pl_new
""").repartition(1).write.mode("overwrite").format("orc").saveAsTable(monthly_table01+'_money_prochv')

In [ ]:
%%time
spark.sql("""
SELECT 
    t1.regid 
    , t1.app_n 
	, date_format(to_date(t1.table_business_month), 'yyyyMM') as period 
	, t2.product_pl_new
    , sum(t1.fclc_trans_part) as gprs 
FROM """ + source01 + """ as t1
left join """ + sch + """ as t2
		on round(t1.group_id, 0) = round(t2.group_id, 0) 
		and left(t1.sub_acc_trans, 8) = t2.sub_acc_trans
WHERE t1.regid||t1.app_n IN (SELECT distinct a.regid||a.app_n FROM """ + monthly_table + """ a) 
    and lower(t1.service_id1) not like '%штраф%'
    and lower(t1.service_id1) not like '%дебет%коррект%'
	and lower(t2.product_pl_new) = 'моб'
	and lower(t2.product_pl_det2) = 'gprs'
    and lower(t1.service_id1) not like 'b2b%оборудование%обслуживание%новая%цена%'
group by
    t1.regid
    , t1.app_n
    , period
	, t2.product_pl_new
""").repartition(1).write.mode("overwrite").format("orc").saveAsTable(monthly_table01+'_money_gprs')

In [ ]:
%%time
spark.sql("""
SELECT 
    t1.regid 
    , t1.app_n 
	, date_format(to_date(t1.table_business_month), 'yyyyMM') as period 
	, t2.product_pl_new
    , sum(t1.fclc_trans_part) as golos 
FROM """ + source01 + """ as t1
left join """ + sch + """ as t2
		on round(t1.group_id, 0) = round(t2.group_id, 0) 
		and left(t1.sub_acc_trans, 8) = t2.sub_acc_trans
WHERE t1.regid||t1.app_n IN (SELECT distinct a.regid||a.app_n FROM """ + monthly_table + """ a) 
    and lower(t1.service_id1) not like '%штраф%'
    and lower(t1.service_id1) not like '%дебет%коррект%'
	and lower(t2.product_pl_new) = 'моб'
	and lower(t2.product_pl_det2) = 'голос'
group by
    t1.regid
    , t1.app_n
    , period
	, t2.product_pl_new
""").repartition(1).write.mode("overwrite").format("orc").saveAsTable(monthly_table01+'_money_golos')

In [ ]:
%%time
spark.sql("""
SELECT 
    t1.regid 
    , t1.app_n 
	, date_format(to_date(t1.table_business_month), 'yyyyMM') as period 
	, t2.product_pl_new
    , sum(t1.fclc_trans_part) as sms 
FROM """ + source01 + """ as t1
left join """ + sch + """ as t2
		on round(t1.group_id, 0) = round(t2.group_id, 0) 
		and left(t1.sub_acc_trans, 8) = t2.sub_acc_trans
WHERE t1.regid||t1.app_n IN (SELECT distinct a.regid||a.app_n FROM """ + monthly_table + """ a) 
    and lower(t1.service_id1) not like '%штраф%'
    and lower(t1.service_id1) not like '%дебет%коррект%'
	and lower(t2.product_pl_new) = 'моб'
	and lower(t2.product_pl_det2) = 'sms'
group by
    t1.regid
    , t1.app_n
    , period
	, t2.product_pl_new
""").repartition(1).write.mode("overwrite").format("orc").saveAsTable(monthly_table01+'_money_sms')

In [ ]:
%%time
spark.sql("""
SELECT 
    t1.regid 
    , t1.app_n 
	, date_format(to_date(t1.table_business_month), 'yyyyMM') as period 
	, t2.product_pl_new
    , sum(t1.fclc_trans_part) as vsr 
FROM """ + source01 + """ as t1
left join """ + sch + """ as t2
		on round(t1.group_id, 0) = round(t2.group_id, 0) 
		and left(t1.sub_acc_trans, 8) = t2.sub_acc_trans
WHERE t1.regid||t1.app_n IN (SELECT distinct a.regid||a.app_n FROM """ + monthly_table + """ a) 
    and lower(t1.service_id1) not like '%штраф%'
    and lower(t1.service_id1) not like '%дебет%коррект%'
	and lower(t2.product_pl_new) = 'моб'
	and lower(t2.product_pl_det2) = 'вср'
group by
    t1.regid
    , t1.app_n
    , period
	, t2.product_pl_new
""").repartition(1).write.mode("overwrite").format("orc").saveAsTable(monthly_table01+'_money_vsr')

In [ ]:
%%time
spark.sql("""
SELECT 
    t1.regid 
    , t1.app_n 
	, date_format(to_date(t1.table_business_month), 'yyyyMM') as period 
	, t2.product_pl_new
    , sum(t1.fclc_trans_part) as vz_mg_mn 
FROM """ + source01 + """ as t1
left join """ + sch + """ as t2
		on round(t1.group_id, 0) = round(t2.group_id, 0) 
		and left(t1.sub_acc_trans, 8) = t2.sub_acc_trans
WHERE t1.regid||t1.app_n IN (SELECT distinct a.regid||a.app_n FROM """ + monthly_table + """ a) 
    and lower(t1.service_id1) not like '%штраф%'
    and lower(t1.service_id1) not like '%дебет%коррект%'
	and lower(t2.product_pl_new) = 'моб'
	and lower(t2.product_pl_det2) = 'вз-мг-мн'
group by
    t1.regid
    , t1.app_n
    , period
	, t2.product_pl_new
""").repartition(1).write.mode("overwrite").format("orc").saveAsTable(monthly_table01+'_money_vz')

In [ ]:
%%time
spark.sql("""
SELECT 
    t1.regid 
    , t1.app_n 
	, date_format(to_date(t1.table_business_month), 'yyyyMM') as period 
	, t2.product_pl_new
    , sum(t1.fclc_trans_part) as abon_plata 
FROM """ + source01 + """ as t1
left join """ + sch + """ as t2
		on round(t1.group_id, 0) = round(t2.group_id, 0) 
		and left(t1.sub_acc_trans, 8) = t2.sub_acc_trans
WHERE t1.regid||t1.app_n IN (SELECT distinct a.regid||a.app_n FROM """ + monthly_table + """ a) 
    and lower(t1.service_id1) not like '%штраф%'
    and lower(t1.service_id1) not like '%дебет%коррект%'
	and lower(t2.product_pl_new) = 'моб'
	and lower(t2.product_pl_det2) = 'абон.плата'
group by
    t1.regid
    , t1.app_n
    , period
	, t2.product_pl_new
""").repartition(3).write.mode("overwrite").format("orc").saveAsTable(monthly_table01+'_money_abon')

In [ ]:
%%time
spark.sql("""
SELECT 
    t1.regid 
    , t1.app_n 
	, date_format(to_date(t1.table_business_month), 'yyyyMM') as period 
	, sum(t1.fclc_trans_part) as fclc_total 
FROM """ + source + """ as t1
WHERE t1.regid||t1.app_n IN (SELECT distinct a.regid||a.app_n FROM """ + monthly_table + """ a)
    and t1.table_business_month = '2023-01-01'
group by
    t1.regid
    , t1.app_n
    , period
""").repartition(9).write.mode("overwrite").format("orc").saveAsTable(monthly_table01+'_money_total')

In [ ]:
%%time
spark.sql("""
SELECT 
    t1.regid 
    , t1.app_n 
	, date_format(to_date(t1.table_business_month), 'yyyyMM') as period 
	, sum(t1.fclc_trans_part) as fclc_aff_serv 
FROM """ + source01 + """ as t1
WHERE t1.regid||t1.app_n IN (SELECT distinct a.regid||a.app_n FROM """ + monthly_table + """ a)
	and (lower(t1.service_id1) like '%добровольная%блокировка%'
        or lower(t1.service_id1) like '%ежесуточная%плата%по%тарифу%180%дней%' 
        or lower(t1.service_id1) like '%ежесуточная%плата%по%тарифу%90%дней%')
group by
    t1.regid
    , t1.app_n
    , period
""").repartition(1).write.mode("overwrite").format("orc").saveAsTable(monthly_table01+'_money_aff_serv')

In [ ]:
#202302

In [ ]:
%%time
spark.sql("""
select
	t1.table_business_month
	, t1.regid
	, t1.app_n
	, t1.network_operator_id
	, t1.sub_acc_trans
	, t1.fclc_trans_part
	, t1.service_id1
	, t1.group_id
from """ + source + """ t1
where t1.table_business_month = '2023-02-01'
 and left(t1.sub_acc_trans, 8) IN 
	   (
		select distinct b.sub_acc_trans from """ + sch + """ as b where b.product_pl_new = "Моб"
		)
""").repartition(294).write.mode("overwrite").format("orc").saveAsTable("analyticsb2b_sb.rkahmetov_correct_202302")

In [ ]:
%%time
spark.sql("""
select
	t1.table_business_month
	, t1.regid
	, t1.app_n
	, t1.network_operator_id
	, t1.sub_acc_trans
	, t1.fclc_trans_part
	, t1.service_id1
	, t1.group_id
	, t2.place_of_call as place_of_call
from analyticsb2b_sb.rkahmetov_correct_202302 t1
left join """ + mnr02 + """ t2
    on round(t1.network_operator_id, 3) = round(t2.network_operator_id, 3)
""").repartition(294).write.mode("overwrite").format("orc").saveAsTable("analyticsb2b_sb.rkahmetov_correct_202302_1")

In [ ]:
%%time
spark.sql("""
select
	t1.table_business_month
	, t1.regid
	, t1.app_n
	, t1.network_operator_id
	, t1.sub_acc_trans
	, t1.fclc_trans_part
	, t1.service_id1
	, t1.group_id
	, t1.place_of_call
from analyticsb2b_sb.rkahmetov_correct_202302_1 t1
where t1.place_of_call <> 4 or t1.place_of_call is null
""").repartition(294).write.mode("overwrite").format("orc").saveAsTable("analyticsb2b_sb.rkahmetov_correct_202302_2")

In [ ]:
%%time
spark.sql("""
SELECT 
    t1.regid 
    , t1.app_n 
	, date_format(to_date(t1.table_business_month), 'yyyyMM') as period 
	, t2.product_pl_new
    , sum(t1.fclc_trans_part) as fclc_mob_total 
FROM """ + source02 + """ as t1
left join """ + sch + """ as t2
		on round(t1.group_id, 0) = round(t2.group_id, 0) 
		and left(t1.sub_acc_trans, 8) = t2.sub_acc_trans
WHERE t1.regid||t1.app_n IN (SELECT distinct a.regid||a.app_n FROM """ + monthly_table + """ a) 
    and lower(t1.service_id1) not like '%штраф%'
    and lower(t1.service_id1) not like '%дебет%коррект%'
	and lower(t2.product_pl_new) = 'моб'
group by
    t1.regid
    , t1.app_n
    , period
	, t2.product_pl_new
""").repartition(5).write.mode("overwrite").format("orc").saveAsTable(monthly_table02+'_money')

In [ ]:
%%time
spark.sql("""
SELECT 
    t1.regid 
    , t1.app_n 
	, date_format(to_date(t1.table_business_month), 'yyyyMM') as period 
	, t2.product_pl_new
    , sum(t1.fclc_trans_part) as sum_fclc_no_vas 
FROM """ + source02 + """ as t1
left join """ + sch + """ as t2
		on round(t1.group_id, 0) = round(t2.group_id, 0) 
		and left(t1.sub_acc_trans, 8) = t2.sub_acc_trans
WHERE t1.regid||t1.app_n IN (SELECT distinct a.regid||a.app_n FROM """ + monthly_table + """ a) 
    and lower(t1.service_id1) not like '%штраф%'
    and lower(t1.service_id1) not like '%дебет%коррект%'
	and lower(t2.product_pl_new) = 'моб'
	and lower(t2.product_pl_det2) <> 'прочие vas'
    and lower(t1.service_id1) not like '%mobile%сервис%'
    and lower(t1.service_id1) not like 'b2b%оборудование%обслуживание%новая%цена%'
group by
    t1.regid
    , t1.app_n
    , period
	, t2.product_pl_new
""").repartition(5).write.mode("overwrite").format("orc").saveAsTable(monthly_table02+'_money_no_vas')

In [ ]:
%%time
spark.sql("""
SELECT 
    t1.regid 
    , t1.app_n 
	, date_format(to_date(t1.table_business_month), 'yyyyMM') as period 
	, t2.product_pl_new
    , sum(t1.fclc_trans_part) as prochie_mob 
FROM """ + source02 + """ as t1
left join """ + sch + """ as t2
		on round(t1.group_id, 0) = round(t2.group_id, 0) 
		and left(t1.sub_acc_trans, 8) = t2.sub_acc_trans
WHERE t1.regid||t1.app_n IN (SELECT distinct a.regid||a.app_n FROM """ + monthly_table + """ a) 
    and lower(t1.service_id1) not like '%штраф%'
    and lower(t1.service_id1) not like '%дебет%коррект%'
	and lower(t2.product_pl_new) = 'моб'
	and lower(t2.product_pl_det2) = 'прочие моб'
    and lower(t1.service_id1) not like '%mobile%сервис%'
    and lower(t1.service_id1) not like 'b2b%оборудование%обслуживание%новая%цена%'
group by
    t1.regid
    , t1.app_n
    , period
	, t2.product_pl_new
""").repartition(10).write.mode("overwrite").format("orc").saveAsTable(monthly_table02+'_money_prochm')

In [ ]:
%%time
spark.sql("""
SELECT 
    t1.regid 
    , t1.app_n 
	, date_format(to_date(t1.table_business_month), 'yyyyMM') as period 
	, t2.product_pl_new
    , sum(t1.fclc_trans_part) as prochie_vas 
FROM """ + source02 + """ as t1
left join """ + sch + """ as t2
		on round(t1.group_id, 0) = round(t2.group_id, 0) 
		and left(t1.sub_acc_trans, 8) = t2.sub_acc_trans
WHERE t1.regid||t1.app_n IN (SELECT distinct a.regid||a.app_n FROM """ + monthly_table + """ a) 
    and lower(t1.service_id1) not like '%штраф%'
    and lower(t1.service_id1) not like '%дебет%коррект%'
	and lower(t2.product_pl_new) = 'моб'
	and (lower(t2.product_pl_det2) = 'прочие vas'
    or lower(t1.service_id1) like '%mobile%сервис%'
    or lower(t1.service_id1) like 'b2b%оборудование%обслуживание%новая%цена%')
group by
    t1.regid
    , t1.app_n
    , period
	, t2.product_pl_new
""").repartition(1).write.mode("overwrite").format("orc").saveAsTable(monthly_table02+'_money_prochv')

In [ ]:
%%time
spark.sql("""
SELECT 
    t1.regid 
    , t1.app_n 
	, date_format(to_date(t1.table_business_month), 'yyyyMM') as period 
	, t2.product_pl_new
    , sum(t1.fclc_trans_part) as gprs 
FROM """ + source02 + """ as t1
left join """ + sch + """ as t2
		on round(t1.group_id, 0) = round(t2.group_id, 0) 
		and left(t1.sub_acc_trans, 8) = t2.sub_acc_trans
WHERE t1.regid||t1.app_n IN (SELECT distinct a.regid||a.app_n FROM """ + monthly_table + """ a) 
    and lower(t1.service_id1) not like '%штраф%'
    and lower(t1.service_id1) not like '%дебет%коррект%'
	and lower(t2.product_pl_new) = 'моб'
	and lower(t2.product_pl_det2) = 'gprs'
    and lower(t1.service_id1) not like 'b2b%оборудование%обслуживание%новая%цена%'
group by
    t1.regid
    , t1.app_n
    , period
	, t2.product_pl_new
""").repartition(1).write.mode("overwrite").format("orc").saveAsTable(monthly_table02+'_money_gprs')

In [ ]:
%%time
spark.sql("""
SELECT 
    t1.regid 
    , t1.app_n 
	, date_format(to_date(t1.table_business_month), 'yyyyMM') as period 
	, t2.product_pl_new
    , sum(t1.fclc_trans_part) as golos 
FROM """ + source02 + """ as t1
left join """ + sch + """ as t2
		on round(t1.group_id, 0) = round(t2.group_id, 0) 
		and left(t1.sub_acc_trans, 8) = t2.sub_acc_trans
WHERE t1.regid||t1.app_n IN (SELECT distinct a.regid||a.app_n FROM """ + monthly_table + """ a) 
    and lower(t1.service_id1) not like '%штраф%'
    and lower(t1.service_id1) not like '%дебет%коррект%'
	and lower(t2.product_pl_new) = 'моб'
	and lower(t2.product_pl_det2) = 'голос'
group by
    t1.regid
    , t1.app_n
    , period
	, t2.product_pl_new
""").repartition(1).write.mode("overwrite").format("orc").saveAsTable(monthly_table02+'_money_golos')

In [ ]:
%%time
spark.sql("""
SELECT 
    t1.regid 
    , t1.app_n 
	, date_format(to_date(t1.table_business_month), 'yyyyMM') as period 
	, t2.product_pl_new
    , sum(t1.fclc_trans_part) as sms 
FROM """ + source02 + """ as t1
left join """ + sch + """ as t2
		on round(t1.group_id, 0) = round(t2.group_id, 0) 
		and left(t1.sub_acc_trans, 8) = t2.sub_acc_trans
WHERE t1.regid||t1.app_n IN (SELECT distinct a.regid||a.app_n FROM """ + monthly_table + """ a) 
    and lower(t1.service_id1) not like '%штраф%'
    and lower(t1.service_id1) not like '%дебет%коррект%'
	and lower(t2.product_pl_new) = 'моб'
	and lower(t2.product_pl_det2) = 'sms'
group by
    t1.regid
    , t1.app_n
    , period
	, t2.product_pl_new
""").repartition(1).write.mode("overwrite").format("orc").saveAsTable(monthly_table02+'_money_sms')

In [ ]:
%%time
spark.sql("""
SELECT 
    t1.regid 
    , t1.app_n 
	, date_format(to_date(t1.table_business_month), 'yyyyMM') as period 
	, t2.product_pl_new
    , sum(t1.fclc_trans_part) as vsr 
FROM """ + source02 + """ as t1
left join """ + sch + """ as t2
		on round(t1.group_id, 0) = round(t2.group_id, 0) 
		and left(t1.sub_acc_trans, 8) = t2.sub_acc_trans
WHERE t1.regid||t1.app_n IN (SELECT distinct a.regid||a.app_n FROM """ + monthly_table + """ a) 
    and lower(t1.service_id1) not like '%штраф%'
    and lower(t1.service_id1) not like '%дебет%коррект%'
	and lower(t2.product_pl_new) = 'моб'
	and lower(t2.product_pl_det2) = 'вср'
group by
    t1.regid
    , t1.app_n
    , period
	, t2.product_pl_new
""").repartition(1).write.mode("overwrite").format("orc").saveAsTable(monthly_table02+'_money_vsr')

In [ ]:
%%time
spark.sql("""
SELECT 
    t1.regid 
    , t1.app_n 
	, date_format(to_date(t1.table_business_month), 'yyyyMM') as period 
	, t2.product_pl_new
    , sum(t1.fclc_trans_part) as vz_mg_mn 
FROM """ + source02 + """ as t1
left join """ + sch + """ as t2
		on round(t1.group_id, 0) = round(t2.group_id, 0) 
		and left(t1.sub_acc_trans, 8) = t2.sub_acc_trans
WHERE t1.regid||t1.app_n IN (SELECT distinct a.regid||a.app_n FROM """ + monthly_table + """ a) 
    and lower(t1.service_id1) not like '%штраф%'
    and lower(t1.service_id1) not like '%дебет%коррект%'
	and lower(t2.product_pl_new) = 'моб'
	and lower(t2.product_pl_det2) = 'вз-мг-мн'
group by
    t1.regid
    , t1.app_n
    , period
	, t2.product_pl_new
""").repartition(1).write.mode("overwrite").format("orc").saveAsTable(monthly_table02+'_money_vz')

In [ ]:
%%time
spark.sql("""
SELECT 
    t1.regid 
    , t1.app_n 
	, date_format(to_date(t1.table_business_month), 'yyyyMM') as period 
	, t2.product_pl_new
    , sum(t1.fclc_trans_part) as abon_plata 
FROM """ + source02 + """ as t1
left join """ + sch + """ as t2
		on round(t1.group_id, 0) = round(t2.group_id, 0) 
		and left(t1.sub_acc_trans, 8) = t2.sub_acc_trans
WHERE t1.regid||t1.app_n IN (SELECT distinct a.regid||a.app_n FROM """ + monthly_table + """ a) 
    and lower(t1.service_id1) not like '%штраф%'
    and lower(t1.service_id1) not like '%дебет%коррект%'
	and lower(t2.product_pl_new) = 'моб'
	and lower(t2.product_pl_det2) = 'абон.плата'
group by
    t1.regid
    , t1.app_n
    , period
	, t2.product_pl_new
""").repartition(3).write.mode("overwrite").format("orc").saveAsTable(monthly_table02+'_money_abon')

In [ ]:
%%time
spark.sql("""
SELECT 
    t1.regid 
    , t1.app_n 
	, date_format(to_date(t1.table_business_month), 'yyyyMM') as period 
	, sum(t1.fclc_trans_part) as fclc_total 
FROM """ + source + """ as t1
WHERE t1.regid||t1.app_n IN (SELECT distinct a.regid||a.app_n FROM """ + monthly_table + """ a)
    and t1.table_business_month = '2023-02-01'
group by
    t1.regid
    , t1.app_n
    , period
""").repartition(9).write.mode("overwrite").format("orc").saveAsTable(monthly_table02+'_money_total')

In [ ]:
%%time
spark.sql("""
SELECT 
    t1.regid 
    , t1.app_n 
	, date_format(to_date(t1.table_business_month), 'yyyyMM') as period 
	, sum(t1.fclc_trans_part) as fclc_aff_serv 
FROM """ + source02 + """ as t1
WHERE t1.regid||t1.app_n IN (SELECT distinct a.regid||a.app_n FROM """ + monthly_table + """ a)
	and (lower(t1.service_id1) like '%добровольная%блокировка%'
        or lower(t1.service_id1) like '%ежесуточная%плата%по%тарифу%180%дней%' 
        or lower(t1.service_id1) like '%ежесуточная%плата%по%тарифу%90%дней%')
group by
    t1.regid
    , t1.app_n
    , period
""").repartition(1).write.mode("overwrite").format("orc").saveAsTable(monthly_table02+'_money_aff_serv')

In [ ]:
#202303

In [ ]:
%%time
spark.sql("""
select
	t1.table_business_month
	, t1.regid
	, t1.app_n
	, t1.network_operator_id
	, t1.sub_acc_trans
	, t1.fclc_trans_part
	, t1.service_id1
	, t1.group_id
from """ + source + """ t1
where t1.table_business_month = '2023-03-01'
 and left(t1.sub_acc_trans, 8) IN 
	   (
		select distinct b.sub_acc_trans from """ + sch + """ as b where b.product_pl_new = "Моб"
		)
""").repartition(294).write.mode("overwrite").format("orc").saveAsTable("analyticsb2b_sb.rkahmetov_correct_202303")

In [ ]:
%%time
spark.sql("""
select
	t1.table_business_month
	, t1.regid
	, t1.app_n
	, t1.network_operator_id
	, t1.sub_acc_trans
	, t1.fclc_trans_part
	, t1.service_id1
	, t1.group_id
	, t2.place_of_call as place_of_call
from analyticsb2b_sb.rkahmetov_correct_202303 t1
left join """ + mnr03 + """ t2
    on round(t1.network_operator_id, 3) = round(t2.network_operator_id, 3)
""").repartition(294).write.mode("overwrite").format("orc").saveAsTable("analyticsb2b_sb.rkahmetov_correct_202303_1")

In [ ]:
%%time
spark.sql("""
select
	t1.table_business_month
	, t1.regid
	, t1.app_n
	, t1.network_operator_id
	, t1.sub_acc_trans
	, t1.fclc_trans_part
	, t1.service_id1
	, t1.group_id
	, t1.place_of_call
from analyticsb2b_sb.rkahmetov_correct_202303_1 t1
where t1.place_of_call <> 4 or t1.place_of_call is null
""").repartition(294).write.mode("overwrite").format("orc").saveAsTable("analyticsb2b_sb.rkahmetov_correct_202303_2")

In [ ]:
%%time
spark.sql("""
SELECT 
    t1.regid 
    , t1.app_n 
	, date_format(to_date(t1.table_business_month), 'yyyyMM') as period 
	, t2.product_pl_new
    , sum(t1.fclc_trans_part) as fclc_mob_total 
FROM """ + source03 + """ as t1
left join """ + sch + """ as t2
		on round(t1.group_id, 0) = round(t2.group_id, 0) 
		and left(t1.sub_acc_trans, 8) = t2.sub_acc_trans
WHERE t1.regid||t1.app_n IN (SELECT distinct a.regid||a.app_n FROM """ + monthly_table + """ a) 
    and lower(t1.service_id1) not like '%штраф%'
    and lower(t1.service_id1) not like '%дебет%коррект%'
	and lower(t2.product_pl_new) = 'моб'
group by
    t1.regid
    , t1.app_n
    , period
	, t2.product_pl_new
""").repartition(5).write.mode("overwrite").format("orc").saveAsTable(monthly_table03+'_money')

In [ ]:
%%time
spark.sql("""
SELECT 
    t1.regid 
    , t1.app_n 
	, date_format(to_date(t1.table_business_month), 'yyyyMM') as period 
	, t2.product_pl_new
    , sum(t1.fclc_trans_part) as sum_fclc_no_vas 
FROM """ + source03 + """ as t1
left join """ + sch + """ as t2
		on round(t1.group_id, 0) = round(t2.group_id, 0) 
		and left(t1.sub_acc_trans, 8) = t2.sub_acc_trans
WHERE t1.regid||t1.app_n IN (SELECT distinct a.regid||a.app_n FROM """ + monthly_table + """ a) 
    and lower(t1.service_id1) not like '%штраф%'
    and lower(t1.service_id1) not like '%дебет%коррект%'
	and lower(t2.product_pl_new) = 'моб'
	and lower(t2.product_pl_det2) <> 'прочие vas'
    and lower(t1.service_id1) not like '%mobile%сервис%'
    and lower(t1.service_id1) not like 'b2b%оборудование%обслуживание%новая%цена%'
group by
    t1.regid
    , t1.app_n
    , period
	, t2.product_pl_new
""").repartition(5).write.mode("overwrite").format("orc").saveAsTable(monthly_table03+'_money_no_vas')

In [ ]:
%%time
spark.sql("""
SELECT 
    t1.regid 
    , t1.app_n 
	, date_format(to_date(t1.table_business_month), 'yyyyMM') as period 
	, t2.product_pl_new
    , sum(t1.fclc_trans_part) as prochie_mob 
FROM """ + source03 + """ as t1
left join """ + sch + """ as t2
		on round(t1.group_id, 0) = round(t2.group_id, 0) 
		and left(t1.sub_acc_trans, 8) = t2.sub_acc_trans
WHERE t1.regid||t1.app_n IN (SELECT distinct a.regid||a.app_n FROM """ + monthly_table + """ a) 
    and lower(t1.service_id1) not like '%штраф%'
    and lower(t1.service_id1) not like '%дебет%коррект%'
	and lower(t2.product_pl_new) = 'моб'
	and lower(t2.product_pl_det2) = 'прочие моб'
    and lower(t1.service_id1) not like '%mobile%сервис%'
    and lower(t1.service_id1) not like 'b2b%оборудование%обслуживание%новая%цена%'
group by
    t1.regid
    , t1.app_n
    , period
	, t2.product_pl_new
""").repartition(10).write.mode("overwrite").format("orc").saveAsTable(monthly_table03+'_money_prochm')

In [ ]:
%%time
spark.sql("""
SELECT 
    t1.regid 
    , t1.app_n 
	, date_format(to_date(t1.table_business_month), 'yyyyMM') as period 
	, t2.product_pl_new
    , sum(t1.fclc_trans_part) as prochie_vas 
FROM """ + source03 + """ as t1
left join """ + sch + """ as t2
		on round(t1.group_id, 0) = round(t2.group_id, 0) 
		and left(t1.sub_acc_trans, 8) = t2.sub_acc_trans
WHERE t1.regid||t1.app_n IN (SELECT distinct a.regid||a.app_n FROM """ + monthly_table + """ a) 
    and lower(t1.service_id1) not like '%штраф%'
    and lower(t1.service_id1) not like '%дебет%коррект%'
	and lower(t2.product_pl_new) = 'моб'
	and (lower(t2.product_pl_det2) = 'прочие vas'
    or lower(t1.service_id1) like '%mobile%сервис%'
    or lower(t1.service_id1) like 'b2b%оборудование%обслуживание%новая%цена%')
group by
    t1.regid
    , t1.app_n
    , period
	, t2.product_pl_new
""").repartition(1).write.mode("overwrite").format("orc").saveAsTable(monthly_table03+'_money_prochv')

In [ ]:
%%time
spark.sql("""
SELECT 
    t1.regid 
    , t1.app_n 
	, date_format(to_date(t1.table_business_month), 'yyyyMM') as period 
	, t2.product_pl_new
    , sum(t1.fclc_trans_part) as gprs 
FROM """ + source03 + """ as t1
left join """ + sch + """ as t2
		on round(t1.group_id, 0) = round(t2.group_id, 0) 
		and left(t1.sub_acc_trans, 8) = t2.sub_acc_trans
WHERE t1.regid||t1.app_n IN (SELECT distinct a.regid||a.app_n FROM """ + monthly_table + """ a) 
    and lower(t1.service_id1) not like '%штраф%'
    and lower(t1.service_id1) not like '%дебет%коррект%'
	and lower(t2.product_pl_new) = 'моб'
	and lower(t2.product_pl_det2) = 'gprs'
    and lower(t1.service_id1) not like 'b2b%оборудование%обслуживание%новая%цена%'
group by
    t1.regid
    , t1.app_n
    , period
	, t2.product_pl_new
""").repartition(1).write.mode("overwrite").format("orc").saveAsTable(monthly_table03+'_money_gprs')

In [ ]:
%%time
spark.sql("""
SELECT 
    t1.regid 
    , t1.app_n 
	, date_format(to_date(t1.table_business_month), 'yyyyMM') as period 
	, t2.product_pl_new
    , sum(t1.fclc_trans_part) as golos 
FROM """ + source03 + """ as t1
left join """ + sch + """ as t2
		on round(t1.group_id, 0) = round(t2.group_id, 0) 
		and left(t1.sub_acc_trans, 8) = t2.sub_acc_trans
WHERE t1.regid||t1.app_n IN (SELECT distinct a.regid||a.app_n FROM """ + monthly_table + """ a) 
    and lower(t1.service_id1) not like '%штраф%'
    and lower(t1.service_id1) not like '%дебет%коррект%'
	and lower(t2.product_pl_new) = 'моб'
	and lower(t2.product_pl_det2) = 'голос'
group by
    t1.regid
    , t1.app_n
    , period
	, t2.product_pl_new
""").repartition(1).write.mode("overwrite").format("orc").saveAsTable(monthly_table03+'_money_golos')

In [ ]:
%%time
spark.sql("""
SELECT 
    t1.regid 
    , t1.app_n 
	, date_format(to_date(t1.table_business_month), 'yyyyMM') as period 
	, t2.product_pl_new
    , sum(t1.fclc_trans_part) as sms 
FROM """ + source03 + """ as t1
left join """ + sch + """ as t2
		on round(t1.group_id, 0) = round(t2.group_id, 0) 
		and left(t1.sub_acc_trans, 8) = t2.sub_acc_trans
WHERE t1.regid||t1.app_n IN (SELECT distinct a.regid||a.app_n FROM """ + monthly_table + """ a) 
    and lower(t1.service_id1) not like '%штраф%'
    and lower(t1.service_id1) not like '%дебет%коррект%'
	and lower(t2.product_pl_new) = 'моб'
	and lower(t2.product_pl_det2) = 'sms'
group by
    t1.regid
    , t1.app_n
    , period
	, t2.product_pl_new
""").repartition(1).write.mode("overwrite").format("orc").saveAsTable(monthly_table03+'_money_sms')

In [ ]:
%%time
spark.sql("""
SELECT 
    t1.regid 
    , t1.app_n 
	, date_format(to_date(t1.table_business_month), 'yyyyMM') as period 
	, t2.product_pl_new
    , sum(t1.fclc_trans_part) as vsr 
FROM """ + source03 + """ as t1
left join """ + sch + """ as t2
		on round(t1.group_id, 0) = round(t2.group_id, 0) 
		and left(t1.sub_acc_trans, 8) = t2.sub_acc_trans
WHERE t1.regid||t1.app_n IN (SELECT distinct a.regid||a.app_n FROM """ + monthly_table + """ a) 
    and lower(t1.service_id1) not like '%штраф%'
    and lower(t1.service_id1) not like '%дебет%коррект%'
	and lower(t2.product_pl_new) = 'моб'
	and lower(t2.product_pl_det2) = 'вср'
group by
    t1.regid
    , t1.app_n
    , period
	, t2.product_pl_new
""").repartition(1).write.mode("overwrite").format("orc").saveAsTable(monthly_table03+'_money_vsr')

In [ ]:
%%time
spark.sql("""
SELECT 
    t1.regid 
    , t1.app_n 
	, date_format(to_date(t1.table_business_month), 'yyyyMM') as period 
	, t2.product_pl_new
    , sum(t1.fclc_trans_part) as vz_mg_mn 
FROM """ + source03 + """ as t1
left join """ + sch + """ as t2
		on round(t1.group_id, 0) = round(t2.group_id, 0) 
		and left(t1.sub_acc_trans, 8) = t2.sub_acc_trans
WHERE t1.regid||t1.app_n IN (SELECT distinct a.regid||a.app_n FROM """ + monthly_table + """ a) 
    and lower(t1.service_id1) not like '%штраф%'
    and lower(t1.service_id1) not like '%дебет%коррект%'
	and lower(t2.product_pl_new) = 'моб'
	and lower(t2.product_pl_det2) = 'вз-мг-мн'
group by
    t1.regid
    , t1.app_n
    , period
	, t2.product_pl_new
""").repartition(1).write.mode("overwrite").format("orc").saveAsTable(monthly_table03+'_money_vz')

In [ ]:
%%time
spark.sql("""
SELECT 
    t1.regid 
    , t1.app_n 
	, date_format(to_date(t1.table_business_month), 'yyyyMM') as period 
	, t2.product_pl_new
    , sum(t1.fclc_trans_part) as abon_plata 
FROM """ + source03 + """ as t1
left join """ + sch + """ as t2
		on round(t1.group_id, 0) = round(t2.group_id, 0) 
		and left(t1.sub_acc_trans, 8) = t2.sub_acc_trans
WHERE t1.regid||t1.app_n IN (SELECT distinct a.regid||a.app_n FROM """ + monthly_table + """ a) 
    and lower(t1.service_id1) not like '%штраф%'
    and lower(t1.service_id1) not like '%дебет%коррект%'
	and lower(t2.product_pl_new) = 'моб'
	and lower(t2.product_pl_det2) = 'абон.плата'
group by
    t1.regid
    , t1.app_n
    , period
	, t2.product_pl_new
""").repartition(3).write.mode("overwrite").format("orc").saveAsTable(monthly_table03+'_money_abon')

In [ ]:
%%time
spark.sql("""
SELECT 
    t1.regid 
    , t1.app_n 
	, date_format(to_date(t1.table_business_month), 'yyyyMM') as period 
	, sum(t1.fclc_trans_part) as fclc_total 
FROM """ + source + """ as t1
WHERE t1.regid||t1.app_n IN (SELECT distinct a.regid||a.app_n FROM """ + monthly_table + """ a)
    and t1.table_business_month = '2023-03-01'
group by
    t1.regid
    , t1.app_n
    , period
""").repartition(9).write.mode("overwrite").format("orc").saveAsTable(monthly_table03+'_money_total')

In [ ]:
%%time
spark.sql("""
SELECT 
    t1.regid 
    , t1.app_n 
	, date_format(to_date(t1.table_business_month), 'yyyyMM') as period 
	, sum(t1.fclc_trans_part) as fclc_aff_serv 
FROM """ + source03 + """ as t1
WHERE t1.regid||t1.app_n IN (SELECT distinct a.regid||a.app_n FROM """ + monthly_table + """ a)
	and (lower(t1.service_id1) like '%добровольная%блокировка%'
        or lower(t1.service_id1) like '%ежесуточная%плата%по%тарифу%180%дней%' 
        or lower(t1.service_id1) like '%ежесуточная%плата%по%тарифу%90%дней%')
group by
    t1.regid
    , t1.app_n
    , period
""").repartition(1).write.mode("overwrite").format("orc").saveAsTable(monthly_table03+'_money_aff_serv')

In [ ]:
%%time
spark.sql("""
SELECT 
    t1.*
FROM """ + monthly_table + '_money' """ as t1
union all
SELECT 
    t2.*
FROM """ + monthly_table01 + '_money' """ as t2
union all
SELECT 
    t3.*
FROM """ + monthly_table02 + '_money' """ as t3
union all
SELECT 
    t4.*
FROM """ + monthly_table03 + '_money' """ as t4

""").repartition(200).write.mode("overwrite").format("orc").saveAsTable(monthly_table+'_money_end')

In [ ]:
%%time
spark.sql("""
SELECT 
    t1.*
FROM """ + monthly_table + '_money_no_vas' """ as t1
union all
SELECT 
    t2.*
FROM """ + monthly_table01 + '_money_no_vas' """ as t2
union all
SELECT 
    t3.*
FROM """ + monthly_table02 + '_money_no_vas' """ as t3
union all
SELECT 
    t4.*
FROM """ + monthly_table03 + '_money_no_vas' """ as t4
""").repartition(200).write.mode("overwrite").format("orc").saveAsTable(monthly_table+'_money_no_vas_end')

In [ ]:
%%time
spark.sql("""
SELECT 
    t1.*
FROM """ + monthly_table + '_money_prochm' """ as t1
union all
SELECT 
    t2.*
FROM """ + monthly_table01 + '_money_prochm' """ as t2
union all
SELECT 
    t3.*
FROM """ + monthly_table02 + '_money_prochm' """ as t3
union all
SELECT 
    t4.*
FROM """ + monthly_table03 + '_money_prochm' """ as t4
""").repartition(90).write.mode("overwrite").format("orc").saveAsTable(monthly_table+'_money_prochm_end')

In [ ]:
%%time
spark.sql("""
SELECT 
    t1.*
FROM """ + monthly_table + '_money_prochv' """ as t1
union all
SELECT 
    t2.*
FROM """ + monthly_table01 + '_money_prochv' """ as t2
union all
SELECT 
    t3.*
FROM """ + monthly_table02 + '_money_prochv' """ as t3
union all
SELECT 
    t4.*
FROM """ + monthly_table03 + '_money_prochv' """ as t4
""").repartition(10).write.mode("overwrite").format("orc").saveAsTable(monthly_table+'_money_prochv_end')

In [ ]:
%%time
spark.sql("""
SELECT 
    t1.*
FROM """ + monthly_table + '_money_gprs' """ as t1
union all
SELECT 
    t2.*
FROM """ + monthly_table01 + '_money_gprs' """ as t2
union all
SELECT 
    t3.*
FROM """ + monthly_table02 + '_money_gprs' """ as t3
union all
SELECT 
    t4.*
FROM """ + monthly_table03 + '_money_gprs' """ as t4
""").repartition(50).write.mode("overwrite").format("orc").saveAsTable(monthly_table+'_money_gprs_end')

In [ ]:
%%time
spark.sql("""
SELECT 
    t1.*
FROM """ + monthly_table + '_money_golos' """ as t1
union all
SELECT 
    t2.*
FROM """ + monthly_table01 + '_money_golos' """ as t2
union all
SELECT 
    t3.*
FROM """ + monthly_table02 + '_money_golos' """ as t3
union all
SELECT 
    t4.*
FROM """ + monthly_table03 + '_money_golos' """ as t4
""").repartition(10).write.mode("overwrite").format("orc").saveAsTable(monthly_table+'_money_golos_end')

In [ ]:
%%time
spark.sql("""
SELECT 
    t1.*
FROM """ + monthly_table + '_money_sms' """ as t1
union all
SELECT 
    t2.*
FROM """ + monthly_table01 + '_money_sms' """ as t2
union all
SELECT 
    t3.*
FROM """ + monthly_table02 + '_money_sms' """ as t3
union all
SELECT 
    t4.*
FROM """ + monthly_table03 + '_money_sms' """ as t4
""").repartition(40).write.mode("overwrite").format("orc").saveAsTable(monthly_table+'_money_sms_end')

In [ ]:
%%time
spark.sql("""
SELECT 
    t1.*
FROM """ + monthly_table + '_money_vsr' """ as t1
union all
SELECT 
    t2.*
FROM """ + monthly_table01 + '_money_vsr' """ as t2
union all
SELECT 
    t3.*
FROM """ + monthly_table02 + '_money_vsr' """ as t3
union all
SELECT 
    t4.*
FROM """ + monthly_table03 + '_money_vsr' """ as t4
""").repartition(10).write.mode("overwrite").format("orc").saveAsTable(monthly_table+'_money_vsr_end')

In [ ]:
%%time
spark.sql("""
SELECT 
    t1.*
FROM """ + monthly_table + '_money_vz' """ as t1
union all
SELECT 
    t2.*
FROM """ + monthly_table01 + '_money_vz' """ as t2
union all
SELECT 
    t3.*
FROM """ + monthly_table02 + '_money_vz' """ as t3
union all
SELECT 
    t4.*
FROM """ + monthly_table03 + '_money_vz' """ as t4
""").repartition(50).write.mode("overwrite").format("orc").saveAsTable(monthly_table+'_money_vz_end')

In [ ]:
%%time
spark.sql("""
SELECT 
    t1.*
FROM """ + monthly_table + '_money_abon' """ as t1
union all
SELECT 
    t2.*
FROM """ + monthly_table01 + '_money_abon' """ as t2
union all
SELECT 
    t3.*
FROM """ + monthly_table02 + '_money_abon' """ as t3
union all
SELECT 
    t4.*
FROM """ + monthly_table03 + '_money_abon' """ as t4
""").repartition(110).write.mode("overwrite").format("orc").saveAsTable(monthly_table+'_money_abon_end')

In [ ]:
%%time
spark.sql("""
SELECT 
    t1.*
FROM """ + monthly_table + '_money_total' """ as t1
union all
SELECT 
    t2.*
FROM """ + monthly_table01 + '_money_total' """ as t2
union all
SELECT 
    t3.*
FROM """ + monthly_table02 + '_money_total' """ as t3
union all
SELECT 
    t4.*
FROM """ + monthly_table03 + '_money_total' """ as t4
""").repartition(350).write.mode("overwrite").format("orc").saveAsTable(monthly_table+'_money_total_end')

In [ ]:
%%time
spark.sql("""
SELECT 
    t1.*
FROM """ + monthly_table + '_money_aff_serv' """ as t1
union all
SELECT 
    t2.*
FROM """ + monthly_table01 + '_money_aff_serv' """ as t2
union all
SELECT 
    t3.*
FROM """ + monthly_table02 + '_money_aff_serv' """ as t3
union all
SELECT 
    t4.*
FROM """ + monthly_table03 + '_money_aff_serv' """ as t4
""").repartition(10).write.mode("overwrite").format("orc").saveAsTable(monthly_table+'_money_aff_serv_end')

In [ ]:
%%time
spark.sql("""
SELECT 
    t1.period
    , t1.regid
    , t1.app_n
    , t2.prochie_mob
    , t3.prochie_vas
    , t4.gprs
    , t5.golos
    , t6.sms
    , t7.vsr
    , t8.vz_mg_mn
    , t9.abon_plata
    , t10.fclc_mob_total as sum_fclc
    , t11.sum_fclc_no_vas
    , t12.fclc_total
    , t13.fclc_aff_serv
FROM analyticsb2b_sb.rkahmetov_vr_mob_po_period as t1
LEFT JOIN """ + monthly_table + '_money_prochm_end' """ as t2
    ON t1.app_n = t2.app_n 
    and t1.regid = t2.regid
    and t1.period = t2.period
LEFT JOIN """ + monthly_table + '_money_prochv_end' """ as t3
    ON t1.app_n = t3.app_n 
    and t1.regid = t3.regid
    and t1.period = t3.period
LEFT JOIN """ + monthly_table + '_money_gprs_end' """ as t4
    ON t1.app_n = t4.app_n 
    and t1.regid = t4.regid
    and t1.period = t4.period
LEFT JOIN """ + monthly_table + '_money_golos_end' """ as t5
    ON t1.app_n = t5.app_n 
    and t1.regid = t5.regid
    and t1.period = t5.period
LEFT JOIN """ + monthly_table + '_money_sms_end' """ as t6
    ON t1.app_n = t6.app_n 
    and t1.regid = t6.regid
    and t1.period = t6.period
LEFT JOIN """ + monthly_table + '_money_vsr_end' """ as t7
    ON t1.app_n = t7.app_n 
    and t1.regid = t7.regid
    and t1.period = t7.period
LEFT JOIN """ + monthly_table + '_money_vz_end' """ as t8
    ON t1.app_n = t8.app_n 
    and t1.regid = t8.regid
    and t1.period = t8.period
LEFT JOIN """ + monthly_table + '_money_abon_end' """ as t9
    ON t1.app_n = t9.app_n 
    and t1.regid = t9.regid
    and t1.period = t9.period
LEFT JOIN """ + monthly_table + '_money_end' """ as t10
    ON t1.app_n = t10.app_n 
    and t1.regid = t10.regid
    and t1.period = t10.period
LEFT JOIN """ + monthly_table + '_money_no_vas_end' """ as t11
    ON t1.app_n = t11.app_n 
    and t1.regid = t11.regid
    and t1.period = t11.period
left join """ + monthly_table + '_money_total_end' """ as t12
    ON t1.app_n = t12.app_n 
    and t1.regid = t12.regid
    and t1.period = t12.period
LEFT JOIN """ + monthly_table + '_money_aff_serv_end' """ as t13 
    ON t1.app_n = t13.app_n 
    and t1.regid = t13.regid
    and t1.period = t13.period
""").repartition(620).write.mode("overwrite").format("orc").saveAsTable("analyticsb2b_sb.rkahmetov_vr_mob_vrem_1")

In [ ]:
%%time
spark.sql("""
SELECT distinct
    date_format(to_date(t1.table_business_month), 'yyyyMM') as period
    , t1.regid 
    , t1.app_n 
    , t1.phone_num
    , t1.con_n 
	, t1.kontr_name 
	, t1.activation_date 
	, t1.region
    , t1.con_typ
	, t1.con_typ1 
    , t1.tariff_plan
	, t1.tariff_plan1 
    , t1.tp_group
	, t1.tp_group1
FROM """ + source + """ as t1 
WHERE t1.regid||'-'||t1.app_n IN (select distinct d1.regid||'-'||d1.app_n from analyticsb2b_sb.rkahmetov_vr_mob_vrem_1 d1)
""").repartition(300).write.mode("overwrite").format("orc").saveAsTable("analyticsb2b_sb.rkahmetov_vr_mob_vrem_2")

In [ ]:
%%time
spark.sql("""
SELECT
    t1.period
    ,t1.regid
    ,t1.app_n
    ,t1.prochie_mob as fclc_prochie_mob1
    ,t1.prochie_vas as fclc_prochie_vas1
    ,t1.golos as fclc_golos1
    ,t1.gprs as fclc_gprs1
    ,t1.sms as fclc_sms1
    ,t1.vsr as fclc_vsr1
    ,t1.vz_mg_mn as fclc_vz_mg_mn1
    ,t1.abon_plata as fclc_abon_plata1
    ,t1.sum_fclc as fclc_mob_total1
    ,t1.sum_fclc_no_vas as fclc_mob_no_vas1
    ,t1.fclc_total as fclc_total1
    ,t1.fclc_aff_serv as fclc_aff_serv1
	,t2.phone_num
    ,t2.con_n 
	,t2.kontr_name 
	,t2.activation_date 
	,t2.region
    ,t2.con_typ
	,t2.con_typ1 
    ,t2.tariff_plan
	,t2.tariff_plan1 
    ,t2.tp_group
	,t2.tp_group1
from analyticsb2b_sb.rkahmetov_vr_mob_vrem_1 t1
left join analyticsb2b_sb.rkahmetov_vr_mob_vrem_2 t2
	on t1.period = t2.period
	and t1.regid = t2.regid
	and t1.app_n = t2.app_n
""").repartition(550).write.mode("overwrite").format("orc").saveAsTable("analyticsb2b_sb.rkahmetov_vr_mob_vrem_3")

In [ ]:
%%time
spark.sql("""
SELECT
    t1.* 
	, CASE
        when t1.con_n is NULL 
            then first_value(t1.con_n) over (partition by t1.regid, t1.app_n, t1.part_con_n order by t1.period desc)
        else t1.con_n
        end as con_n_x
	, CASE
        when t1.kontr_name is NULL 
            then first_value(t1.kontr_name) over (partition by t1.regid, t1.app_n, t1.part_kontr_name order by t1.period desc)
        else t1.kontr_name
        end as kontr_name_x
	, CASE
        when t1.region is NULL 
            then first_value(t1.region) over (partition by t1.regid, t1.app_n, t1.part_region order by t1.period desc)
        else t1.region
        end as region_x
	, CASE
        when t1.activation_date is NULL 
            then first_value(t1.activation_date) over (partition by t1.regid, t1.app_n, t1.part_activation_date order by t1.period desc)
        else t1.activation_date
        end as activation_date_x
	, CASE
        when t1.tariff_plan is NULL 
            then first_value(t1.tariff_plan) over (partition by t1.regid, t1.app_n, t1.part_tariff_plan order by t1.period desc)
        else t1.tariff_plan
        end as tariff_plan_x
	, CASE
        when t1.tp_group is NULL 
            then first_value(t1.tp_group) over (partition by t1.regid, t1.app_n, t1.part_tp_group order by t1.period desc)
        else t1.tp_group
        end as tp_group_x
	, CASE
        when t1.con_typ is NULL 
            then first_value(t1.con_typ) over (partition by t1.regid, t1.app_n, t1.part_con_typ order by t1.period desc)
        else t1.con_typ
        end as con_typ_x
    

	, CASE
        when t1.tariff_plan1 is NULL 
            then first_value(t1.tariff_plan1) over (partition by t1.regid, t1.app_n, t1.part_tariff_plan1 order by t1.period desc)
        else t1.tariff_plan1
        end as tariff_plan1_x
	, CASE
        when t1.tp_group1 is NULL 
            then first_value(t1.tp_group1) over (partition by t1.regid, t1.app_n, t1.part_tp_group1 order by t1.period desc)
        else t1.tp_group1
        end as tp_group1_x
	, CASE
        when t1.con_typ1 is NULL 
            then first_value(t1.con_typ1) over (partition by t1.regid, t1.app_n, t1.part_con_typ1 order by t1.period desc)
        else t1.con_typ1
        end as con_typ1_x
	, CASE
        when t1.phone_num is NULL 
            then first_value(t1.phone_num) over (partition by t1.regid, t1.app_n, t1.part_phone_num order by t1.period desc)
        else t1.phone_num
        end as phone_num_x    
FROM 
   (
    select 
        a.*
        , sum(case when a.con_n is null then 0 else 1 end) 
			over (partition by a.regid, a.app_n order by a.period desc) part_con_n
		, sum(case when a.kontr_name is null then 0 else 1 end) 
			over (partition by a.regid, a.app_n order by a.period desc) part_kontr_name
		, sum(case when a.region is null then 0 else 1 end) 
			over (partition by a.regid, a.app_n order by a.period desc) part_region
		, sum(case when a.activation_date is null then 0 else 1 end) 
			over (partition by a.regid, a.app_n order by a.period desc) part_activation_date
		, sum(case when a.tariff_plan is null then 0 else 1 end) 
			over (partition by a.regid, a.app_n order by a.period desc) part_tariff_plan
		, sum(case when a.tp_group is null then 0 else 1 end) 
			over (partition by a.regid, a.app_n order by a.period desc) part_tp_group
		, sum(case when a.con_typ is null then 0 else 1 end) 
			over (partition by a.regid, a.app_n order by a.period desc) part_con_typ
        
		
        , sum(case when a.tariff_plan1 is null then 0 else 1 end) 
			over (partition by a.regid, a.app_n order by a.period desc) part_tariff_plan1
		, sum(case when a.tp_group1 is null then 0 else 1 end) 
			over (partition by a.regid, a.app_n order by a.period desc) part_tp_group1
		, sum(case when a.con_typ1 is null then 0 else 1 end) 
			over (partition by a.regid, a.app_n order by a.period desc) part_con_typ1
		, sum(case when a.phone_num is null then 0 else 1 end) 
			over (partition by a.regid, a.app_n order by a.period desc) part_phone_num
    from analyticsb2b_sb.rkahmetov_vr_mob_vrem_3 as a
    ) as t1
""").repartition(620).write.mode("overwrite").format("orc").saveAsTable("analyticsb2b_sb.rkahmetov_vr_mob_vrem_4")

In [ ]:
%%time
spark.sql("""
SELECT
    t1.*
    , CASE
        when 
           (
            row_number() OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period)
            ) = 1
            THEN t1.con_n_x
        ELSE t1.con_n
        end as con_n_x1
	, CASE
        when 
           (
            row_number() OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period)
            ) = 1
            THEN t1.kontr_name_x
        ELSE t1.kontr_name
        end as kontr_name_x1
	, CASE
        when 
           (
            row_number() OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period)
            ) = 1
			THEN t1.region_x
        ELSE t1.region
        end as region_x1
	, CASE
        when 
           (
            row_number() OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period)
            ) = 1
            THEN t1.activation_date_x
        ELSE t1.activation_date
        end as activation_date_x1
	, CASE
        when 
           (
            row_number() OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period)
            ) = 1
            THEN t1.tariff_plan_x
        ELSE t1.tariff_plan
        end as tariff_plan_x1 
	, CASE
        when 
           (
            row_number() OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period)
            ) = 1
            THEN t1.tp_group_x
        ELSE t1.tp_group
        end as tp_group_x1 
	, CASE
        when 
           (
            row_number() OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period)
            ) = 1
            THEN t1.con_typ_x
        ELSE t1.con_typ
        end as con_typ_x1
		
	
	, CASE
        when 
           (
            row_number() OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period)
            ) = 1
            THEN t1.tariff_plan1_x
        ELSE t1.tariff_plan1
        end as tariff_plan1_x1 
	, CASE
        when 
           (
            row_number() OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period)
            ) = 1
            THEN t1.tp_group1_x
        ELSE t1.tp_group1
        end as tp_group1_x1 
	, CASE
        when 
           (
            row_number() OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period)
            ) = 1
            THEN t1.con_typ1_x
        ELSE t1.con_typ1
        end as con_typ1_x1
	, CASE
        when 
           (
            row_number() OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period)
            ) = 1
            THEN t1.phone_num_x
        ELSE t1.phone_num
        end as phone_num_x1
FROM analyticsb2b_sb.rkahmetov_vr_mob_vrem_4 as t1
""").repartition(620).write.mode("overwrite").format("orc").saveAsTable("analyticsb2b_sb.rkahmetov_vr_mob_vrem_5")

In [ ]:
%%time
spark.sql("""
SELECT
    t1.* 
    , CASE
        when t1.con_n_x1 is NULL 
            then first_value(t1.con_n_x1) over (partition by t1.regid, t1.app_n, t1.part_con_n_x1 order by t1.period)
        else t1.con_n_x1
        end as con_n_x2 
	, CASE
        when t1.kontr_name_x1 is NULL 
            then first_value(t1.kontr_name_x1) over (partition by t1.regid, t1.app_n, t1.part_kontr_name_x1 order by t1.period)
        else t1.kontr_name_x1
        end as kontr_name_x2 
	, CASE
        when t1.region_x1 is NULL 
            then first_value(t1.region_x1) over (partition by t1.regid, t1.app_n, t1.part_region_x1 order by t1.period)
        else t1.region_x1
        end as region_x2 
	, CASE
        when t1.activation_date_x1 is NULL 
            then first_value(t1.activation_date_x1) over (partition by t1.regid, t1.app_n, t1.part_activation_date_x1 order by t1.period)
        else t1.activation_date_x1
        end as activation_date_x2 
	, CASE
        when t1.tariff_plan_x1 is NULL 
            then first_value(t1.tariff_plan_x1) over (partition by t1.regid, t1.app_n, t1.part_tariff_plan_x1 order by t1.period)
        else t1.tariff_plan_x1
        end as tariff_plan_x2 
	, CASE
        when t1.tp_group_x1 is NULL 
            then first_value(t1.tp_group_x1) over (partition by t1.regid, t1.app_n, t1.part_tp_group_x1 order by t1.period)
        else t1.tp_group_x1
        end as tp_group_x2 
	, CASE
        when t1.con_typ_x1 is NULL 
            then first_value(t1.con_typ_x1) over (partition by t1.regid, t1.app_n, t1.part_con_typ_x1 order by t1.period)
        else t1.con_typ_x1
        end as con_typ_x2
	
	
	, CASE
        when t1.tariff_plan1_x1 is NULL 
            then first_value(t1.tariff_plan1_x1) over (partition by t1.regid, t1.app_n, t1.part_tariff_plan1_x1 order by t1.period)
        else t1.tariff_plan1_x1
        end as tariff_plan1_x2 
	, CASE
        when t1.tp_group1_x1 is NULL 
            then first_value(t1.tp_group1_x1) over (partition by t1.regid, t1.app_n, t1.part_tp_group1_x1 order by t1.period)
        else t1.tp_group1_x1
        end as tp_group1_x2 
	, CASE
        when t1.con_typ1_x1 is NULL 
            then first_value(t1.con_typ1_x1) over (partition by t1.regid, t1.app_n, t1.part_con_typ1_x1 order by t1.period)
        else t1.con_typ1_x1
        end as con_typ1_x2
	, CASE
        when t1.phone_num_x1 is NULL 
            then first_value(t1.phone_num_x1) over (partition by t1.regid, t1.app_n, t1.part_con_typ1_x1 order by t1.period)
        else t1.phone_num_x1
        end as phone_num_x2
FROM 
   (
    select 
        a.*
        , sum(case when a.con_n_x1 is null then 0 else 1 end) 
			over (partition by a.regid, a.app_n order by a.period) part_con_n_x1
		, sum(case when a.kontr_name_x1 is null then 0 else 1 end) 
			over (partition by a.regid, a.app_n order by a.period) part_kontr_name_x1
		, sum(case when a.region_x1 is null then 0 else 1 end) 
			over (partition by a.regid, a.app_n order by a.period) part_region_x1
		, sum(case when a.activation_date_x1 is null then 0 else 1 end) 
			over (partition by a.regid, a.app_n order by a.period) part_activation_date_x1
		, sum(case when a.tariff_plan_x1 is null then 0 else 1 end) 
			over (partition by a.regid, a.app_n order by a.period) part_tariff_plan_x1
		, sum(case when a.tp_group_x1 is null then 0 else 1 end) 
			over (partition by a.regid, a.app_n order by a.period) part_tp_group_x1
		, sum(case when a.con_typ_x1 is null then 0 else 1 end) 
			over (partition by a.regid, a.app_n order by a.period) part_con_typ_x1
			
		
		, sum(case when a.tariff_plan1_x1 is null then 0 else 1 end) 
			over (partition by a.regid, a.app_n order by a.period) part_tariff_plan1_x1
		, sum(case when a.tp_group1_x1 is null then 0 else 1 end) 
			over (partition by a.regid, a.app_n order by a.period) part_tp_group1_x1
		, sum(case when a.con_typ1_x1 is null then 0 else 1 end) 
			over (partition by a.regid, a.app_n order by a.period) part_con_typ1_x1
		, sum(case when a.phone_num_x1 is null then 0 else 1 end) 
			over (partition by a.regid, a.app_n order by a.period) part_phone_num_x1
    from analyticsb2b_sb.rkahmetov_vr_mob_vrem_5 as a
    ) as t1
""").repartition(620).write.mode("overwrite").format("orc").saveAsTable("analyticsb2b_sb.rkahmetov_vr_mob_vrem_6")

In [ ]:
%%time
spark.sql("""
select
	t1.period
	,t1.regid
	,t1.app_n
	,t1.phone_num_x2
	,t1.fclc_prochie_mob1
	,t1.fclc_prochie_vas1
	,t1.fclc_golos1
	,t1.fclc_gprs1
	,t1.fclc_sms1
	,t1.fclc_vsr1
	,t1.fclc_vz_mg_mn1
	,t1.fclc_abon_plata1
	,t1.fclc_mob_total1
	,t1.fclc_mob_no_vas1
	,t1.fclc_total1
	,t1.fclc_aff_serv1
	,t1.con_n_x2 
	,t1.kontr_name_x2
	,t1.activation_date_x2
	,t1.region_x2
	,t1.con_typ_x2
	,t1.con_typ1_x2 
	,t1.tariff_plan_x2
	,t1.tariff_plan1_x2 
	,t1.tp_group_x2
	,case 
		when lower(t1.tariff_plan1_x2) in ('москва - комфорт xl - 250 мбит/с плюс тв 062022 (шпд) (eth) (впо) (фс) (scp)', 
		                                   'москва - комфорт xl - 500 мбит/с плюс тв (шпд) (eth) (впо) (фс) (scp)', 
										   'москва - комфорт xl - 500 мбит/с плюс тв 062022 (шпд) (eth) (впо) (фс) (scp)') 
			then 'Конвергент'
		when lower(t1.tariff_plan1_x2) = 'казань фс - базис интернет 500 мбит/с (шпд) (eth) (ск) (фс) (scp)' 
			then 'Интернет безлимитный масс'
		when lower(t1.tariff_plan1_x2) = 'казань фс - пакет услуг' 
			then 'Интернет+ТВ'
		else t1.tp_group1_x2 end as tp_group1_x2_1
from analyticsb2b_sb.rkahmetov_vr_mob_vrem_6 t1
""").repartition(620).write.mode("overwrite").format("orc").saveAsTable("analyticsb2b_sb.rkahmetov_vr_mob_vrem_6_1")

In [ ]:
%%time
spark.sql("""
select 
    t2.period
	,t2.regid
	,t2.app_n
    ,t2.phone_num_x2
	,t2.fclc_prochie_mob1
	,t2.fclc_prochie_vas1
	,t2.fclc_golos1
	,t2.fclc_gprs1
	,t2.fclc_sms1
	,t2.fclc_vsr1
	,t2.fclc_vz_mg_mn1
	,t2.fclc_abon_plata1
	,t2.fclc_mob_total1
	,t2.fclc_mob_no_vas1
	,t2.fclc_total1
	,t2.fclc_aff_serv1
	,t2.con_n_x2 
	,t2.kontr_name_x2
	,t2.activation_date_x2
	,t2.region_x2
	,t3.region_txt as region_txt
	,t2.con_typ_x2
	,t2.con_typ1_x2 
	,t2.tariff_plan_x2
	,t2.tariff_plan1_x2 
	,t2.tp_group_x2
	,t2.tp_group1_x2
	,min(t2.activat_date_x1) OVER (PARTITION BY t2.regid, t2.app_n) as activat_date_x2
	,t4.segment as segment
	,t5.tp_group_big as tp_big
from 
   (
    select
        t1.period
		,t1.regid
		,t1.app_n
        ,t1.phone_num_x2
		,t1.fclc_prochie_mob1
		,t1.fclc_prochie_vas1
		,t1.fclc_golos1
		,t1.fclc_gprs1
		,t1.fclc_sms1
		,t1.fclc_vsr1
		,t1.fclc_vz_mg_mn1
		,t1.fclc_abon_plata1
		,t1.fclc_mob_total1
		,t1.fclc_mob_no_vas1
		,t1.fclc_total1
		,t1.fclc_aff_serv1
		,t1.con_n_x2 
		,t1.kontr_name_x2
		,t1.activation_date_x2
		,t1.region_x2
		,t1.con_typ_x2
		,t1.con_typ1_x2 
		,t1.tariff_plan_x2
		,t1.tariff_plan1_x2 
		,t1.tp_group_x2
		,t1.tp_group1_x2_1 as tp_group1_x2
		,date_format(to_date(t1.activation_date_x2), 'yyyyMM') as activat_date_x1
	from analyticsb2b_sb.rkahmetov_vr_mob_vrem_6_1 t1
    ) t2
left join analyticsb2b_sb.rkahmetov_sprav_regions t3
	on t2.region_x2 = t3.region
left join analyticsb2b_sb.rkahmetov_sprav_segment t4
	on lower(t2.con_typ1_x2) = lower(t4.m_categ)
left join analyticsb2b_sb.rkahmetov_sprav_tp_group_big t5
	on lower(t2.tp_group1_x2) = lower(t5.tp_group)
""").repartition(620).write.mode("overwrite").format("orc").saveAsTable("analyticsb2b_sb.rkahmetov_vr_mob_vrem_7")

In [ ]:
%%time
spark.sql("""
select
	t2.*
	, case 
        when (COALESCE(t2.fclc_mob_total2, 0)) > 0 
            then 1
            else 0
        end as activnost_mob_total
    , case 
        when (COALESCE(t2.fclc_mob_no_vas2, 0)) > 0 
            then 1
            else 0
        end as activnost_mob_no_vas
from 
   (
	select 
    t1.period
	,t1.regid
	,t1.app_n
	,t1.fclc_total1 as fclc_total2
	,t1.phone_num_x2
	,t1.con_n_x2 
	,t1.kontr_name_x2
	,t1.activation_date_x2
	,t1.region_x2
	,t1.region_txt
	,t1.con_typ_x2
	,t1.con_typ1_x2 
	,t1.tariff_plan_x2
	,t1.tariff_plan1_x2 
	,t1.tp_group_x2
	,t1.tp_group1_x2
	,t1.activat_date_x2
    ,case when t1.segment is null then 'B2C' else t1.segment end as segment1
	,t1.tp_big
    ,CASE 
        WHEN t1.activat_date_x2 = t1.period 
            then 1 
            else 0 
        end as activation_marker
    ,LAG(t1.tariff_plan_x2) OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period) as tariff_plan_x2_prev
    ,LAG(t1.tariff_plan1_x2) OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period) as tariff_plan1_x2_prev
	,CASE 
        WHEN t1.tariff_plan1_x2 <> LAG(t1.tariff_plan1_x2) OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period) 
            THEN 1 
            else 0 
        end as tariff_mgr
    ,LAG(t1.tp_group_x2) OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period) as tp_group_x2_prev
	,LAG(t1.tp_group1_x2) OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period) as tp_group1_x2_prev
    ,CASE 
        WHEN t1.tp_group1_x2 <> LAG(t1.tp_group1_x2) OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period) 
            THEN 1 
            else 0 
        end as tp_group_mgr
    ,LAG(t1.tp_big) OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period) as tp_big_prev
    ,CASE 
        WHEN t1.tp_big <> LAG(t1.tp_big) OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period) 
            THEN 1 
            else 0 
        end as tp_big_mgr
    ,LAG(t1.con_typ_x2) OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period) as con_typ_x2_prev
	,LAG(t1.con_typ1_x2) OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period) as con_typ1_x2_prev
    ,CASE 
        WHEN t1.con_typ1_x2 <> LAG(t1.con_typ1_x2) OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period) 
            THEN 1 
            else 0 
        end as categ_mgr
    ,LAG(t1.segment) OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period) as segment_prev
    ,CASE 
        WHEN t1.segment <> LAG(t1.segment) OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period) 
            THEN 1 
            else 0 
        end as segment_mgr
    ,case 
        when t1.segment in ('B2C', 'Не в B2C') then null
        else t1.fclc_prochie_mob1 end as fclc_prochie_mob2
    ,case 
        when t1.segment in ('B2C', 'Не в B2C') then null
        else t1.fclc_prochie_vas1 end as fclc_prochie_vas2
    ,case 
        when t1.segment in ('B2C', 'Не в B2C') then null
        else t1.fclc_golos1 end as fclc_golos2
    ,case 
        when t1.segment in ('B2C', 'Не в B2C') then null
        else t1.fclc_gprs1 end as fclc_gprs2
    ,case 
        when t1.segment in ('B2C', 'Не в B2C') then null
        else t1.fclc_sms1 end as fclc_sms2
    ,case 
        when t1.segment in ('B2C', 'Не в B2C') then null
        else t1.fclc_vsr1 end as fclc_vsr2
    ,case 
        when t1.segment in ('B2C', 'Не в B2C') then null
        else t1.fclc_vz_mg_mn1 end as fclc_vz_mg_mn2
    ,case 
        when t1.segment in ('B2C', 'Не в B2C') then null
        else t1.fclc_abon_plata1 end as fclc_abon_plata2
    ,case 
        when t1.segment in ('B2C', 'Не в B2C') then null
        else t1.fclc_mob_total1 end as fclc_mob_total2
    ,case 
        when t1.segment in ('B2C', 'Не в B2C') then null
        else t1.fclc_mob_no_vas1 end as fclc_mob_no_vas2
	,case 
        when t1.segment in ('B2C', 'Не в B2C') then null
        else t1.fclc_aff_serv1 end as fclc_aff_serv2
	from analyticsb2b_sb.rkahmetov_vr_mob_vrem_7 t1
	) t2
""").repartition(550).write.mode("overwrite").format("orc").saveAsTable("analyticsb2b_sb.rkahmetov_vr_mob_vrem_8")

In [ ]:
%%time
spark.sql("""
select distinct 
    t1.regid 
, t1.app_n 
, 'po_B2B' as is_B2B
FROM analyticsb2b_sb.rkahmetov_vr_mob_vrem_8 as t1 
where t1.segment1 not in ('B2C', 'Не в B2C')
""").repartition(15).write.mode("overwrite").format("orc").saveAsTable("analyticsb2b_sb.rkahmetov_vr_vrem_po_b2b")

In [ ]:
%%time
spark.sql("""
select 
    t1.period
	,t1.regid
	,t1.app_n
	,t1.phone_num_x2
	,t1.con_n_x2
	,t3.inn as inn1	
	,t1.kontr_name_x2
	,t1.activation_date_x2
	,case 
        when t1.activat_date_x2 is NULL
            then '197001'
            else t1.activat_date_x2
		end as activat_date_x3
	,t1.activation_marker
	,t1.region_x2
	,t1.region_txt
	
	,t1.tariff_mgr
	,case 
		when (row_number() OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period)) = 1 
			and t1.tariff_plan_x2_prev is NULL 
            then t1.tariff_plan_x2
		else t1.tariff_plan_x2_prev 
		end as tariff_plan_x3_prev
	,t1.tariff_plan_x2
	,case 
		when (row_number() OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period)) = 1 
			and t1.tariff_plan1_x2_prev is NULL 
            then t1.tariff_plan1_x2
		else t1.tariff_plan1_x2_prev 
		end as tariff_plan1_x3_prev
	,t1.tariff_plan1_x2
	
	,t1.tp_group_mgr
	,case 
		when (row_number() OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period)) = 1 
			and t1.tp_group_x2_prev is NULL 
            then t1.tp_group_x2
		else t1.tp_group_x2_prev 
		end as tp_group_x3_prev
	,t1.tp_group_x2
	,case 
		when (row_number() OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period)) = 1 
			and t1.tp_group1_x2_prev is NULL 
            then t1.tp_group1_x2
		else t1.tp_group1_x2_prev 
		end as tp_group1_x3_prev
	,t1.tp_group1_x2
	
	,t1.tp_big_mgr
	,case 
		when (row_number() OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period)) = 1 
			and t1.tp_big_prev is NULL 
            then t1.tp_big
		else t1.tp_big_prev 
		end as tp_big_prev1
	,t1.tp_big
		
	,t1.categ_mgr
	,case 
		when (row_number() OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period)) = 1 
			and t1.con_typ_x2_prev is NULL 
            then t1.con_typ_x2
		else t1.con_typ_x2_prev 
		end as con_typ_x3_prev
	,t1.con_typ_x2
	,case 
		when (row_number() OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period)) = 1 
			and t1.con_typ1_x2_prev is NULL 
            then t1.con_typ1_x2
		else t1.con_typ1_x2_prev 
		end as con_typ1_x3_prev
	,t1.con_typ1_x2
	
	,t1.segment_mgr
	,case 
		when (row_number() OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period)) = 1 
			and t1.segment_prev is NULL 
            then t1.segment1
		else t1.segment_prev 
		end as segment_prev1
	,t1.segment1 as segment
	
	,t1.fclc_prochie_mob2
    ,t1.fclc_prochie_vas2
    ,t1.fclc_golos2
    ,t1.fclc_gprs2
    ,t1.fclc_sms2
    ,t1.fclc_vsr2
    ,t1.fclc_vz_mg_mn2
    ,t1.fclc_abon_plata2
    ,t1.fclc_mob_total2
    ,t1.fclc_mob_no_vas2
	,t1.fclc_total2
	,t1.fclc_aff_serv2
	,t1.activnost_mob_total
	,t1.activnost_mob_no_vas
	,t2.is_B2B as is_B2B
	,case 
        WHEN COALESCE(t1.fclc_mob_total2, 0) = 0 
			then '0'
        WHEN COALESCE(t1.fclc_mob_total2, 0) > 0
			and COALESCE(t1.fclc_mob_total2, 0) < 50 
			then '0_50'
        WHEN COALESCE(t1.fclc_mob_total2, 0) >= 50
			and COALESCE(t1.fclc_mob_total2, 0) < 100 
			then '50_100'
        WHEN COALESCE(t1.fclc_mob_total2, 0) >= 100
			and COALESCE(t1.fclc_mob_total2, 0) < 250 
			then '100_250'
        WHEN COALESCE(t1.fclc_mob_total2, 0) >= 250
			and COALESCE(t1.fclc_mob_total2, 0) < 500 
			then '250_500'
        WHEN COALESCE(t1.fclc_mob_total2, 0) >= 500 
			then '500_i_bolshe'
		END AS ARPU_categ_mob_total
    ,case 
        WHEN COALESCE(t1.fclc_mob_no_vas2, 0) = 0 
			then '0'
        WHEN COALESCE(t1.fclc_mob_no_vas2, 0) > 0
			and COALESCE(t1.fclc_mob_no_vas2, 0) < 50 
			then '0_50'
        WHEN COALESCE(t1.fclc_mob_no_vas2, 0) >= 50
			and COALESCE(t1.fclc_mob_no_vas2, 0) < 100 
			then '50_100'
        WHEN COALESCE(t1.fclc_mob_no_vas2, 0) >= 100
			and COALESCE(t1.fclc_mob_no_vas2, 0) < 250 
			then '100_250'
        WHEN COALESCE(t1.fclc_mob_no_vas2, 0) >= 250
			and COALESCE(t1.fclc_mob_no_vas2, 0) < 500 
			then '250_500'
        WHEN COALESCE(t1.fclc_mob_no_vas2, 0) >= 500 
			then '500_i_bolshe'
		END AS ARPU_categ_mob_no_vas
from analyticsb2b_sb.rkahmetov_vr_mob_vrem_8 t1
left join analyticsb2b_sb.rkahmetov_vr_vrem_po_b2b t2
    on t1.regid = t2.regid 
    and t1.app_n = t2.app_n
LEFT JOIN 
       (
        SELECT DISTINCT 
            a3.regid
            , a3.con_n
            , a3.inn 
        FROM raw.mtsru_tdwh_smr_tdprd__uat_v_basefull_drkk_con_inn as a3 
        WHERE a3.is_closed = 0
            and a3.con_status = 1
        ) t3
    ON t1.regid = t3.regid
    and t1.con_n_x2 = t3.con_n
""").repartition(550).write.mode("overwrite").format("orc").saveAsTable("analyticsb2b_sb.rkahmetov_vr_mob_vrem_9")

In [ ]:
%%time
spark.sql("""
select 
    t1.period
	,t1.regid
	,t1.app_n
	,t1.phone_num_x2
	,t1.con_n_x2
	,case 
        when t1.inn1 is null 
			or t1.segment in ('B2C', 'Не в B2C') then t1.con_n_x2
            else t1.inn1
        end as inn2
	,t1.kontr_name_x2
	,t1.activation_date_x2
	,t1.activat_date_x3
	,t1.activation_marker
	,t1.region_x2
	,t1.region_txt
	
	,t1.tariff_mgr
	,case 
        when t1.activation_marker = 1 
            then t1.tariff_plan_x2
            else t1.tariff_plan_x3_prev
        end tariff_plan_x4_prev
	,t1.tariff_plan_x2
	,case 
        when t1.activation_marker = 1 
            then t1.tariff_plan1_x2
            else t1.tariff_plan1_x3_prev
        end tariff_plan1_x4_prev
	,t1.tariff_plan1_x2
	
	,t1.tp_group_mgr
	,case 
        when t1.activation_marker = 1 
            then t1.tp_group_x2
            else t1.tp_group_x3_prev
        end tp_group_x4_prev
	,t1.tp_group_x2
	,case 
        when t1.activation_marker = 1 
            then t1.tp_group1_x2
            else t1.tp_group1_x3_prev
        end tp_group1_x4_prev
	,t1.tp_group1_x2
	
	,t1.tp_big_mgr
	,case 
        when t1.activation_marker = 1 
            then t1.tp_big
            else t1.tp_big_prev1
        end tp_big_prev2
	,t1.tp_big
		
	,t1.categ_mgr
	,case 
        when t1.activation_marker = 1 
            then t1.con_typ_x2
            else t1.con_typ_x3_prev
        end con_typ_x4_prev
	,t1.con_typ_x2
	,case 
        when t1.activation_marker = 1 
            then t1.con_typ1_x2
            else t1.con_typ1_x3_prev
        end con_typ1_x4_prev
	,t1.con_typ1_x2
	
	,t1.segment_mgr
	,case 
        when t1.activation_marker = 1 
            then t1.segment
            else t1.segment_prev1
        end segment_prev2
	,t1.segment
	
	,t1.fclc_prochie_mob2
    ,t1.fclc_prochie_vas2
    ,t1.fclc_golos2
    ,t1.fclc_gprs2
    ,t1.fclc_sms2
    ,t1.fclc_vsr2
    ,t1.fclc_vz_mg_mn2
    ,t1.fclc_abon_plata2
    ,t1.fclc_mob_total2
    ,t1.fclc_mob_no_vas2
	,t1.fclc_total2
	,t1.fclc_aff_serv2
	,t1.activnost_mob_total
	,t1.activnost_mob_no_vas
	,t1.is_B2B
	,t1.ARPU_categ_mob_total
    ,t1.ARPU_categ_mob_no_vas
from analyticsb2b_sb.rkahmetov_vr_mob_vrem_9 t1
""").repartition(550).write.mode("overwrite").format("orc").saveAsTable("analyticsb2b_sb.rkahmetov_vr_mob_vrem_10")

In [ ]:
%%time
spark.sql("""
select 
t2.*
, t3.label 
from 
   (
    select 
        t1.regid
        , t1.app_n
        , date_format(to_date(t1.date_from), 'yyyyMM') as period
        , date_format(to_date(t1.date_to), 'yyyyMM') as period_x
        , t1.code
    FROM raw.mtsru_tdwh_smr_tdprd__uat_v_basefull_dapp_sblo t1
    where t1.regid||'-'||t1.app_n in (select distinct a.regid||'-'||a.app_n from analyticsb2b_sb.rkahmetov_vr_mob_vrem_10 a)
    ) t2
left join 
   (
    select 
        c.start
        , c.label
    from
       (
        select distinct
            b.start
            , b.label
            , rank()Over(PARTITION BY b.start ORDER BY b.version_dt DESC) as rer1
        from raw.mtsru_sdwh_smr__mts_fmt_formats2_f67c b
        ) c
    where c.rer1 = 1
    ) t3
on t2.code = t3.start
where lower(t3.label) like '%заключит%'
""").repartition(5).write.mode("overwrite").format("orc").saveAsTable("analyticsb2b_sb.rkahmetov_vr_mob_vrem_block")

In [ ]:
%%time
spark.sql("""
select 
a.*
from
(select
t1.*
, case when t1.period = t1.period_x then 1 else 0 end as mark_isk
from analyticsb2b_sb.rkahmetov_vr_mob_vrem_block t1) a
where a.mark_isk = 0
""").repartition(3).write.mode("overwrite").format("orc").saveAsTable("analyticsb2b_sb.rkahmetov_vr_mob_vrem_block_1")

In [ ]:
%%time
spark.sql("""
select 
    t1.period
	,t1.regid
	,t1.app_n
    ,t1.phone_num_x2
	,t1.con_n_x2
	,t1.inn2
	,t1.kontr_name_x2
	,t1.activation_date_x2
	,t1.activat_date_x3
	,t1.activation_marker
	,t1.region_x2
	,t1.region_txt
	
	,t1.tariff_mgr
	,t1.tariff_plan_x4_prev
	,t1.tariff_plan_x2
	,t1.tariff_plan1_x4_prev
	,t1.tariff_plan1_x2
	
	,t1.tp_group_mgr
	,t1.tp_group_x4_prev
	,t1.tp_group_x2
	,t1.tp_group1_x4_prev
	,t1.tp_group1_x2
	
	,t1.tp_big_mgr
	,t1.tp_big_prev2
	,t1.tp_big
		
	,t1.categ_mgr
	,t1.con_typ_x4_prev
	,t1.con_typ_x2
	,t1.con_typ1_x4_prev
	,t1.con_typ1_x2
	
	,t1.segment_mgr
	,t1.segment_prev2
	,t1.segment
	,CASE 
        WHEN t1.segment <> t1.segment_prev2 AND (COALESCE(t1.segment_mgr, 0)) = 1
            THEN t1.segment_prev2||'-'||t1.segment
        END AS smena_segment
	
	,t1.fclc_prochie_mob2
    ,t1.fclc_prochie_vas2
    ,t1.fclc_golos2
    ,t1.fclc_gprs2
    ,t1.fclc_sms2
    ,t1.fclc_vsr2
    ,t1.fclc_vz_mg_mn2
    ,t1.fclc_abon_plata2
    ,t1.fclc_mob_total2
    ,t1.fclc_mob_no_vas2
	,t1.fclc_total2
	,t1.fclc_aff_serv2
	,t1.activnost_mob_total
	,t1.activnost_mob_no_vas
	,t1.is_B2B
	,t1.ARPU_categ_mob_total
    ,t1.ARPU_categ_mob_no_vas
	,t2.period as zak_block1
	,case 
		when COALESCE(t1.fclc_mob_total2, 0) - COALESCE(t1.fclc_aff_serv2, 0) = 0 and COALESCE(t1.fclc_mob_total2, 0) > 0 then 1
		else 0 
		end as metk_aff_ser
from analyticsb2b_sb.rkahmetov_vr_mob_vrem_10 t1
left join (select distinct a.regid, a.app_n, a.period from analyticsb2b_sb.rkahmetov_vr_mob_vrem_block_1 a) t2
	on t1.regid = t2.regid 
	and t1.app_n = t2.app_n 
	and t1.period = t2.period
where t1.is_B2B = 'po_B2B'
""").repartition(550).write.mode("overwrite").format("orc").saveAsTable("analyticsb2b_sb.rkahmetov_vr_mob_vrem_11")

In [ ]:
%%time
spark.sql("""
SELECT
    b.* 
    , CASE
        when b.metk_zak_block1 is NULL 
            then first_value(b.metk_zak_block1) over (partition by b.regid, b.app_n, b.part_metk_block1 order by b.period)
        else b.metk_zak_block1
        end as mark_zak_block 
	, CASE
        when b.zak_block1 is NULL 
            then first_value(b.zak_block1) over (partition by b.regid, b.app_n, b.part_month_block1 order by b.period)
        else b.zak_block1
        end as month_zak_block 
from 
(select 
        a.*
        , sum(case when a.metk_zak_block1 is null then 0 else 1 end) 
			over (partition by a.regid, a.app_n order by a.period) part_metk_block1
		, sum(case when a.zak_block1 is null then 0 else 1 end) 
			over (partition by a.regid, a.app_n order by a.period) part_month_block1
from 			
(select
	t1.period
	,t1.regid
	,t1.app_n
    ,t1.phone_num_x2
	,t1.con_n_x2
	,t1.inn2
	,case 
		when t1.con_n_x2 = t1.inn2 then 'con_n'
		else 'inn' end as is_con_inn
	,t1.kontr_name_x2
	,t1.activation_date_x2
	,t1.activat_date_x3
	,t1.activation_marker
	,t1.region_x2
	,t1.region_txt
	,t2.Reg_Week2 as Reg_Week2
	,t2.TR as TR
	
	,t1.tariff_mgr
	,t1.tariff_plan_x4_prev
	,t1.tariff_plan_x2
	,t1.tariff_plan1_x4_prev
	,t1.tariff_plan1_x2
	
	,t1.tp_group_mgr
	,t1.tp_group_x4_prev
	,t1.tp_group_x2
	,t1.tp_group1_x4_prev
	,t1.tp_group1_x2
	
	,t1.tp_big_mgr
	,t1.tp_big_prev2
	,t1.tp_big
		
	,t1.categ_mgr
	,t1.con_typ_x4_prev
	,t1.con_typ_x2
	,t1.con_typ1_x4_prev
	,t1.con_typ1_x2
	
	,t1.segment_mgr
	,t1.segment_prev2
	,t1.segment
	,t1.smena_segment
	
	,t1.fclc_prochie_mob2
    ,t1.fclc_prochie_vas2
    ,t1.fclc_golos2
    ,t1.fclc_gprs2
    ,t1.fclc_sms2
    ,t1.fclc_vsr2
    ,t1.fclc_vz_mg_mn2
    ,t1.fclc_abon_plata2
    ,t1.fclc_mob_total2
    ,t1.fclc_mob_no_vas2
	,t1.fclc_total2
	,t1.fclc_aff_serv2
	,t1.activnost_mob_total
	,t1.activnost_mob_no_vas
	,t1.is_B2B
	,t1.ARPU_categ_mob_total
    ,t1.ARPU_categ_mob_no_vas
	,t1.zak_block1
	,t1.metk_aff_ser
	,case 
		when t1.zak_block1 is not null then 1
		else null
		end as metk_zak_block1
from analyticsb2b_sb.rkahmetov_vr_mob_vrem_11 t1
left join analyticsb2b_sb.rkahmetov_sprav_tr t2
	on lower(t1.region_txt) = lower(t2.region)
) a
) b
""").repartition(412).write.mode("overwrite").format("orc").saveAsTable("analyticsb2b_sb.rkahmetov_vr_mob_vrem_12")

In [ ]:
%%time
spark.sql("""
select
	t1.period
	,t1.regid
	,t1.app_n
    ,t1.phone_num_x2 as phone_num
	,t1.con_n_x2 as con_n
	,t1.inn2 as inn
	,t1.is_con_inn
	,t1.kontr_name_x2 as kontr_name
	,t1.activation_date_x2 as abonent_activation_date
	,t1.activat_date_x3 as abonent_activation_month
	,t1.activation_marker
	,t1.region_x2 as region
	,t1.region_txt
	,t1.Reg_Week2
	,t1.TR
	
	,t1.tariff_mgr
	,t1.tariff_plan_x4_prev as tariff_id_prev
	,t1.tariff_plan_x2 as tariff_id
	,t1.tariff_plan1_x4_prev as tariff_prev
	,t1.tariff_plan1_x2 as tariff
	
	,t1.tp_group_mgr
	,t1.tp_group_x4_prev as tp_group_id_prev
	,t1.tp_group_x2 as tp_group_id
	,t1.tp_group1_x4_prev as tp_group_prev
	,t1.tp_group1_x2 as tp_group
	
	,t1.tp_big_mgr
	,t1.tp_big_prev2 as tp_big_prev
	,t1.tp_big
		
	,t1.categ_mgr as m_categ_mgr
	,t1.con_typ_x4_prev as m_categ_id_prev
	,t1.con_typ_x2 as m_categ_id
	,t1.con_typ1_x4_prev as m_categ_prev
	,t1.con_typ1_x2 as m_categ
	
	,t1.segment_mgr
	,t1.segment_prev2 as segment_prev
	,t1.segment
	,t1.smena_segment as smena_segmenta
	
	,t1.fclc_prochie_mob2 as fclc_prochie_mob
    ,t1.fclc_prochie_vas2 as fclc_prochie_vas
    ,t1.fclc_golos2 as fclc_golos
    ,t1.fclc_gprs2 as fclc_gprs
    ,t1.fclc_sms2 as fclc_sms
    ,t1.fclc_vsr2 as fclc_vsr
    ,t1.fclc_vz_mg_mn2 as fclc_vz_mg_mn
    ,t1.fclc_abon_plata2 as fclc_abon_plata
    ,t1.fclc_mob_total2 as fclc_mob_total
    ,t1.fclc_mob_no_vas2 as fclc_mob_no_vas
	,t1.fclc_total2 as fclc_total
	,t1.fclc_aff_serv2 as fclc_aff_serv
	,t1.activnost_mob_total
	,t1.activnost_mob_no_vas
	,t1.ARPU_categ_mob_total as abonent_ARPU_categ_mob_total
    ,t1.ARPU_categ_mob_no_vas as abonent_ARPU_categ_mob_no_vas
	,t1.mark_zak_block
	,t1.month_zak_block
	,t1.metk_aff_ser
from analyticsb2b_sb.rkahmetov_vr_mob_vrem_12 t1
""").repartition(550).write.mode("overwrite").format("orc").saveAsTable("analyticsb2b_sb.rkahmetov_vr_mob_vrem_13")

In [ ]:
%%time
spark.sql("""
select
	t1.period
	,t1.regid
	,t1.app_n
    ,t1.phone_num
	,t1.con_n
	,t1.inn
	,t1.is_con_inn
	,t1.kontr_name
	,t1.abonent_activation_date
	,t1.abonent_activation_month
	,t1.activation_marker
	,t1.region
	,t1.region_txt
	,t1.Reg_Week2
	,t1.TR
	
	,t1.tariff_mgr
	,t1.tariff_id_prev
	,t1.tariff_id
	,t1.tariff_prev
	,t1.tariff
	
	,t1.tp_group_mgr
	,t1.tp_group_id_prev
	,t1.tp_group_id
	,t1.tp_group_prev
	,t1.tp_group
	
	,t1.tp_big_mgr
	,t1.tp_big_prev
	,t1.tp_big
		
	,t1.m_categ_mgr
	,t1.m_categ_id_prev
	,t1.m_categ_id
	,t1.m_categ_prev
	,t1.m_categ
	
	,t1.segment_mgr
	,t1.segment_prev
	,t1.segment
	,t1.smena_segmenta
	
	,COALESCE(t1.fclc_prochie_mob,0) as fclc_prochie_mob1
	,COALESCE(t1.fclc_prochie_vas,0) as fclc_prochie_vas1
	,COALESCE(t1.fclc_golos,0) as fclc_golos1
	,COALESCE(t1.fclc_gprs,0) as fclc_gprs1
	,COALESCE(t1.fclc_sms,0) as fclc_sms1
	,COALESCE(t1.fclc_vsr,0) as fclc_vsr1
	,COALESCE(t1.fclc_vz_mg_mn,0) as fclc_vz_mg_mn1
	,COALESCE(t1.fclc_abon_plata,0) as fclc_abon_plata1
	,COALESCE(t1.fclc_mob_total,0) as fclc_mob_total1
	,COALESCE(t1.fclc_mob_no_vas,0) as fclc_mob_no_vas1
	,COALESCE(t1.fclc_total,0) as fclc_total1
	,COALESCE(t1.fclc_aff_serv,0) as fclc_aff_serv1
	
	,CASE
		when COALESCE(t1.fclc_mob_total,0)=0 then NULL
		else t1.fclc_mob_total end as fclc_mob_total_x
	,CASE
		when COALESCE(t1.fclc_mob_no_vas,0)=0 then NULL
		else t1.fclc_mob_no_vas end as fclc_mob_no_vas_x
	,CASE
		when COALESCE(t1.fclc_total,0)=0 then NULL
		else t1.fclc_total end as fclc_total_x
	
	,t1.activnost_mob_total
	,t1.activnost_mob_no_vas
	
	,t1.abonent_ARPU_categ_mob_total
    ,t1.abonent_ARPU_categ_mob_no_vas
	
	,t1.mark_zak_block
	,t1.month_zak_block
	
	,t1.metk_aff_ser
	
--1 Приток моб тотал
	,case 
		when t1.period < t1.abonent_activation_month
			and t1.segment IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
			and t1.tp_big<>'M2M/IOT' 
			and coalesce(t1.fclc_mob_total,0)>0 
		then 'Активный без активации'


		when coalesce(t1.fclc_mob_total,0)>0 
			and t1.activation_marker=1 and tp_big<>'M2M/IOT' 
			and t1.segment IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
		then 'Продажа'

		
		when coalesce(t1.fclc_mob_total,0)>0 
			and t1.tp_big_prev='M2M/IOT' 
			and t1.tp_big<>'M2M/IOT' 
			and t1.segment IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
			and LAG(COALESCE(t1.fclc_total,0)) OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period)<>0 
		then 'Миграция из ТП IOT'
	
	
		when coalesce(t1.fclc_mob_total,0)>0 
			and t1.tp_big<>'M2M/IOT' 
			and t1.segment_prev IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
			and t1.segment IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
			'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
			and LAG(COALESCE(t1.fclc_total,0)) OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period)<>0
			and LAG(COALESCE(t1.fclc_mob_total,0)) OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period)=0
		then 'Миграция из др. бизнеса'

	
		when coalesce(t1.fclc_mob_total,0)>0 
			and t1.tp_big<>'M2M/IOT' 
			and t1.segment IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
			and LAG(COALESCE(t1.fclc_total,0)) OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period)=0 
		then 'Реактивация'

		
		when coalesce(t1.fclc_mob_total,0)>0 
			and t1.tp_big<>'M2M/IOT' 
			and t1.segment IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
			and t1.segment_mgr=1 
		then 'Миграция в сегмент'
	
		
		when coalesce(t1.fclc_mob_total,0)>0 
			and t1.tp_big<>'M2M/IOT' 
			and t1.segment IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
			AND COALESCE(t1.fclc_mob_total,0)>= 
				(LAG(COALESCE(t1.fclc_mob_total,0)) OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period)*1.15)
		then 'Развитие'
		
		
		when coalesce(t1.fclc_mob_total,0)>0 
			and t1.tp_big<>'M2M/IOT' 
			and t1.segment IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
			AND COALESCE(t1.fclc_mob_total,0)> 
				(LAG(COALESCE(t1.fclc_mob_total,0)) OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period)*0.85)
			and COALESCE(t1.fclc_mob_total,0)< 
				(LAG(COALESCE(t1.fclc_mob_total,0)) OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period)*1.15)
		then 'Стабильно платят'
	
		
		when coalesce(t1.fclc_mob_total,0)>0 
			and t1.tp_big<>'M2M/IOT' 
			and t1.segment IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг')
			AND COALESCE(t1.fclc_mob_total,0)<=
				(LAG(COALESCE(t1.fclc_mob_total,0)) OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period)*0.85)
		then 'Оптимизация' 
	
	
	else 'Пусто'
	end as pritok
		
		
--2 Приток моб без вас
	,case 
		when t1.period < t1.abonent_activation_month
			and t1.segment IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
			and t1.tp_big<>'M2M/IOT' 
			and coalesce(t1.fclc_mob_no_vas,0)>0 
		then 'Активный без активации'
	
	
		when coalesce(t1.fclc_mob_no_vas,0)>0 
			and t1.activation_marker=1 and tp_big<>'M2M/IOT' 
			and t1.segment IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
		then 'Продажа'

		
		when coalesce(t1.fclc_mob_no_vas,0)>0 
			and t1.tp_big_prev='M2M/IOT' 
			and t1.tp_big<>'M2M/IOT' 
			and t1.segment IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
			and LAG(COALESCE(t1.fclc_total,0)) OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period)<>0 
		then 'Миграция из ТП IOT'
	
	
		when coalesce(t1.fclc_mob_no_vas,0)>0 
			and t1.tp_big<>'M2M/IOT' 
			and t1.segment_prev IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
			and t1.segment IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
			and LAG(COALESCE(t1.fclc_total,0)) OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period)<>0
			and LAG(COALESCE(t1.fclc_mob_no_vas,0)) OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period)=0
		then 'Миграция из др. бизнеса'

	
		when coalesce(t1.fclc_mob_no_vas,0)>0 
			and t1.tp_big<>'M2M/IOT' 
			and t1.segment IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
			and LAG(COALESCE(t1.fclc_total,0)) OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period)=0 
		then 'Реактивация'

		
		when coalesce(t1.fclc_mob_no_vas,0)>0 
			and t1.tp_big<>'M2M/IOT' 
			and t1.segment IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
			and t1.segment_mgr=1 
		then 'Миграция в сегмент'
	
		
		when coalesce(t1.fclc_mob_no_vas,0)>0 
			and t1.tp_big<>'M2M/IOT' 
			and t1.segment IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
			AND COALESCE(t1.fclc_mob_no_vas,0)>= 
				(LAG(COALESCE(t1.fclc_mob_no_vas,0)) OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period)*1.15)
		then 'Развитие'
		
		
		when coalesce(t1.fclc_mob_no_vas,0)>0 
			and t1.tp_big<>'M2M/IOT' 
			and t1.segment IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
			AND COALESCE(t1.fclc_mob_no_vas,0)> 
				(LAG(COALESCE(t1.fclc_mob_no_vas,0)) OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period)*0.85)
			and COALESCE(t1.fclc_mob_no_vas,0)< 
				(LAG(COALESCE(t1.fclc_mob_no_vas,0)) OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period)*1.15)
		then 'Стабильно платят'
	
		
		when coalesce(t1.fclc_mob_no_vas,0)>0 
			and t1.tp_big<>'M2M/IOT' 
			and t1.segment IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг')
			AND COALESCE(t1.fclc_mob_no_vas,0)<=
				(LAG(COALESCE(t1.fclc_mob_no_vas,0)) OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period)*0.85)
		then 'Оптимизация' 
	
	
	else 'Пусто'
	end as pritok_no_vas		
		
		
--3 Приток моб тотал Общ		
	,case
		when t1.period < t1.abonent_activation_month
			and t1.segment IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
			and t1.tp_big<>'M2M/IOT' 
			and coalesce(t1.fclc_mob_total,0)>0 
		then 'Активный без активации'
	
	
		when coalesce(t1.fclc_mob_total,0)>0 
			and t1.activation_marker=1 
			and t1.tp_big<>'M2M/IOT' 
			and t1.segment IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
		then 'Продажа'


		when coalesce(t1.fclc_mob_total,0)>0 
			and t1.tp_big_prev='M2M/IOT' 
			and t1.tp_big<>'M2M/IOT' 
			and t1.segment IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
			and LAG(COALESCE(t1.fclc_total,0)) OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period)<>0 
		then 'Миграция из ТП IOT'


		when coalesce(t1.fclc_mob_total,0)>0 
			and t1.tp_big<>'M2M/IOT' 
			and t1.segment_prev IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
			and t1.segment IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг')  
			and LAG(COALESCE(t1.fclc_total,0)) OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period)<>0
			and LAG(COALESCE(t1.fclc_mob_total,0)) OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period)=0
		then 'Миграция из др. бизнеса'


		when coalesce(fclc_mob_total,0)>0 
			and t1.tp_big<>'M2M/IOT' 
			and t1.segment IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
			and LAG(COALESCE(t1.fclc_total,0)) OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period)=0 
		then 'Реактивация'


		when coalesce(t1.fclc_mob_total,0)>0 
			and t1.tp_big<>'M2M/IOT' 
			and t1.segment IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
			and t1.segment_prev NOT IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
			and LAG(COALESCE(t1.fclc_total,0)) OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period)<>0
			and LAG(COALESCE(t1.fclc_mob_total,0)) OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period)=0
		then 'Миграция в B2B'


		when coalesce(t1.fclc_mob_total,0)>0 
			and t1.tp_big<>'M2M/IOT' 
			and t1.segment IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
			AND COALESCE(t1.fclc_mob_total,0)>= 
				(LAG(COALESCE(t1.fclc_mob_total,0)) OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period)*1.15)
		then 'Развитие'


		when coalesce(t1.fclc_mob_total,0)>0 
			and t1.tp_big<>'M2M/IOT' 
			and t1.segment IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
			AND COALESCE(t1.fclc_mob_total,0)> 
				(LAG(COALESCE(t1.fclc_mob_total,0)) OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period)*0.85)
			and COALESCE(t1.fclc_mob_total,0)< 
				(LAG(COALESCE(t1.fclc_mob_total,0)) OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period)*1.15)
		then 'Стабильно платят'


		when coalesce(t1.fclc_mob_total,0)>0 
			and t1.tp_big<>'M2M/IOT' 
			and t1.segment IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
			AND COALESCE(t1.fclc_mob_total,0)<=
				(LAG(COALESCE(t1.fclc_mob_total,0)) OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period)*0.85)
		then 'Оптимизация' 


	else 'Пусто'
	end as pritok_all
	
	
--4 Приток моб без вас Общ
	,case
		when t1.period < t1.abonent_activation_month
			and t1.segment IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
			and t1.tp_big<>'M2M/IOT' 
			and coalesce(t1.fclc_mob_no_vas,0)>0 
		then 'Активный без активации'
	
	
		when coalesce(t1.fclc_mob_no_vas,0)>0 
			and t1.activation_marker=1 
			and t1.tp_big<>'M2M/IOT' 
			and t1.segment IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
		then 'Продажа'


		when coalesce(t1.fclc_mob_no_vas,0)>0 
			and t1.tp_big_prev='M2M/IOT' 
			and t1.tp_big<>'M2M/IOT' 
			and t1.segment IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
			and LAG(COALESCE(t1.fclc_total,0)) OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period)<>0 
		then 'Миграция из ТП IOT'


		when coalesce(t1.fclc_mob_no_vas,0)>0 
			and t1.tp_big<>'M2M/IOT' 
			and t1.segment_prev IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
			and t1.segment IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг')  
			and LAG(COALESCE(t1.fclc_total,0)) OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period)<>0
			and LAG(COALESCE(t1.fclc_mob_no_vas,0)) OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period)=0
		then 'Миграция из др. бизнеса'


		when coalesce(fclc_mob_no_vas,0)>0 
			and t1.tp_big<>'M2M/IOT' 
			and t1.segment IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
			and LAG(COALESCE(t1.fclc_total,0)) OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period)=0 
		then 'Реактивация'


		when coalesce(t1.fclc_mob_no_vas,0)>0 
			and t1.tp_big<>'M2M/IOT' 
			and t1.segment IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
			and t1.segment_prev NOT IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
			and LAG(COALESCE(t1.fclc_total,0)) OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period)<>0
			and LAG(COALESCE(t1.fclc_mob_no_vas,0)) OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period)=0
		then 'Миграция в B2B'


		when coalesce(t1.fclc_mob_no_vas,0)>0 
			and t1.tp_big<>'M2M/IOT' 
			and t1.segment IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
			AND COALESCE(t1.fclc_mob_no_vas,0)>= 
				(LAG(COALESCE(t1.fclc_mob_no_vas,0)) OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period)*1.15)
		then 'Развитие'


		when coalesce(t1.fclc_mob_no_vas,0)>0 
			and t1.tp_big<>'M2M/IOT' 
			and t1.segment IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
			AND COALESCE(t1.fclc_mob_no_vas,0)> 
				(LAG(COALESCE(t1.fclc_mob_no_vas,0)) OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period)*0.85)
			and COALESCE(t1.fclc_mob_no_vas,0)< 
				(LAG(COALESCE(t1.fclc_mob_no_vas,0)) OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period)*1.15)
		then 'Стабильно платят'


		when coalesce(t1.fclc_mob_no_vas,0)>0 
			and t1.tp_big<>'M2M/IOT' 
			and t1.segment IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
			AND COALESCE(t1.fclc_mob_no_vas,0)<=
				(LAG(COALESCE(t1.fclc_mob_no_vas,0)) OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period)*0.85)
		then 'Оптимизация' 


	else 'Пусто'
	end as pritok_all_no_vas	
	

--5 Отток моб тотал
	,CASE 
		when LAG(COALESCE(t1.fclc_mob_total,0)) OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period)=0 
			and coalesce(t1.fclc_mob_total, 0)=0 
		then 'Ранее перешёл в отток'

	
		when LAG(COALESCE(t1.fclc_mob_total,0)) OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period)<>0 
			and coalesce(t1.fclc_mob_total, 0)=0 
			and t1.segment_prev IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
			and LAG(t1.period) OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period)<t1.abonent_activation_month
		then 'Отток с активацией'
	
	
		when coalesce(t1.fclc_mob_total,0)=0 
			and LAG(COALESCE(t1.fclc_mob_total,0)) OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period)<>0 
			and coalesce(t1.fclc_total,0)=0 
			and t1.tp_big_prev<>'M2M/IOT' 
			and t1.segment_prev IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
			'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
		then 'Отток'


		when LAG(COALESCE(t1.fclc_mob_total,0)) OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period)<>0
			and coalesce(t1.fclc_mob_total,0)=0            
			and coalesce(t1.fclc_total,0)<>0 
			and t1.tp_big_prev<>'M2M/IOT' 
			and t1.tp_big<>'M2M/IOT' 
			and t1.segment_prev IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
			and t1.segment IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
		then 'Миграция в др. бизнес'


		when LAG(COALESCE(t1.fclc_mob_total,0)) OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period)<>0 
			and t1.tp_big_prev<>'M2M/IOT' 
			and t1.tp_big='M2M/IOT' 
			and COALESCE(t1.activation_marker,0) <> 1
			and t1.segment_prev IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
		then 'Миграция в ТП IOT'


		when LAG(COALESCE(t1.fclc_mob_total,0)) OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period)<>0 
			and t1.tp_big_prev<>'M2M/IOT' 
			and t1.segment_prev IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
			and t1.segment_mgr=1 
		then 'Миграция из сегмента'


	else 'Пусто'
	end as ottok
	
	
--6 Отток моб без вас	
	,CASE 
		when LAG(COALESCE(t1.fclc_mob_no_vas,0)) OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period)=0 
			and coalesce(t1.fclc_mob_no_vas, 0)=0 
		then 'Ранее перешёл в отток'


		when LAG(COALESCE(t1.fclc_mob_no_vas,0)) OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period)<>0 
			and coalesce(t1.fclc_mob_no_vas, 0)=0 
			and t1.segment_prev IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
			and LAG(t1.period) OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period)<t1.abonent_activation_month
		then 'Отток с активацией'


		when coalesce(t1.fclc_mob_no_vas,0)=0 
			and LAG(COALESCE(t1.fclc_mob_no_vas,0)) OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period)<>0 
			and coalesce(t1.fclc_total,0)=0 
			and t1.tp_big_prev<>'M2M/IOT' 
			and t1.segment_prev IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
		then 'Отток'


		when LAG(COALESCE(t1.fclc_mob_no_vas,0)) OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period)<>0 
			and coalesce(t1.fclc_mob_no_vas,0)=0
			and coalesce(t1.fclc_total,0)<>0 
			and t1.tp_big_prev<>'M2M/IOT' 
			and t1.tp_big<>'M2M/IOT' 
			and t1.segment_prev IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
			and t1.segment IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
		then 'Миграция в др. бизнес'


		when LAG(COALESCE(t1.fclc_mob_no_vas,0)) OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period)<>0 
			and t1.tp_big_prev<>'M2M/IOT' 
			and t1.tp_big='M2M/IOT'
			and COALESCE(t1.activation_marker,0) <> 1
			and t1.segment_prev IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
		then 'Миграция в ТП IOT'


		when LAG(COALESCE(t1.fclc_mob_no_vas,0)) OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period)<>0 
			and t1.tp_big_prev<>'M2M/IOT' 
			and t1.segment_prev IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
			and t1.segment_mgr=1 
		then 'Миграция из сегмента'


	else 'Пусто'
	end as ottok_no_vas	
	
	
--7 Отток моб тотал Общ
	,CASE 
		when LAG(COALESCE(t1.fclc_mob_total,0)) OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period)=0 
			and coalesce(t1.fclc_mob_total,0)=0 
		then 'Ранее перешёл в отток'

	
		when LAG(COALESCE(t1.fclc_mob_total,0)) OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period)<>0 
			and coalesce(t1.fclc_mob_total, 0)=0 
			and t1.segment_prev IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
			and LAG(t1.period) OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period)<t1.abonent_activation_month
		then 'Отток с активацией'
	
	
		when coalesce(t1.fclc_mob_total,0)=0 
			and LAG(COALESCE(t1.fclc_mob_total,0)) OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period)<>0 
			and coalesce(t1.fclc_total, 0)=0 
			and t1.tp_big_prev<>'M2M/IOT' 
			and t1.segment_prev IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
		then 'Отток'


		when LAG(COALESCE(t1.fclc_mob_total,0)) OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period)<>0
			and coalesce(t1.fclc_mob_total,0)=0            
			and coalesce(t1.fclc_total,0)<>0 
			and t1.tp_big_prev<>'M2M/IOT' 
			and t1.tp_big<>'M2M/IOT' 
			and t1.segment_prev IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
			and t1.segment IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
		then 'Миграция в др. бизнес'


		when LAG(COALESCE(t1.fclc_mob_total,0)) OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period)<>0 
			and t1.tp_big_prev<>'M2M/IOT' 
			and t1.tp_big='M2M/IOT' 
			and COALESCE(t1.activation_marker,0) <> 1
			and t1.segment_prev IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
		then 'Миграция в ТП IOT'


		when LAG(COALESCE(t1.fclc_mob_total,0)) OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period)<>0 
			and t1.tp_big_prev<>'M2M/IOT' 
			and t1.segment_prev IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
			and t1.segment NOT IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
		then 'Миграция из B2B'
	
	
	else 'Пусто' 
	end as ottok_all


--8 Отток моб без вас Общ
	,CASE 
		when LAG(COALESCE(t1.fclc_mob_no_vas,0)) OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period)=0 
			and coalesce(t1.fclc_mob_no_vas,0)=0 
		then 'Ранее перешёл в отток'

	
		when LAG(COALESCE(t1.fclc_mob_no_vas,0)) OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period)<>0 
			and coalesce(t1.fclc_mob_no_vas, 0)=0 
			and t1.segment_prev IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
			and LAG(t1.period) OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period)<t1.abonent_activation_month
		then 'Отток с активацией'
	
	
		when coalesce(t1.fclc_mob_no_vas,0)=0 
			and LAG(COALESCE(t1.fclc_mob_no_vas,0)) OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period)<>0 
			and coalesce(t1.fclc_total, 0)=0 
			and t1.tp_big_prev<>'M2M/IOT' 
			and t1.segment_prev IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
		then 'Отток'


		when LAG(COALESCE(t1.fclc_mob_no_vas,0)) OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period)<>0 
			and coalesce(t1.fclc_mob_no_vas,0)=0
			and coalesce(t1.fclc_total,0)<>0 
			and t1.tp_big_prev<>'M2M/IOT' 
			and t1.tp_big<>'M2M/IOT' 
			and t1.segment_prev IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
			and t1.segment IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
		then 'Миграция в др. бизнес'



		when LAG(COALESCE(t1.fclc_mob_no_vas,0)) OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period)<>0 
			and t1.tp_big_prev<>'M2M/IOT' 
			and t1.tp_big='M2M/IOT' 
			and COALESCE(t1.activation_marker,0) <> 1
			and t1.segment_prev IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
		then 'Миграция в ТП IOT'


		When LAG(COALESCE(t1.fclc_mob_no_vas,0)) OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period)<>0 
			and t1.tp_big_prev<>'M2M/IOT' 
			and t1.segment_prev IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
			and t1.segment NOT IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
		then 'Миграция из B2B'
	
	else 'Пусто' 
	end as ottok_all_no_vas
	
from analyticsb2b_sb.rkahmetov_vr_mob_vrem_13 t1
""").repartition(550).write.mode("overwrite").format("orc").saveAsTable("analyticsb2b_sb.rkahmetov_vr_mob_vrem_14")

In [ ]:
%%time
spark.sql("""
select
	t1.regid
	,t1.app_n
	,t1.tariff_id
	,t1.tariff
	,t1.tp_group_id
	,t1.tp_group
	,t1.tp_big
	,t1.m_categ_id
	,t1.m_categ
	,t1.segment
	,t1.inn
    ,t1.kontr_name
FROM (select a.*, row_number() OVER (PARTITION BY a.regid, a.app_n ORDER BY a.period desc) as rer from 
analyticsb2b_sb.rkahmetov_vr_mob_vrem_14 a) as t1
where t1.rer = 1 
""").repartition(15).write.mode("overwrite").format("orc").saveAsTable("analyticsb2b_sb.rkahmetov_vr_mob_vrem_15")

In [ ]:
%%time
spark.sql("""
SELECT
	b.period
    ,b.inn
	,count(*) as rew
from
   (
	select distinct
		t1.period
        ,t1.inn
		, t1.tp_big
	from analyticsb2b_sb.rkahmetov_vr_mob_vrem_14 t1
	where t1.inn||'-'||t1.period in (select distinct a.inn||'-'||a.period from analyticsb2b_sb.rkahmetov_vr_mob_vrem_14 a where a.tp_big = 'M2M/IOT')
	) b
group by 
	b.period
    ,b.inn
having 
	count(*) = 1
""").repartition(1).write.mode("overwrite").format("orc").saveAsTable("analyticsb2b_sb.rkahmetov_vr_mob_vrem_16")

In [ ]:
%%time
spark.sql("""
select
	t1.period
	, case 
		when t1.period = 202101 then '2021-01-01'
		when t1.period = 202102 then '2021-02-01'
		when t1.period = 202103 then '2021-03-01'
		when t1.period = 202104 then '2021-04-01'
		when t1.period = 202105 then '2021-05-01'
		when t1.period = 202106 then '2021-06-01'
		when t1.period = 202107 then '2021-07-01'
		when t1.period = 202108 then '2021-08-01'
		when t1.period = 202109 then '2021-09-01'
		when t1.period = 202110 then '2021-10-01'
		when t1.period = 202111 then '2021-11-01'
		when t1.period = 202112 then '2021-12-01'
		when t1.period = 202201 then '2022-01-01'
		when t1.period = 202202 then '2022-02-01'
		when t1.period = 202203 then '2022-03-01'
		when t1.period = 202204 then '2022-04-01'
		when t1.period = 202205 then '2022-05-01'
		when t1.period = 202206 then '2022-06-01'
		when t1.period = 202207 then '2022-07-01'
		when t1.period = 202208 then '2022-08-01'
		when t1.period = 202209 then '2022-09-01'
		when t1.period = 202210 then '2022-10-01'
		when t1.period = 202211 then '2022-11-01'
		when t1.period = 202212 then '2022-12-01'
		when t1.period = 202301 then '2023-01-01'
		when t1.period = 202302 then '2023-02-01'
		when t1.period = 202303 then '2023-03-01'
		when t1.period = 202304 then '2023-04-01'
		when t1.period = 202305 then '2023-05-01'
		when t1.period = 202306 then '2023-06-01'
		when t1.period = 202307 then '2023-07-01'
		when t1.period = 202308 then '2023-08-01'
		when t1.period = 202309 then '2023-09-01'
		when t1.period = 202310 then '2023-10-01'
		when t1.period = 202311 then '2023-11-01'
		when t1.period = 202312 then '2023-12-01'
        when t1.period = 202401 then '2024-01-01'
        when t1.period = 202402 then '2024-02-01'
        when t1.period = 202403 then '2024-03-01'
		when t1.period = 202404 then '2024-04-01'
		end as period_date
	,t1.regid
	,t1.app_n
    ,t1.phone_num
	,t1.con_n
	,t1.inn
	,t1.is_con_inn
	,t1.kontr_name
	,t1.abonent_activation_date
	,t1.abonent_activation_month
	,t1.activation_marker
	,t1.region
	,t1.region_txt
	,t1.Reg_Week2
	,t1.TR
	
	,t1.tariff_mgr
	,t1.tariff_id_prev
	,t1.tariff_id
	,t1.tariff_prev
	,t1.tariff
	
	,t1.tp_group_mgr
	,t1.tp_group_id_prev
	,t1.tp_group_id
	,t1.tp_group_prev
	,t1.tp_group
	
	,t1.tp_big_mgr
	,t1.tp_big_prev
	,t1.tp_big
		
	,t1.m_categ_mgr
	,t1.m_categ_id_prev
	,t1.m_categ_id
	,t1.m_categ_prev
	,t1.m_categ
	
	,t1.segment_mgr
	,t1.segment_prev
	,t1.segment
	,t1.smena_segmenta
	
	,t1.fclc_prochie_mob1
	,t1.fclc_prochie_vas1
	,t1.fclc_golos1
	,t1.fclc_gprs1
	,t1.fclc_sms1
	,t1.fclc_vsr1
	,t1.fclc_vz_mg_mn1
	,t1.fclc_abon_plata1
	,t1.fclc_mob_total1
	,t1.fclc_mob_no_vas1
	,t1.fclc_total1
	,t1.fclc_aff_serv1
	
	,LAG(COALESCE(t1.fclc_mob_total1,0)) OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period) as fclc_mob_total1_prev
	,LAG(COALESCE(t1.fclc_mob_no_vas1,0)) OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period) as fclc_mob_no_vas1_prev
	,LAG(COALESCE(t1.fclc_total1,0)) OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period) as fclc_total1_prev
	
	,t1.fclc_mob_total_x
	,t1.fclc_mob_no_vas_x
	,t1.fclc_total_x
	
	,t1.activnost_mob_total
	,t1.activnost_mob_no_vas
	
	,t1.abonent_ARPU_categ_mob_total
    ,t1.abonent_ARPU_categ_mob_no_vas
	
	,t1.mark_zak_block
	,t1.month_zak_block
	
	,t1.metk_aff_ser
	
	,t1.pritok as pritok_mob_total
	,t1.pritok_no_vas as pritok_mob_no_vas
	,t1.pritok_all as pritok_mob_total_all
	,t1.pritok_all_no_vas as pritok_mob_no_vas_all
	,t1.ottok as ottok_mob_total
	,t1.ottok_no_vas as ottok_mob_no_vas
	,t1.ottok_all as ottok_mob_total_all
	,t1.ottok_all_no_vas as ottok_mob_no_vas_all
	
	,AVG(t1.fclc_mob_total_x) 
		OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period ROWS BETWEEN 3 PRECEDING AND 1 PRECEDING) 
		as avg_do_ottoka_abonenta_mob_total
	,AVG(t1.fclc_mob_no_vas_x) 
		OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period ROWS BETWEEN 3 PRECEDING AND 1 PRECEDING) 
		as avg_do_ottoka_abonenta_mob_no_vas
	
	,t2.tariff_id as tariff_id_as_is
	,t2.tariff as tariff_as_is
	,t2.tp_group_id as tp_group_id_as_is
	,t2.tp_group as tp_group_as_is
	,t2.tp_big as tp_big_as_is
	,t2.m_categ_id as m_categ_id_as_is
	,t2.m_categ as m_categ_as_is
	,t2.segment as segment_as_is
	,t2.inn as inn_as_is
    ,t2.kontr_name as kontr_name_as_is
	
	,case 
		when t3.inn is null then 0
		else 1
		end as mark_inn_m2m_iot
	
from analyticsb2b_sb.rkahmetov_vr_mob_vrem_14 t1
left join analyticsb2b_sb.rkahmetov_vr_mob_vrem_15 t2
	on t1.regid = t2.regid
	and t1.app_n = t2.app_n
left join analyticsb2b_sb.rkahmetov_vr_mob_vrem_16 t3
	on t1.inn = t3.inn
	and t1.period = t3.period
""").repartition(550).write.mode("overwrite").format("orc").saveAsTable("analyticsb2b_sb.rkahmetov_vr_mob_vrem_17")

In [ ]:
%%time
spark.sql("""
select
	t1.period
	,t1.period_date
	,t1.regid
	,t1.app_n
    ,t1.phone_num
	,t1.con_n
	,t1.inn
	,t1.is_con_inn
	,t1.kontr_name
    ,t1.kontr_name_as_is
    
	,t1.abonent_activation_date
	,t1.abonent_activation_month
	,t1.activation_marker
	,t1.region
	,t1.region_txt
	,t1.Reg_Week2
	,t1.TR
	
	,t1.tariff_mgr
	,t1.tariff_id_prev
	,t1.tariff_id
	,t1.tariff_prev
	,t1.tariff
	
	,t1.tp_group_mgr
	,t1.tp_group_id_prev
	,t1.tp_group_id
	,t1.tp_group_prev
	,t1.tp_group
	
	,t1.tp_big_mgr
	,t1.tp_big_prev
	,t1.tp_big
		
	,t1.m_categ_mgr
	,t1.m_categ_id_prev
	,t1.m_categ_id
	,t1.m_categ_prev
	,t1.m_categ
	
	,t1.segment_mgr
	,t1.segment_prev
	,t1.segment
	,t1.smena_segmenta
	
	,t1.fclc_prochie_mob1
	,t1.fclc_prochie_vas1
	,t1.fclc_golos1
	,t1.fclc_gprs1
	,t1.fclc_sms1
	,t1.fclc_vsr1
	,t1.fclc_vz_mg_mn1
	,t1.fclc_abon_plata1
	,t1.fclc_mob_total1
	,t1.fclc_mob_no_vas1
	,t1.fclc_total1
	,t1.fclc_aff_serv1
	
	,t1.fclc_mob_total1_prev
	,t1.fclc_mob_no_vas1_prev
	,t1.fclc_total1_prev
	
	,t1.fclc_mob_total_x
	,t1.fclc_mob_no_vas_x
	,t1.fclc_total_x
	
	,t1.activnost_mob_total
	,t1.activnost_mob_no_vas
	
	,t1.abonent_ARPU_categ_mob_total
    ,t1.abonent_ARPU_categ_mob_no_vas
	
	,t1.mark_zak_block
	,t1.month_zak_block
	
	,t1.metk_aff_ser
	
	,t1.pritok_mob_total
	,t1.pritok_mob_no_vas
	,t1.pritok_mob_total_all
	,t1.pritok_mob_no_vas_all
	,t1.ottok_mob_total
	,t1.ottok_mob_no_vas
	,t1.ottok_mob_total_all
	,t1.ottok_mob_no_vas_all
	
	,t1.avg_do_ottoka_abonenta_mob_total
	,t1.avg_do_ottoka_abonenta_mob_no_vas
	,case 
		when t1.ottok_mob_total in ('Отток', 'Отток с активацией') then t1.avg_do_ottoka_abonenta_mob_total
		else null end as avg_do_ottoka_abonenta_mob_total1
	,case 
		when t1.ottok_mob_no_vas in ('Отток', 'Отток с активацией') then t1.avg_do_ottoka_abonenta_mob_no_vas
		else null end as avg_do_ottoka_abonenta_mob_no_vas1
	
	,t1.fclc_mob_total1 - t1.fclc_mob_total1_prev as sum_status_mob_total
	,t1.fclc_mob_no_vas1 - t1.fclc_mob_no_vas1_prev as sum_status_mob_no_vas
	
	,t1.tariff_id_as_is
	,t1.tariff_as_is
	,t1.tp_group_id_as_is
	,t1.tp_group_as_is
	,t1.tp_big_as_is
	,t1.m_categ_id_as_is
	,t1.m_categ_as_is
	,t1.segment_as_is
	,t1.inn_as_is
	
	,t1.mark_inn_m2m_iot
	
from analyticsb2b_sb.rkahmetov_vr_mob_vrem_17 t1
""").repartition(550).write.mode("overwrite").format("orc").saveAsTable("analyticsb2b_sb.rkahmetov_vr_mob_vrem_18")

In [ ]:
%%time
spark.sql("""
SELECT DISTINCT 
    regid
    , app_n
    , LAST_VALUE(period) OVER (PARTITION BY regid, app_n order by period) as last_ottok_mob_total
FROM analyticsb2b_sb.rkahmetov_vr_mob_vrem_18
WHERE ottok_mob_total in ('Отток', 'Отток с активацией')
""").repartition(5).write.mode("overwrite").format("orc").saveAsTable("analyticsb2b_sb.rkahmetov_vr_mob_vrem_19")

In [ ]:
%%time
spark.sql("""
SELECT DISTINCT 
    regid
    , app_n
    , LAST_VALUE(period) OVER (PARTITION BY regid, app_n order by period) as last_ottok_mob_no_vas
FROM analyticsb2b_sb.rkahmetov_vr_mob_vrem_18
WHERE ottok_mob_no_vas in ('Отток', 'Отток с активацией')
""").repartition(5).write.mode("overwrite").format("orc").saveAsTable("analyticsb2b_sb.rkahmetov_vr_mob_vrem_20")

In [ ]:
%%time
spark.sql("""
SELECT 
    t1.*
    , t2.last_ottok_mob_total
    , t3.last_ottok_mob_no_vas
FROM analyticsb2b_sb.rkahmetov_vr_mob_vrem_18 as t1
LEFT JOIN analyticsb2b_sb.rkahmetov_vr_mob_vrem_19 as t2
    ON t1.period = t2.last_ottok_mob_total 
    and t1.regid = t2.regid 
    and t1.app_n = t2.app_n
LEFT JOIN analyticsb2b_sb.rkahmetov_vr_mob_vrem_20 as t3
    ON t1.period = t3.last_ottok_mob_no_vas 
    and t1.regid = t3.regid 
    and t1.app_n = t3.app_n
""").repartition(550).write.mode("overwrite").format("orc").saveAsTable("analyticsb2b_sb.rkahmetov_vr_mob_vrem_21")

In [ ]:
%%time
spark.sql("""
SELECT 
    t1.*
    , CASE 
        WHEN t1.ottok_mob_total = 'Ранее перешёл в отток' 
            then (last_value(t1.last_ottok_mob_total, true) over (partition by t1.regid, t1.app_n order by t1.period ROWS UNBOUNDED PRECEDING))
        WHEN t1.ottok_mob_total in ('Отток', 'Отток с активацией') 
            then t1.last_ottok_mob_total 
        end as last_ottok_mob_total1
    , CASE 
        WHEN t1.ottok_mob_no_vas = 'Ранее перешёл в отток' 
            then (last_value(t1.last_ottok_mob_no_vas, true) over (partition by t1.regid, t1.app_n order by t1.period ROWS UNBOUNDED PRECEDING))
        WHEN t1.ottok_mob_no_vas in ('Отток', 'Отток с активацией') 
            then t1.last_ottok_mob_no_vas 
        end as last_ottok_mob_no_vas1
FROM analyticsb2b_sb.rkahmetov_vr_mob_vrem_21 as t1
""").repartition(550).write.mode("overwrite").format("orc").saveAsTable("analyticsb2b_sb.rkahmetov_vr_mob_vrem_22")

In [ ]:
%%time
spark.sql("""

SELECT 
    t1.*
    , CASE 
        WHEN t1.ottok_mob_total in ('Отток', 'Отток с активацией')
            and t1.period = t1.last_ottok_mob_total1 
            then t1.avg_do_ottoka_abonenta_mob_total1
        WHEN t1.ottok_mob_total = 'Ранее перешёл в отток' 
            then (last_value(t1.avg_do_ottoka_abonenta_mob_total1, true) 
				over (partition by t1.regid, t1.app_n, t1.last_ottok_mob_total1 
					order by t1.period ROWS UNBOUNDED PRECEDING))
        END AS rl_po_ottoku_mob_total 
FROM analyticsb2b_sb.rkahmetov_vr_mob_vrem_22 as t1  
""").repartition(550).write.mode("overwrite").format("orc").saveAsTable("analyticsb2b_sb.rkahmetov_vr_mob_vrem_23")

In [ ]:
%%time
spark.sql("""

SELECT 
    t1.*
    , CASE 
        WHEN t1.ottok_mob_no_vas in ('Отток', 'Отток с активацией')
            and t1.period = t1.last_ottok_mob_no_vas1 
            then t1.avg_do_ottoka_abonenta_mob_no_vas1
        WHEN t1.ottok_mob_no_vas = 'Ранее перешёл в отток'
            then (last_value(t1.avg_do_ottoka_abonenta_mob_no_vas1, true) 
				over (partition by t1.regid, t1.app_n, t1.last_ottok_mob_no_vas1 
					order by t1.period ROWS UNBOUNDED PRECEDING))
        END AS rl_po_ottoku_mob_no_vas 
FROM analyticsb2b_sb.rkahmetov_vr_mob_vrem_23 as t1  
""").repartition(550).write.mode("overwrite").format("orc").saveAsTable("analyticsb2b_sb.rkahmetov_vr_mob_vrem_24")

In [ ]:
%%time
spark.sql("""
select
	t1.period
	,t1.period_date
	,t1.regid
	,t1.app_n
    ,t1.phone_num
	,t1.con_n
	,t1.inn
	,t1.is_con_inn
	,t1.kontr_name
	,t1.abonent_activation_date
	,t1.abonent_activation_month
	,t1.activation_marker
	,t1.region
	,t1.region_txt
	,t1.Reg_Week2
	,t1.TR
	
	,t1.tariff_mgr
	,t1.tariff_id_prev
	,t1.tariff_id
	,t1.tariff_prev
	,t1.tariff
	
	,t1.tp_group_mgr
	,t1.tp_group_id_prev
	,t1.tp_group_id
	,t1.tp_group_prev
	,t1.tp_group
	
	,t1.tp_big_mgr
	,t1.tp_big_prev
	,t1.tp_big
		
	,t1.m_categ_mgr
	,t1.m_categ_id_prev
	,t1.m_categ_id
	,t1.m_categ_prev
	,t1.m_categ
	
	,t1.segment_mgr
	,t1.segment_prev
	,t1.segment
	,t1.smena_segmenta
	
	,t1.fclc_prochie_mob1
	,t1.fclc_prochie_vas1
	,t1.fclc_golos1
	,t1.fclc_gprs1
	,t1.fclc_sms1
	,t1.fclc_vsr1
	,t1.fclc_vz_mg_mn1
	,t1.fclc_abon_plata1
	,t1.fclc_mob_total1
	,t1.fclc_mob_no_vas1
	,t1.fclc_total1
	,t1.fclc_aff_serv1
	
	,t1.fclc_mob_total1_prev
	,t1.fclc_mob_no_vas1_prev
	,t1.fclc_total1_prev
	
	,t1.fclc_mob_total_x
	,t1.fclc_mob_no_vas_x
	,t1.fclc_total_x
	
	,t1.activnost_mob_total
	,t1.activnost_mob_no_vas
	
	,t1.abonent_ARPU_categ_mob_total
    ,t1.abonent_ARPU_categ_mob_no_vas
	,case 
		when coalesce(t1.avg_do_ottoka_abonenta_mob_total, 0) = 0 
			then '0'
		when coalesce(t1.avg_do_ottoka_abonenta_mob_total, 0) > 0
			and coalesce(t1.avg_do_ottoka_abonenta_mob_total, 0) < 50 
			then '0_50'
		when coalesce(t1.avg_do_ottoka_abonenta_mob_total, 0) >= 50
			and coalesce(t1.avg_do_ottoka_abonenta_mob_total, 0) < 100 
			then '50_100'
		when coalesce(t1.avg_do_ottoka_abonenta_mob_total, 0) >= 100
			and coalesce(t1.avg_do_ottoka_abonenta_mob_total, 0) < 250 
			then '100_250'
		when coalesce(t1.avg_do_ottoka_abonenta_mob_total, 0) >= 250
			and coalesce(t1.avg_do_ottoka_abonenta_mob_total, 0) < 500 
			then '250_500'
		when coalesce(t1.avg_do_ottoka_abonenta_mob_total, 0) >= 500 
			then '500_i_bolshe'
		end as abonent_arpu_categ_ottoka_mob_total
    ,case 
		when coalesce(t1.avg_do_ottoka_abonenta_mob_no_vas, 0) = 0 
			then '0'
		when coalesce(t1.avg_do_ottoka_abonenta_mob_no_vas, 0) > 0
			and coalesce(t1.avg_do_ottoka_abonenta_mob_no_vas, 0) < 50 
			then '0_50'
		when coalesce(t1.avg_do_ottoka_abonenta_mob_no_vas, 0) >= 50
			and coalesce(t1.avg_do_ottoka_abonenta_mob_no_vas, 0) < 100 
			then '50_100'
		when coalesce(t1.avg_do_ottoka_abonenta_mob_no_vas, 0) >= 100
			and coalesce(t1.avg_do_ottoka_abonenta_mob_no_vas, 0) < 250 
			then '100_250'
		when coalesce(t1.avg_do_ottoka_abonenta_mob_no_vas, 0) >= 250
			and coalesce(t1.avg_do_ottoka_abonenta_mob_no_vas, 0) < 500 
			then '250_500'
		when coalesce(t1.avg_do_ottoka_abonenta_mob_no_vas, 0) >= 500 
			then '500_i_bolshe'
		end as abonent_arpu_categ_ottoka_mob_no_vas
	
	,t1.mark_zak_block
	,t1.month_zak_block
	
	,t1.metk_aff_ser
	
	,t1.pritok_mob_total
	,t1.pritok_mob_no_vas
	,t1.pritok_mob_total_all
	,t1.pritok_mob_no_vas_all
	,t1.ottok_mob_total
	,t1.ottok_mob_no_vas
	,t1.ottok_mob_total_all
	,t1.ottok_mob_no_vas_all
	
	,t1.avg_do_ottoka_abonenta_mob_total
	,t1.avg_do_ottoka_abonenta_mob_no_vas
	,t1.rl_po_ottoku_mob_total
	,t1.rl_po_ottoku_mob_no_vas
	,t1.last_ottok_mob_total1 as last_date_ottok_mob_total
	,t1.last_ottok_mob_no_vas1 as last_date_ottok_mob_no_vas
	
	,case 
		when t1.fclc_mob_total1_prev = 0 then 100
		else (t1.fclc_mob_total1/t1.fclc_mob_total1_prev - 1) * 100
		end as proc_status_mob_total
	,case 
		when t1.fclc_mob_no_vas1_prev = 0 then 100
		else (t1.fclc_mob_no_vas1/t1.fclc_mob_no_vas1_prev - 1) * 100
		end as proc_status_mob_no_vas
	
	,t1.sum_status_mob_total
	,t1.sum_status_mob_no_vas
	
	,t1.tariff_id_as_is
	,t1.tariff_as_is
	,t1.tp_group_id_as_is
	,t1.tp_group_as_is
	,t1.tp_big_as_is
	,t1.m_categ_id_as_is
	,t1.m_categ_as_is
	,t1.segment_as_is
	,t1.inn_as_is
    ,t1.kontr_name_as_is
	
	,t1.mark_inn_m2m_iot
	
from analyticsb2b_sb.rkahmetov_vr_mob_vrem_24 t1
""").repartition(550).write.mode("overwrite").format("orc").saveAsTable("analyticsb2b_sb.rkahmetov_vr_mob_vrem_25")

In [ ]:
%%time
spark.sql("""
select
	t1.period
	,t1.period_date
	,t1.regid
	,t1.app_n
	,t1.phone_num
	,t1.con_n
	,t1.inn
	,t1.is_con_inn
	,t1.kontr_name
	,t1.abonent_activation_date
	,t1.abonent_activation_month
	,t1.activation_marker
	,t1.region
	,t1.region_txt
	,t1.Reg_Week2
	,t1.TR
	
	,t1.tariff_mgr
	,t1.tariff_id_prev
	,t1.tariff_id
	,t1.tariff_prev
	,t1.tariff
	
	,t1.tp_group_mgr
	,t1.tp_group_id_prev
	,t1.tp_group_id
	,t1.tp_group_prev
	,t1.tp_group
	
	,t1.tp_big_mgr
	,t1.tp_big_prev
	,t1.tp_big
		
	,t1.m_categ_mgr
	,t1.m_categ_id_prev
	,t1.m_categ_id
	,t1.m_categ_prev
	,t1.m_categ
	
	,t1.segment_mgr
	,t1.segment_prev
	,t1.segment
	,t1.smena_segmenta
	
	,t1.fclc_prochie_mob1 as fclc_prochie_mob
	,t1.fclc_prochie_vas1 as fclc_prochie_vas
	,t1.fclc_golos1 as fclc_golos
	,t1.fclc_gprs1 as fclc_gprs
	,t1.fclc_sms1 as fclc_sms
	,t1.fclc_vsr1 as fclc_vsr
	,t1.fclc_vz_mg_mn1 as fclc_vz_mg_mn
	,t1.fclc_abon_plata1 as fclc_abon_plata
	,t1.fclc_mob_total1 as fclc_mob_total
	,t1.fclc_mob_no_vas1 as fclc_mob_no_vas
	,t1.fclc_total1 as fclc_total
	,t1.fclc_aff_serv1 as fclc_aff_serv
	
	,t1.fclc_mob_total1_prev as fclc_mob_total_prev
	,t1.fclc_mob_no_vas1_prev as fclc_mob_no_vas_prev
	,t1.fclc_total1_prev as fclc_total_prev
	
	,t1.fclc_mob_total_x
	,t1.fclc_mob_no_vas_x
	,t1.fclc_total_x
	
	,t1.activnost_mob_total
	,t1.activnost_mob_no_vas
	
	,t1.abonent_ARPU_categ_mob_total as ARPU_abonent_categ_mob_total
    ,t1.abonent_ARPU_categ_mob_no_vas as ARPU_abonent_categ_mob_no_vas
	,t1.abonent_arpu_categ_ottoka_mob_total as ARPU_abonent_categ_ottoka_mob_total
    ,t1.abonent_arpu_categ_ottoka_mob_no_vas as ARPU_abonent_categ_ottoka_mob_no_vas
	
	,t1.mark_zak_block
	,t1.month_zak_block
	
	,t1.metk_aff_ser
	
	,t1.pritok_mob_total
	,t1.pritok_mob_no_vas
	,t1.pritok_mob_total_all
	,t1.pritok_mob_no_vas_all
	,t1.ottok_mob_total
	,t1.ottok_mob_no_vas
	,t1.ottok_mob_total_all
	,t1.ottok_mob_no_vas_all
	
	,t1.avg_do_ottoka_abonenta_mob_total
	,t1.avg_do_ottoka_abonenta_mob_no_vas
	,t1.rl_po_ottoku_mob_total as rl_po_ottoku_abonenta_mob_total
	,t1.rl_po_ottoku_mob_no_vas as rl_po_ottoku_abonenta_mob_no_vas
	,t1.last_date_ottok_mob_total as ottok_last_month_mob_total 
	,t1.last_date_ottok_mob_no_vas as ottok_last_month_mob_no_vas 
	
	,t1.proc_status_mob_total
	,t1.proc_status_mob_no_vas
	,case 
		when t1.proc_status_mob_total = 0 then '0'
		when t1.proc_status_mob_total < 0 and t1.proc_status_mob_total >= -5 then '<0_>=-5%'
		when t1.proc_status_mob_total < -5 and t1.proc_status_mob_total >= -10 then '<-5_>=-10%'
		when t1.proc_status_mob_total < -10 and t1.proc_status_mob_total >= -15 then '<-10_>=-15%'
		when t1.proc_status_mob_total < -15 and t1.proc_status_mob_total >= -20 then '<-15_>=-20%'
		when t1.proc_status_mob_total < -20 and t1.proc_status_mob_total >= -25 then '<-20_>=-25%'
		when t1.proc_status_mob_total < -25 and t1.proc_status_mob_total >= -30 then '<-25_>=-30%'
		when t1.proc_status_mob_total < -30 and t1.proc_status_mob_total >= -35 then '<-30_>=-35%'
		when t1.proc_status_mob_total < -35 and t1.proc_status_mob_total >= -40 then '<-35_>=-40%'
		when t1.proc_status_mob_total < -40 and t1.proc_status_mob_total >= -45 then '<-40_>=-45%'
		when t1.proc_status_mob_total < -45 and t1.proc_status_mob_total >= -50 then '<-45_>=-50%'
		when t1.proc_status_mob_total < -50 and t1.proc_status_mob_total >= -55 then '<-50_>=-55%'
		when t1.proc_status_mob_total < -55 and t1.proc_status_mob_total >= -60 then '<-55_>=-60%'
		when t1.proc_status_mob_total < -60 and t1.proc_status_mob_total >= -65 then '<-60_>=-65%'
		when t1.proc_status_mob_total < -65 and t1.proc_status_mob_total >= -70 then '<-65_>=-70%'
		when t1.proc_status_mob_total < -70 and t1.proc_status_mob_total >= -75 then '<-70_>=-75%'
		when t1.proc_status_mob_total < -75 and t1.proc_status_mob_total >= -80 then '<-75_>=-80%'
		when t1.proc_status_mob_total < -80 and t1.proc_status_mob_total >= -85 then '<-80_>=-85%'
		when t1.proc_status_mob_total < -85 and t1.proc_status_mob_total >= -90 then '<-85_>=-90%'
		when t1.proc_status_mob_total < -90 and t1.proc_status_mob_total >= -95 then '<-90_>=-95%'
		when t1.proc_status_mob_total < -95 and t1.proc_status_mob_total >= -100 then '<-95_>=-100%'
		
		when t1.proc_status_mob_total > 0 and t1.proc_status_mob_total <= 5 then '>0_<=5%'
		when t1.proc_status_mob_total > 5 and t1.proc_status_mob_total <= 10 then '>5_<=10%'
		when t1.proc_status_mob_total > 10 and t1.proc_status_mob_total <= 15 then '>10_<=15%'
		when t1.proc_status_mob_total > 15 and t1.proc_status_mob_total <= 20 then '>15_<=20%'
		when t1.proc_status_mob_total > 20 and t1.proc_status_mob_total <= 25 then '>20_<=25%'
		when t1.proc_status_mob_total > 25 and t1.proc_status_mob_total <= 30 then '>25_<=30%'
		when t1.proc_status_mob_total > 30 and t1.proc_status_mob_total <= 35 then '>30_<=35%'
		when t1.proc_status_mob_total > 35 and t1.proc_status_mob_total <= 40 then '>35_<=40%'
		when t1.proc_status_mob_total > 40 and t1.proc_status_mob_total <= 45 then '>40_<=45%'
		when t1.proc_status_mob_total > 45 and t1.proc_status_mob_total <= 50 then '>45_<=50%'
		when t1.proc_status_mob_total > 50 and t1.proc_status_mob_total <= 55 then '>50_<=55%'
		when t1.proc_status_mob_total > 55 and t1.proc_status_mob_total <= 60 then '>55_<=60%'
		when t1.proc_status_mob_total > 60 and t1.proc_status_mob_total <= 65 then '>60_<=65%'
		when t1.proc_status_mob_total > 65 and t1.proc_status_mob_total <= 70 then '>65_<=70%'
		when t1.proc_status_mob_total > 70 and t1.proc_status_mob_total <= 75 then '>70_<=75%'
		when t1.proc_status_mob_total > 75 and t1.proc_status_mob_total <= 80 then '>75_<=80%'
		when t1.proc_status_mob_total > 80 and t1.proc_status_mob_total <= 85 then '>80_<=85%'
		when t1.proc_status_mob_total > 85 and t1.proc_status_mob_total <= 90 then '>85_<=90%'
		when t1.proc_status_mob_total > 90 and t1.proc_status_mob_total <= 95 then '>90_<=95%'
		when t1.proc_status_mob_total > 95 and t1.proc_status_mob_total <= 100 then '>95_<=100%'
		when t1.proc_status_mob_total > 100 then '>100%'
		
		else '0' end as categ_proc_status_mob_total
	,case 
		when t1.proc_status_mob_no_vas = 0 then '0'
		when t1.proc_status_mob_no_vas < 0 and t1.proc_status_mob_no_vas >= -5 then '<0_>=-5%'
		when t1.proc_status_mob_no_vas < -5 and t1.proc_status_mob_no_vas >= -10 then '<-5_>=-10%'
		when t1.proc_status_mob_no_vas < -10 and t1.proc_status_mob_no_vas >= -15 then '<-10_>=-15%'
		when t1.proc_status_mob_no_vas < -15 and t1.proc_status_mob_no_vas >= -20 then '<-15_>=-20%'
		when t1.proc_status_mob_no_vas < -20 and t1.proc_status_mob_no_vas >= -25 then '<-20_>=-25%'
		when t1.proc_status_mob_no_vas < -25 and t1.proc_status_mob_no_vas >= -30 then '<-25_>=-30%'
		when t1.proc_status_mob_no_vas < -30 and t1.proc_status_mob_no_vas >= -35 then '<-30_>=-35%'
		when t1.proc_status_mob_no_vas < -35 and t1.proc_status_mob_no_vas >= -40 then '<-35_>=-40%'
		when t1.proc_status_mob_no_vas < -40 and t1.proc_status_mob_no_vas >= -45 then '<-40_>=-45%'
		when t1.proc_status_mob_no_vas < -45 and t1.proc_status_mob_no_vas >= -50 then '<-45_>=-50%'
		when t1.proc_status_mob_no_vas < -50 and t1.proc_status_mob_no_vas >= -55 then '<-50_>=-55%'
		when t1.proc_status_mob_no_vas < -55 and t1.proc_status_mob_no_vas >= -60 then '<-55_>=-60%'
		when t1.proc_status_mob_no_vas < -60 and t1.proc_status_mob_no_vas >= -65 then '<-60_>=-65%'
		when t1.proc_status_mob_no_vas < -65 and t1.proc_status_mob_no_vas >= -70 then '<-65_>=-70%'
		when t1.proc_status_mob_no_vas < -70 and t1.proc_status_mob_no_vas >= -75 then '<-70_>=-75%'
		when t1.proc_status_mob_no_vas < -75 and t1.proc_status_mob_no_vas >= -80 then '<-75_>=-80%'
		when t1.proc_status_mob_no_vas < -80 and t1.proc_status_mob_no_vas >= -85 then '<-80_>=-85%'
		when t1.proc_status_mob_no_vas < -85 and t1.proc_status_mob_no_vas >= -90 then '<-85_>=-90%'
		when t1.proc_status_mob_no_vas < -90 and t1.proc_status_mob_no_vas >= -95 then '<-90_>=-95%'
		when t1.proc_status_mob_no_vas < -95 and t1.proc_status_mob_no_vas >= -100 then '<-95_>=-100%'
		
		when t1.proc_status_mob_no_vas > 0 and t1.proc_status_mob_no_vas <= 5 then '>0_<=5%'
		when t1.proc_status_mob_no_vas > 5 and t1.proc_status_mob_no_vas <= 10 then '>5_<=10%'
		when t1.proc_status_mob_no_vas > 10 and t1.proc_status_mob_no_vas <= 15 then '>10_<=15%'
		when t1.proc_status_mob_no_vas > 15 and t1.proc_status_mob_no_vas <= 20 then '>15_<=20%'
		when t1.proc_status_mob_no_vas > 20 and t1.proc_status_mob_no_vas <= 25 then '>20_<=25%'
		when t1.proc_status_mob_no_vas > 25 and t1.proc_status_mob_no_vas <= 30 then '>25_<=30%'
		when t1.proc_status_mob_no_vas > 30 and t1.proc_status_mob_no_vas <= 35 then '>30_<=35%'
		when t1.proc_status_mob_no_vas > 35 and t1.proc_status_mob_no_vas <= 40 then '>35_<=40%'
		when t1.proc_status_mob_no_vas > 40 and t1.proc_status_mob_no_vas <= 45 then '>40_<=45%'
		when t1.proc_status_mob_no_vas > 45 and t1.proc_status_mob_no_vas <= 50 then '>45_<=50%'
		when t1.proc_status_mob_no_vas > 50 and t1.proc_status_mob_no_vas <= 55 then '>50_<=55%'
		when t1.proc_status_mob_no_vas > 55 and t1.proc_status_mob_no_vas <= 60 then '>55_<=60%'
		when t1.proc_status_mob_no_vas > 60 and t1.proc_status_mob_no_vas <= 65 then '>60_<=65%'
		when t1.proc_status_mob_no_vas > 65 and t1.proc_status_mob_no_vas <= 70 then '>65_<=70%'
		when t1.proc_status_mob_no_vas > 70 and t1.proc_status_mob_no_vas <= 75 then '>70_<=75%'
		when t1.proc_status_mob_no_vas > 75 and t1.proc_status_mob_no_vas <= 80 then '>75_<=80%'
		when t1.proc_status_mob_no_vas > 80 and t1.proc_status_mob_no_vas <= 85 then '>80_<=85%'
		when t1.proc_status_mob_no_vas > 85 and t1.proc_status_mob_no_vas <= 90 then '>85_<=90%'
		when t1.proc_status_mob_no_vas > 90 and t1.proc_status_mob_no_vas <= 95 then '>90_<=95%'
		when t1.proc_status_mob_no_vas > 95 and t1.proc_status_mob_no_vas <= 100 then '>95_<=100%'
		when t1.proc_status_mob_no_vas > 100 then '>100%'
		
		else '0' end as categ_proc_status_mob_no_vas
	
	,t1.sum_status_mob_total
	,t1.sum_status_mob_no_vas
	
	,t1.tariff_id_as_is
	,t1.tariff_as_is
	,t1.tp_group_id_as_is
	,t1.tp_group_as_is
	,t1.tp_big_as_is
	,t1.m_categ_id_as_is
	,t1.m_categ_as_is
	,t1.segment_as_is
	,t1.inn_as_is
    ,t1.kontr_name_as_is
	
	,t1.mark_inn_m2m_iot
	,1 as count_abonent
	
from analyticsb2b_sb.rkahmetov_vr_mob_vrem_25 t1
""").repartition(550).write.mode("overwrite").format("orc").saveAsTable("analyticsb2b_sb.rkahmetov_vr_mob_vrem_26")

In [ ]:
%%time
spark.sql("""

select
	a.period
	,a.period_date
	,a.regid
	,a.app_n
	,a.phone_num
	,a.con_n
	,a.inn
	,a.is_con_inn
	,a.kontr_name
	,a.abonent_activation_date
	,a.abonent_activation_month
	,a.activation_marker
	,a.region
	,a.region_txt
	,a.Reg_Week2
	,a.TR
	
	,a.tariff_mgr
	,a.tariff_id_prev
	,a.tariff_id
	,a.tariff_prev
	,a.tariff
	
	,a.tp_group_mgr
	,a.tp_group_id_prev
	,a.tp_group_id
	,a.tp_group_prev
	,a.tp_group
	
	,a.tp_big_mgr
	,a.tp_big_prev
	,a.tp_big
		
	,a.m_categ_mgr
	,a.m_categ_id_prev
	,a.m_categ_id
	,a.m_categ_prev
	,a.m_categ
	
	,a.segment_mgr
	,a.segment_prev
	,a.segment
	,a.smena_segmenta
	
	,a.fclc_prochie_mob
	,a.fclc_prochie_vas
	,a.fclc_golos
	,a.fclc_gprs
	,a.fclc_sms
	,a.fclc_vsr
	,a.fclc_vz_mg_mn
	,a.fclc_abon_plata
	,a.fclc_mob_total
	,a.fclc_mob_no_vas
	,a.fclc_total
	,a.fclc_aff_serv
	
	,a.fclc_mob_total_prev
	,a.fclc_mob_no_vas_prev
	,a.fclc_total_prev
	
	,a.fclc_mob_total_x
	,a.fclc_mob_no_vas_x
	,a.fclc_total_x
	
	,a.activnost_mob_total
	,a.activnost_mob_no_vas
	
	,a.ARPU_abonent_categ_mob_total
    ,a.ARPU_abonent_categ_mob_no_vas
	,a.ARPU_abonent_categ_ottoka_mob_total
    ,a.ARPU_abonent_categ_ottoka_mob_no_vas
	
	,a.mark_zak_block
	,a.month_zak_block
	
	,a.metk_aff_ser
	
	,a.pritok_mob_total
	,a.pritok_mob_no_vas
	,a.pritok_mob_total_all
	,a.pritok_mob_no_vas_all
	,a.ottok_mob_total
	,a.ottok_mob_no_vas
	,a.ottok_mob_total_all
	,a.ottok_mob_no_vas_all
	
	,a.avg_do_ottoka_abonenta_mob_total
	,a.avg_do_ottoka_abonenta_mob_no_vas
	,a.rl_po_ottoku_abonenta_mob_total
	,a.rl_po_ottoku_abonenta_mob_no_vas
	,a.ottok_last_month_mob_total 
	,a.ottok_last_month_mob_no_vas 
	
	,a.proc_status_mob_total
	,a.proc_status_mob_no_vas
	,a.categ_proc_status_mob_total
	,a.categ_proc_status_mob_no_vas
	
	,a.sum_status_mob_total
	,a.sum_status_mob_no_vas
	
	,a.tariff_id_as_is
	,a.tariff_as_is
	,a.tp_group_id_as_is
	,a.tp_group_as_is
	,a.tp_big_as_is
	,a.m_categ_id_as_is
	,a.m_categ_as_is
	,a.segment_as_is
	,a.inn_as_is
    ,a.kontr_name_as_is
	
	,a.mark_inn_m2m_iot
	,a.count_abonent
	
	--1 Приток моб тотал
	,case 
		when a.period < a.abonent_activation_month
			and a.segment_as_is IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
			and a.tp_big_as_is<>'M2M/IOT' 
			and coalesce(a.fclc_mob_total,0)>0 
		then 'Активный без активации'


		when coalesce(a.fclc_mob_total,0)>0 
			and a.activation_marker=1 and tp_big_as_is<>'M2M/IOT' 
			and a.segment_as_is IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
		then 'Продажа'

		
		when coalesce(a.fclc_mob_total,0)>0 
			and a.tp_big_as_is<>'M2M/IOT' 
			and a.segment_as_is IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
			and LAG(COALESCE(a.fclc_total,0)) OVER (PARTITION BY a.regid, a.app_n ORDER BY a.period)<>0
			and LAG(COALESCE(a.fclc_mob_total,0)) OVER (PARTITION BY a.regid, a.app_n ORDER BY a.period)=0
		then 'Миграция из др. бизнеса'

	
		when coalesce(a.fclc_mob_total,0)>0 
			and a.tp_big_as_is<>'M2M/IOT' 
			and a.segment_as_is IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
			and LAG(COALESCE(a.fclc_total,0)) OVER (PARTITION BY a.regid, a.app_n ORDER BY a.period)=0 
		then 'Реактивация'

		
                when coalesce(a.fclc_mob_total,0)>0 
			and a.tp_big_as_is<>'M2M/IOT' 
			and a.segment_as_is IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
			AND COALESCE(a.fclc_mob_total,0)>= 
				(LAG(COALESCE(a.fclc_mob_total,0)) OVER (PARTITION BY a.regid, a.app_n ORDER BY a.period)*1.15)
		then 'Развитие'
		
		
		when coalesce(a.fclc_mob_total,0)>0 
			and a.tp_big_as_is<>'M2M/IOT' 
			and a.segment_as_is IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
			AND COALESCE(a.fclc_mob_total,0)> 
				(LAG(COALESCE(a.fclc_mob_total,0)) OVER (PARTITION BY a.regid, a.app_n ORDER BY a.period)*0.85)
			and COALESCE(a.fclc_mob_total,0)< 
				(LAG(COALESCE(a.fclc_mob_total,0)) OVER (PARTITION BY a.regid, a.app_n ORDER BY a.period)*1.15)
		then 'Стабильно платят'
	
		
		when coalesce(a.fclc_mob_total,0)>0 
			and a.tp_big_as_is<>'M2M/IOT' 
			and a.segment_as_is IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг')
			AND COALESCE(a.fclc_mob_total,0)<=
				(LAG(COALESCE(a.fclc_mob_total,0)) OVER (PARTITION BY a.regid, a.app_n ORDER BY a.period)*0.85)
		then 'Оптимизация' 
	
	
	else 'Пусто'
	end as pritok_mob_total_as_is
		
		
--2 Приток моб без вас
	,case 
		when a.period < a.abonent_activation_month
			and a.segment_as_is IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
			and a.tp_big_as_is<>'M2M/IOT' 
			and coalesce(a.fclc_mob_no_vas,0)>0 
		then 'Активный без активации'
	
	
		when coalesce(a.fclc_mob_no_vas,0)>0 
			and a.activation_marker=1 and tp_big_as_is<>'M2M/IOT' 
			and a.segment_as_is IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
		then 'Продажа'

		
		when coalesce(a.fclc_mob_no_vas,0)>0 
			and a.tp_big_as_is<>'M2M/IOT' 
			and a.segment_as_is IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
			and LAG(COALESCE(a.fclc_total,0)) OVER (PARTITION BY a.regid, a.app_n ORDER BY a.period)<>0
			and LAG(COALESCE(a.fclc_mob_no_vas,0)) OVER (PARTITION BY a.regid, a.app_n ORDER BY a.period)=0
		then 'Миграция из др. бизнеса'

	
		when coalesce(a.fclc_mob_no_vas,0)>0 
			and a.tp_big_as_is<>'M2M/IOT' 
			and a.segment_as_is IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
			and LAG(COALESCE(a.fclc_total,0)) OVER (PARTITION BY a.regid, a.app_n ORDER BY a.period)=0 
		then 'Реактивация'

		
		when coalesce(a.fclc_mob_no_vas,0)>0 
			and a.tp_big_as_is<>'M2M/IOT' 
			and a.segment_as_is IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
			AND COALESCE(a.fclc_mob_no_vas,0)>= 
				(LAG(COALESCE(a.fclc_mob_no_vas,0)) OVER (PARTITION BY a.regid, a.app_n ORDER BY a.period)*1.15)
		then 'Развитие'
		
		
		when coalesce(a.fclc_mob_no_vas,0)>0 
			and a.tp_big_as_is<>'M2M/IOT' 
			and a.segment_as_is IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
			AND COALESCE(a.fclc_mob_no_vas,0)> 
				(LAG(COALESCE(a.fclc_mob_no_vas,0)) OVER (PARTITION BY a.regid, a.app_n ORDER BY a.period)*0.85)
			and COALESCE(a.fclc_mob_no_vas,0)< 
				(LAG(COALESCE(a.fclc_mob_no_vas,0)) OVER (PARTITION BY a.regid, a.app_n ORDER BY a.period)*1.15)
		then 'Стабильно платят'
	
		
		when coalesce(a.fclc_mob_no_vas,0)>0 
			and a.tp_big_as_is<>'M2M/IOT' 
			and a.segment_as_is IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг')
			AND COALESCE(a.fclc_mob_no_vas,0)<=
				(LAG(COALESCE(a.fclc_mob_no_vas,0)) OVER (PARTITION BY a.regid, a.app_n ORDER BY a.period)*0.85)
		then 'Оптимизация' 
	
	
	else 'Пусто'
	end as pritok_mob_no_vas_as_is		
		
		
--3 Приток моб тотал Общ		
	,case
		when a.period < a.abonent_activation_month
			and a.segment_as_is IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
			and a.tp_big_as_is<>'M2M/IOT' 
			and coalesce(a.fclc_mob_total,0)>0 
		then 'Активный без активации'
	
	
		when coalesce(a.fclc_mob_total,0)>0 
			and a.activation_marker=1 
			and a.tp_big_as_is<>'M2M/IOT' 
			and a.segment_as_is IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
		then 'Продажа'


		when coalesce(a.fclc_mob_total,0)>0 
			and a.tp_big_as_is<>'M2M/IOT' 
			and a.segment_as_is IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
			and LAG(COALESCE(a.fclc_total,0)) OVER (PARTITION BY a.regid, a.app_n ORDER BY a.period)<>0
			and LAG(COALESCE(a.fclc_mob_total,0)) OVER (PARTITION BY a.regid, a.app_n ORDER BY a.period)=0
		then 'Миграция из др. бизнеса'


		when coalesce(fclc_mob_total,0)>0 
			and a.tp_big_as_is<>'M2M/IOT' 
			and a.segment_as_is IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
			and LAG(COALESCE(a.fclc_total,0)) OVER (PARTITION BY a.regid, a.app_n ORDER BY a.period)=0 
		then 'Реактивация'


		when coalesce(a.fclc_mob_total,0)>0 
			and a.tp_big_as_is<>'M2M/IOT' 
			and a.segment_as_is IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
			AND COALESCE(a.fclc_mob_total,0)>= 
				(LAG(COALESCE(a.fclc_mob_total,0)) OVER (PARTITION BY a.regid, a.app_n ORDER BY a.period)*1.15)
		then 'Развитие'


		when coalesce(a.fclc_mob_total,0)>0 
			and a.tp_big_as_is<>'M2M/IOT' 
			and a.segment_as_is IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
			AND COALESCE(a.fclc_mob_total,0)> 
				(LAG(COALESCE(a.fclc_mob_total,0)) OVER (PARTITION BY a.regid, a.app_n ORDER BY a.period)*0.85)
			and COALESCE(a.fclc_mob_total,0)< 
				(LAG(COALESCE(a.fclc_mob_total,0)) OVER (PARTITION BY a.regid, a.app_n ORDER BY a.period)*1.15)
		then 'Стабильно платят'


		when coalesce(a.fclc_mob_total,0)>0 
			and a.tp_big_as_is<>'M2M/IOT' 
			and a.segment_as_is IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
			AND COALESCE(a.fclc_mob_total,0)<=
				(LAG(COALESCE(a.fclc_mob_total,0)) OVER (PARTITION BY a.regid, a.app_n ORDER BY a.period)*0.85)
		then 'Оптимизация' 


	else 'Пусто'
	end as pritok_mob_total_all_as_is
	
	
--4 Приток моб без вас Общ
	,case
		when a.period < a.abonent_activation_month
			and a.segment_as_is IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
			and a.tp_big_as_is<>'M2M/IOT' 
			and coalesce(a.fclc_mob_no_vas,0)>0 
		then 'Активный без активации'
	
	
		when coalesce(a.fclc_mob_no_vas,0)>0 
			and a.activation_marker=1 
			and a.tp_big_as_is<>'M2M/IOT' 
			and a.segment_as_is IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
		then 'Продажа'


		when coalesce(a.fclc_mob_no_vas,0)>0 
			and a.tp_big_as_is<>'M2M/IOT' 
			and a.segment_as_is IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
			and LAG(COALESCE(a.fclc_total,0)) OVER (PARTITION BY a.regid, a.app_n ORDER BY a.period)<>0
			and LAG(COALESCE(a.fclc_mob_no_vas,0)) OVER (PARTITION BY a.regid, a.app_n ORDER BY a.period)=0
		then 'Миграция из др. бизнеса'


		when coalesce(fclc_mob_no_vas,0)>0 
			and a.tp_big_as_is<>'M2M/IOT' 
			and a.segment_as_is IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
			and LAG(COALESCE(a.fclc_total,0)) OVER (PARTITION BY a.regid, a.app_n ORDER BY a.period)=0 
		then 'Реактивация'


		when coalesce(a.fclc_mob_no_vas,0)>0 
			and a.tp_big_as_is<>'M2M/IOT' 
			and a.segment_as_is IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
			AND COALESCE(a.fclc_mob_no_vas,0)>= 
				(LAG(COALESCE(a.fclc_mob_no_vas,0)) OVER (PARTITION BY a.regid, a.app_n ORDER BY a.period)*1.15)
		then 'Развитие'


		when coalesce(a.fclc_mob_no_vas,0)>0 
			and a.tp_big_as_is<>'M2M/IOT' 
			and a.segment_as_is IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
			AND COALESCE(a.fclc_mob_no_vas,0)> 
				(LAG(COALESCE(a.fclc_mob_no_vas,0)) OVER (PARTITION BY a.regid, a.app_n ORDER BY a.period)*0.85)
			and COALESCE(a.fclc_mob_no_vas,0)< 
				(LAG(COALESCE(a.fclc_mob_no_vas,0)) OVER (PARTITION BY a.regid, a.app_n ORDER BY a.period)*1.15)
		then 'Стабильно платят'


		when coalesce(a.fclc_mob_no_vas,0)>0 
			and a.tp_big_as_is<>'M2M/IOT' 
			and a.segment_as_is IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
			AND COALESCE(a.fclc_mob_no_vas,0)<=
				(LAG(COALESCE(a.fclc_mob_no_vas,0)) OVER (PARTITION BY a.regid, a.app_n ORDER BY a.period)*0.85)
		then 'Оптимизация' 


	else 'Пусто'
	end as pritok_mob_no_vas_all_as_is	
	

--5 Отток моб тотал
	,CASE 
		when LAG(COALESCE(a.fclc_mob_total,0)) OVER (PARTITION BY a.regid, a.app_n ORDER BY a.period)=0 
			and coalesce(a.fclc_mob_total, 0)=0 
		then 'Ранее перешёл в отток'

	
		when LAG(COALESCE(a.fclc_mob_total,0)) OVER (PARTITION BY a.regid, a.app_n ORDER BY a.period)<>0 
			and coalesce(a.fclc_mob_total, 0)=0 
			and a.segment_as_is IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
			and LAG(a.period) OVER (PARTITION BY a.regid, a.app_n ORDER BY a.period)<a.abonent_activation_month
		then 'Отток с активацией'
	
	
		when coalesce(a.fclc_mob_total,0)=0 
			and LAG(COALESCE(a.fclc_mob_total,0)) OVER (PARTITION BY a.regid, a.app_n ORDER BY a.period)<>0 
			and coalesce(a.fclc_total,0)=0 
			and a.tp_big_as_is<>'M2M/IOT' 
			and a.segment_as_is IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
			'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
		then 'Отток'


		when LAG(COALESCE(a.fclc_mob_total,0)) OVER (PARTITION BY a.regid, a.app_n ORDER BY a.period)<>0
			and coalesce(a.fclc_mob_total,0)=0            
			and coalesce(a.fclc_total,0)<>0 
			and a.tp_big_as_is<>'M2M/IOT' 
			and a.segment_as_is IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
		then 'Миграция в др. бизнес'


		
	else 'Пусто'
	end as ottok_mob_total_as_is
	
	
--6 Отток моб без вас	
	,CASE 
		when LAG(COALESCE(a.fclc_mob_no_vas,0)) OVER (PARTITION BY a.regid, a.app_n ORDER BY a.period)=0 
			and coalesce(a.fclc_mob_no_vas, 0)=0 
		then 'Ранее перешёл в отток'


		when LAG(COALESCE(a.fclc_mob_no_vas,0)) OVER (PARTITION BY a.regid, a.app_n ORDER BY a.period)<>0 
			and coalesce(a.fclc_mob_no_vas, 0)=0 
			and a.segment_as_is IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
			and LAG(a.period) OVER (PARTITION BY a.regid, a.app_n ORDER BY a.period)<a.abonent_activation_month
		then 'Отток с активацией'


		when coalesce(a.fclc_mob_no_vas,0)=0 
			and LAG(COALESCE(a.fclc_mob_no_vas,0)) OVER (PARTITION BY a.regid, a.app_n ORDER BY a.period)<>0 
			and coalesce(a.fclc_total,0)=0 
			and a.tp_big_prev<>'M2M/IOT' 
			and a.segment_as_is IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
		then 'Отток'


		when LAG(COALESCE(a.fclc_mob_no_vas,0)) OVER (PARTITION BY a.regid, a.app_n ORDER BY a.period)<>0 
			and coalesce(a.fclc_mob_no_vas,0)=0
			and coalesce(a.fclc_total,0)<>0 
			and a.tp_big_as_is<>'M2M/IOT' 
			and a.segment_as_is IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
		then 'Миграция в др. бизнес'


		
	else 'Пусто'
	end as ottok_mob_no_vas_as_is	
	
	
--7 Отток моб тотал Общ
	,CASE 
		when LAG(COALESCE(a.fclc_mob_total,0)) OVER (PARTITION BY a.regid, a.app_n ORDER BY a.period)=0 
			and coalesce(a.fclc_mob_total,0)=0 
		then 'Ранее перешёл в отток'

	
		when LAG(COALESCE(a.fclc_mob_total,0)) OVER (PARTITION BY a.regid, a.app_n ORDER BY a.period)<>0 
			and coalesce(a.fclc_mob_total, 0)=0 
			and a.segment_as_is IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
			and LAG(a.period) OVER (PARTITION BY a.regid, a.app_n ORDER BY a.period)<a.abonent_activation_month
		then 'Отток с активацией'
	
	
		when coalesce(a.fclc_mob_total,0)=0 
			and LAG(COALESCE(a.fclc_mob_total,0)) OVER (PARTITION BY a.regid, a.app_n ORDER BY a.period)<>0 
			and coalesce(a.fclc_total, 0)=0 
			and a.tp_big_as_is<>'M2M/IOT' 
			and a.segment_as_is IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
		then 'Отток'


		when LAG(COALESCE(a.fclc_mob_total,0)) OVER (PARTITION BY a.regid, a.app_n ORDER BY a.period)<>0
			and coalesce(a.fclc_mob_total,0)=0            
			and coalesce(a.fclc_total,0)<>0 
			and a.tp_big_as_is<>'M2M/IOT' 
			and a.segment_as_is IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
		then 'Миграция в др. бизнес'


		
	else 'Пусто' 
	end as ottok_mob_total_all_as_is


--8 Отток моб без вас Общ
	,CASE 
		when LAG(COALESCE(a.fclc_mob_no_vas,0)) OVER (PARTITION BY a.regid, a.app_n ORDER BY a.period)=0 
			and coalesce(a.fclc_mob_no_vas,0)=0 
		then 'Ранее перешёл в отток'

	
		when LAG(COALESCE(a.fclc_mob_no_vas,0)) OVER (PARTITION BY a.regid, a.app_n ORDER BY a.period)<>0 
			and coalesce(a.fclc_mob_no_vas, 0)=0 
			and a.segment_as_is IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
			and LAG(a.period) OVER (PARTITION BY a.regid, a.app_n ORDER BY a.period)<a.abonent_activation_month
		then 'Отток с активацией'
	
	
		when coalesce(a.fclc_mob_no_vas,0)=0 
			and LAG(COALESCE(a.fclc_mob_no_vas,0)) OVER (PARTITION BY a.regid, a.app_n ORDER BY a.period)<>0 
			and coalesce(a.fclc_total, 0)=0 
			and a.tp_big_as_is<>'M2M/IOT' 
			and a.segment_as_is IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
		then 'Отток'


		when LAG(COALESCE(a.fclc_mob_no_vas,0)) OVER (PARTITION BY a.regid, a.app_n ORDER BY a.period)<>0 
			and coalesce(a.fclc_mob_no_vas,0)=0
			and coalesce(a.fclc_total,0)<>0 
			and a.tp_big_as_is<>'M2M/IOT' 
			and a.segment_as_is IN ('Микро предприятия', 'Малый бизнес', 'Средний бизнес', 'Крупный бизнес', 
				'Национальные клиенты', 'Гос_заказчики', 'Свой круг') 
		then 'Миграция в др. бизнес'

	
	else 'Пусто' 
	end as ottok_mob_no_vas_all_as_is
	
from analyticsb2b_sb.rkahmetov_vr_mob_vrem_26 a
""").repartition(550).write.mode("overwrite").format("orc").saveAsTable("analyticsb2b_sb.rkahmetov_vr_mob_vrem_27")

In [ ]:
%%time
spark.sql("""
SELECT 
    t1.regid 
    , t1.app_n 
    , t1.service_id1
	, date_format(to_date(t1.table_business_month), 'yyyyMM') as period 
	, t2.product_pl_new
    , sum(t1.fclc_trans_part) as sum_fclc_no_vas 
FROM """ + source + """ as t1
left join """ + sch + """ as t2
		on round(t1.group_id, 0) = round(t2.group_id, 0) 
		and left(t1.sub_acc_trans, 8) = t2.sub_acc_trans
WHERE t1.regid||t1.app_n IN (SELECT distinct a.regid||a.app_n FROM analyticsb2b_sb.rkahmetov_vr_mob_vrem_27 a) 
    and lower(t1.service_id1) not like '%штраф%'
    and lower(t1.service_id1) not like '%дебет%коррект%'
    and lower(t1.service_id1) not like '%mobile%сервис%'
    and lower(t1.service_id1) not like 'b2b%оборудование%обслуживание%новая%цена%'
	and lower(t2.product_pl_new) = 'моб'
	and lower(t2.product_pl_det2) <> 'прочие vas'
group by
    t1.regid
    , t1.app_n
    , t1.service_id1
    , period
	, t2.product_pl_new
""").repartition(512).write.mode("overwrite").format("orc").saveAsTable(monthly_table+'_money_no_vas_ser')

In [ ]:
%%time
spark.sql("""
SELECT 
    t1.regid 
    , t1.app_n 
    , t1.service_id1
    , t2.service_type_one
    , t2.service_type_two
	, t1.period 
	, t1.product_pl_new
    , t1.sum_fclc_no_vas 
FROM """ + monthly_table + '_money_no_vas_ser' """ as t1
left join analyticsb2b_sb.rkahmetov_sprav_serv_type as t2
		on lower(t1.service_id1) = lower(t2.service_txt)
""").repartition(512).write.mode("overwrite").format("orc").saveAsTable(monthly_table+'_serv_1')

In [ ]:
%%time
spark.sql("""
SELECT 
    t1.regid 
    , t1.app_n 
    , t1.service_id1
	, date_format(to_date(t1.table_business_month), 'yyyyMM') as period 
	, t2.product_pl_new
    , sum(t1.fclc_trans_part) as sum_fclc_no_vas 
FROM """ + source01 + """ as t1
left join """ + sch + """ as t2
		on round(t1.group_id, 0) = round(t2.group_id, 0) 
		and left(t1.sub_acc_trans, 8) = t2.sub_acc_trans
WHERE t1.regid||t1.app_n IN (SELECT distinct a.regid||a.app_n FROM analyticsb2b_sb.rkahmetov_vr_mob_vrem_27 a) 
    and lower(t1.service_id1) not like '%штраф%'
    and lower(t1.service_id1) not like '%дебет%коррект%'
    and lower(t1.service_id1) not like '%mobile%сервис%'
    and lower(t1.service_id1) not like 'b2b%оборудование%обслуживание%новая%цена%'
	and lower(t2.product_pl_new) = 'моб'
	and lower(t2.product_pl_det2) <> 'прочие vas'
group by
    t1.regid
    , t1.app_n
    , t1.service_id1
    , period
	, t2.product_pl_new
""").repartition(11).write.mode("overwrite").format("orc").saveAsTable(monthly_table01+'_money_no_vas_ser')

In [ ]:
%%time
spark.sql("""
SELECT 
    t1.regid 
    , t1.app_n 
    , t1.service_id1
    , t2.service_type_one
    , t2.service_type_two
	, t1.period 
	, t1.product_pl_new
    , t1.sum_fclc_no_vas 
FROM """ + monthly_table01 + '_money_no_vas_ser' """ as t1
left join analyticsb2b_sb.rkahmetov_sprav_serv_type as t2
		on lower(t1.service_id1) = lower(t2.service_txt)
""").repartition(11).write.mode("overwrite").format("orc").saveAsTable(monthly_table01+'_serv_1')

In [ ]:
%%time
spark.sql("""
SELECT 
    t1.regid 
    , t1.app_n 
    , t1.service_id1
	, date_format(to_date(t1.table_business_month), 'yyyyMM') as period 
	, t2.product_pl_new
    , sum(t1.fclc_trans_part) as sum_fclc_no_vas 
FROM """ + source02 + """ as t1
left join """ + sch + """ as t2
		on round(t1.group_id, 0) = round(t2.group_id, 0) 
		and left(t1.sub_acc_trans, 8) = t2.sub_acc_trans
WHERE t1.regid||t1.app_n IN (SELECT distinct a.regid||a.app_n FROM analyticsb2b_sb.rkahmetov_vr_mob_vrem_27 a) 
    and lower(t1.service_id1) not like '%штраф%'
    and lower(t1.service_id1) not like '%дебет%коррект%'
    and lower(t1.service_id1) not like '%mobile%сервис%'
    and lower(t1.service_id1) not like 'b2b%оборудование%обслуживание%новая%цена%'
	and lower(t2.product_pl_new) = 'моб'
	and lower(t2.product_pl_det2) <> 'прочие vas'
group by
    t1.regid
    , t1.app_n
    , t1.service_id1
    , period
	, t2.product_pl_new
""").repartition(11).write.mode("overwrite").format("orc").saveAsTable(monthly_table02+'_money_no_vas_ser')

In [ ]:
%%time
spark.sql("""
SELECT 
    t1.regid 
    , t1.app_n 
    , t1.service_id1
    , t2.service_type_one
    , t2.service_type_two
	, t1.period 
	, t1.product_pl_new
    , t1.sum_fclc_no_vas 
FROM """ + monthly_table02 + '_money_no_vas_ser' """ as t1
left join analyticsb2b_sb.rkahmetov_sprav_serv_type as t2
		on lower(t1.service_id1) = lower(t2.service_txt)
""").repartition(11).write.mode("overwrite").format("orc").saveAsTable(monthly_table02+'_serv_1')

In [ ]:
%%time
spark.sql("""
SELECT 
    t1.regid 
    , t1.app_n 
    , t1.service_id1
	, date_format(to_date(t1.table_business_month), 'yyyyMM') as period 
	, t2.product_pl_new
    , sum(t1.fclc_trans_part) as sum_fclc_no_vas 
FROM """ + source03 + """ as t1
left join """ + sch + """ as t2
		on round(t1.group_id, 0) = round(t2.group_id, 0) 
		and left(t1.sub_acc_trans, 8) = t2.sub_acc_trans
WHERE t1.regid||t1.app_n IN (SELECT distinct a.regid||a.app_n FROM analyticsb2b_sb.rkahmetov_vr_mob_vrem_27 a) 
    and lower(t1.service_id1) not like '%штраф%'
    and lower(t1.service_id1) not like '%дебет%коррект%'
    and lower(t1.service_id1) not like '%mobile%сервис%'
    and lower(t1.service_id1) not like 'b2b%оборудование%обслуживание%новая%цена%'
	and lower(t2.product_pl_new) = 'моб'
	and lower(t2.product_pl_det2) <> 'прочие vas'
group by
    t1.regid
    , t1.app_n
    , t1.service_id1
    , period
	, t2.product_pl_new
""").repartition(11).write.mode("overwrite").format("orc").saveAsTable(monthly_table03+'_money_no_vas_ser')

In [ ]:
%%time
spark.sql("""
SELECT 
    t1.regid 
    , t1.app_n 
    , t1.service_id1
    , t2.service_type_one
    , t2.service_type_two
	, t1.period 
	, t1.product_pl_new
    , t1.sum_fclc_no_vas 
FROM """ + monthly_table03 + '_money_no_vas_ser' """ as t1
left join analyticsb2b_sb.rkahmetov_sprav_serv_type as t2
		on lower(t1.service_id1) = lower(t2.service_txt)
""").repartition(11).write.mode("overwrite").format("orc").saveAsTable(monthly_table03+'_serv_1')

In [ ]:
%%time
spark.sql("""
SELECT 
    t1.regid 
    , t1.app_n 
    , t1.service_id1
    , t1.service_type_one
    , t1.service_type_two
	, t1.period
    , t1.sum_fclc_no_vas 
FROM analyticsb2b_sb.rkahmetov_vr_mob_serv_1 t1
where t1.period not in ('202301', '202302', '202303')
union ALL
SELECT 
    t2.regid 
    , t2.app_n 
    , t2.service_id1
    , t2.service_type_one
    , t2.service_type_two
	, t2.period
    , t2.sum_fclc_no_vas 
FROM analyticsb2b_sb.rkahmetov_vr_mob_202301_serv_1 t2
union ALL
SELECT 
    t3.regid 
    , t3.app_n 
    , t3.service_id1
    , t3.service_type_one
    , t3.service_type_two
	, t3.period
    , t3.sum_fclc_no_vas 
FROM analyticsb2b_sb.rkahmetov_vr_mob_202302_serv_1 t3
union ALL
SELECT 
    t4.regid 
    , t4.app_n 
    , t4.service_id1
    , t4.service_type_one
    , t4.service_type_two
	, t4.period
    , t4.sum_fclc_no_vas 
FROM analyticsb2b_sb.rkahmetov_vr_mob_202303_serv_1 t4
""").repartition(545).write.mode("overwrite").format("orc").saveAsTable("analyticsb2b_sb.rkahmetov_vr_mob_serv_all")

In [ ]:
%%time
spark.sql("""

SELECT 
    t1.period
	, t1.regid 
    , t1.app_n 
    , t1.service_id1
    , t1.service_type_one
    , t1.service_type_two
	, case 
		when lower(t1.service_id1) like '%держав%' then t1.sum_fclc_no_vas
		else 0 end as fclc_derzava1
	, case 
		when lower(t1.service_id1) like '%плата%при%подключ%' then t1.sum_fclc_no_vas
		else 0 end as fclc_connect1
	, case 
		when lower(t1.service_id1) not like '%держав%' 
			and lower(t1.service_id1) not like '%плата%при%подключ%'
			and lower(t1.service_type_two) = 'периодическая' then t1.sum_fclc_no_vas
		else 0 end as fclc_periodika1
	, case 
		when lower(t1.service_id1) not like '%держав%' 
			and lower(t1.service_id1) not like '%плата%при%подключ%'
			and lower(t1.service_type_two) = 'трафик' then t1.sum_fclc_no_vas
		else 0 end as fclc_trafik1
	, case 
		when lower(t1.service_id1) not like '%держав%' 
			and lower(t1.service_id1) not like '%плата%при%подключ%'
			and lower(t1.service_type_two) = 'ба' then t1.sum_fclc_no_vas
		else 0 end as fclc_bus_abon1
	, case 
		when lower(t1.service_id1) not like '%держав%' 
			and lower(t1.service_id1) not like '%плата%при%подключ%'
			and lower(t1.service_type_two) = 'кат номер' then t1.sum_fclc_no_vas
		else 0 end as fclc_cat_number1
	, case 
		when lower(t1.service_id1) not like '%держав%' 
			and lower(t1.service_id1) not like '%плата%при%подключ%'
			and lower(t1.service_type_two) = 'инстал' then t1.sum_fclc_no_vas
		else 0 end as fclc_instal1
	, case 
		when lower(t1.service_id1) not like '%держав%' 
			and lower(t1.service_id1) not like '%плата%при%подключ%'
			and lower(t1.service_type_two) = 'разовая' then t1.sum_fclc_no_vas
		else 0 end as fclc_other_one_time1
	, t1.sum_fclc_no_vas 
FROM analyticsb2b_sb.rkahmetov_vr_mob_serv_all t1
""").repartition(545).write.mode("overwrite").format("orc").saveAsTable("analyticsb2b_sb.rkahmetov_vr_mob_serv_all_1")

In [ ]:
%%time
spark.sql("""
SELECT 
    t1.period
	, t1.regid 
    , t1.app_n
	, sum(t1.fclc_derzava1) as fclc_derzava1
	, sum(t1.fclc_connect1) as fclc_connect1
	, sum(t1.fclc_periodika1) as fclc_periodika1
	, sum(t1.fclc_trafik1) as fclc_trafik1
	, sum(t1.fclc_bus_abon1) as fclc_bus_abon1
	, sum(t1.fclc_cat_number1) as fclc_cat_number1
	, sum(t1.fclc_instal1) as fclc_instal1
	, sum(t1.fclc_other_one_time1) as fclc_other_one_time1
	, sum(t1.sum_fclc_no_vas) as sum_fclc_no_vas
FROM analyticsb2b_sb.rkahmetov_vr_mob_serv_all_1 t1
group BY
	t1.period
	, t1.regid 
    , t1.app_n
""").repartition(196).write.mode("overwrite").format("orc").saveAsTable("analyticsb2b_sb.rkahmetov_vr_mob_serv_all_2")

In [ ]:
%%time
spark.sql("""
select
	t1.period
	,t1.period_date
	,t1.regid
	,t1.app_n
	,t1.phone_num
	,t1.con_n
	,t1.inn
	,t1.is_con_inn
	,t1.kontr_name
	,t1.abonent_activation_date
	,t1.abonent_activation_month
	,t1.activation_marker
	,t1.region
	,t1.region_txt
	,t1.Reg_Week2
	,t1.TR
	
	,t1.tariff_mgr
	,t1.tariff_id_prev
	,t1.tariff_id
	,t1.tariff_prev
	,t1.tariff
	
	,t1.tp_group_mgr
	,t1.tp_group_id_prev
	,t1.tp_group_id
	,t1.tp_group_prev
	,t1.tp_group
	
	,t1.tp_big_mgr
	,t1.tp_big_prev
	,t1.tp_big
		
	,t1.m_categ_mgr
	,t1.m_categ_id_prev
	,t1.m_categ_id
	,t1.m_categ_prev
	,t1.m_categ
	
	,t1.segment_mgr
	,t1.segment_prev
	,t1.segment
	,t1.smena_segmenta
	
	,t1.fclc_prochie_mob
	,t1.fclc_prochie_vas
	,t1.fclc_golos
	,t1.fclc_gprs
	,t1.fclc_sms
	,t1.fclc_vsr
	,t1.fclc_vz_mg_mn
	,t1.fclc_abon_plata
	,t1.fclc_mob_total
	,t1.fclc_mob_no_vas
	,t1.fclc_total
	,t1.fclc_aff_serv
	
	,t1.fclc_mob_total_prev
	,t1.fclc_mob_no_vas_prev
	,t1.fclc_total_prev
	
	,t1.fclc_mob_total_x
	,t1.fclc_mob_no_vas_x
	,t1.fclc_total_x
	
	,t1.activnost_mob_total
	,t1.activnost_mob_no_vas
	
	,t1.ARPU_abonent_categ_mob_total
    ,t1.ARPU_abonent_categ_mob_no_vas
	,t1.ARPU_abonent_categ_ottoka_mob_total
    ,t1.ARPU_abonent_categ_ottoka_mob_no_vas
	
	,t1.mark_zak_block
	,t1.month_zak_block
	
	,t1.metk_aff_ser
	
	,t1.pritok_mob_total
	,t1.pritok_mob_no_vas
	,t1.pritok_mob_total_all
	,t1.pritok_mob_no_vas_all
	,t1.ottok_mob_total
	,t1.ottok_mob_no_vas
	,t1.ottok_mob_total_all
	,t1.ottok_mob_no_vas_all
	
	,t1.avg_do_ottoka_abonenta_mob_total
	,t1.avg_do_ottoka_abonenta_mob_no_vas
	,t1.rl_po_ottoku_abonenta_mob_total
	,t1.rl_po_ottoku_abonenta_mob_no_vas
	,t1.ottok_last_month_mob_total 
	,t1.ottok_last_month_mob_no_vas 
	
	,t1.proc_status_mob_total
	,t1.proc_status_mob_no_vas
	,t1.categ_proc_status_mob_total
	,t1.categ_proc_status_mob_no_vas
	
	,t1.sum_status_mob_total
	,t1.sum_status_mob_no_vas
	
	,t1.tariff_id_as_is
	,t1.tariff_as_is
	,t1.tp_group_id_as_is
	,t1.tp_group_as_is
	,t1.tp_big_as_is
	,t1.m_categ_id_as_is
	,t1.m_categ_as_is
	,t1.segment_as_is
	,t1.inn_as_is
    ,t1.kontr_name_as_is
	
	,t1.mark_inn_m2m_iot
	,t1.count_abonent
	
	,t1.pritok_mob_total_as_is
	,t1.pritok_mob_no_vas_as_is		
	,t1.pritok_mob_total_all_as_is
	,t1.pritok_mob_no_vas_all_as_is	
	,t1.ottok_mob_total_as_is
	,t1.ottok_mob_no_vas_as_is	
	,t1.ottok_mob_total_all_as_is
	,t1.ottok_mob_no_vas_all_as_is
	
	,case 
		when t1.segment in ('B2C', 'Не в B2C') then 0
		else t2.fclc_derzava1 end as fclc_derzava
	,case 
		when t1.segment in ('B2C', 'Не в B2C') then 0
		else t2.fclc_connect1 end as fclc_connect
	,case 
		when t1.segment in ('B2C', 'Не в B2C') then 0
		else t2.fclc_periodika1 end as fclc_periodika
	,case 
		when t1.segment in ('B2C', 'Не в B2C') then 0
		else t2.fclc_trafik1 end as fclc_trafik
	,case 
		when t1.segment in ('B2C', 'Не в B2C') then 0
		else t2.fclc_bus_abon1 end as fclc_bus_abon
	,case 
		when t1.segment in ('B2C', 'Не в B2C') then 0
		else t2.fclc_cat_number1 end as fclc_cat_number
	,case 
		when t1.segment in ('B2C', 'Не в B2C') then 0
		else t2.fclc_instal1 end as fclc_instal
	,case 
		when t1.segment in ('B2C', 'Не в B2C') then 0
		else t2.fclc_other_one_time1 end as fclc_other_one_time
	
from analyticsb2b_sb.rkahmetov_vr_mob_vrem_27 t1
left join analyticsb2b_sb.rkahmetov_vr_mob_serv_all_2 t2
	on t1.period = t2.period
	and t1.regid = t2.regid
	and t1.app_n = t2.app_n
""").repartition(550).write.mode("overwrite").format("orc").saveAsTable("analyticsb2b_sb.rkahmetov_vr_mob_vrem_28")

In [ ]:
%%time
spark.sql("""
select
	t1.period
	,t1.period_date
	,t1.regid
	,t1.app_n
	,t1.phone_num
	,t1.con_n
	,t1.inn
	,t1.is_con_inn
	,t1.kontr_name
	,t1.abonent_activation_date
	,t1.abonent_activation_month
	,t1.activation_marker
	,t1.region
	,t1.region_txt
	,t1.Reg_Week2
	,t1.TR
	
	,t1.tariff_mgr
	,t1.tariff_id_prev
	,t1.tariff_id
	,t1.tariff_prev
	,t1.tariff
	
	,t1.tp_group_mgr
	,t1.tp_group_id_prev
	,t1.tp_group_id
	,t1.tp_group_prev
	,t1.tp_group
	
	,t1.tp_big_mgr
	,t1.tp_big_prev
	,t1.tp_big
		
	,t1.m_categ_mgr
	,t1.m_categ_id_prev
	,t1.m_categ_id
	,t1.m_categ_prev
	,t1.m_categ
	
	,t1.segment_mgr
	,t1.segment_prev
	,t1.segment
	,t1.smena_segmenta
	
	,t1.fclc_prochie_mob
	,t1.fclc_prochie_vas
	,t1.fclc_golos
	,t1.fclc_gprs
	,t1.fclc_sms
	,t1.fclc_vsr
	,t1.fclc_vz_mg_mn
	,t1.fclc_abon_plata
	,t1.fclc_mob_total
	,t1.fclc_mob_no_vas
	,t1.fclc_total
	,t1.fclc_aff_serv
	
	,t1.fclc_mob_total_prev
	,t1.fclc_mob_no_vas_prev
	,t1.fclc_total_prev
	
	,t1.fclc_mob_total_x
	,t1.fclc_mob_no_vas_x
	,t1.fclc_total_x
	
	,t1.activnost_mob_total
	,t1.activnost_mob_no_vas
	
	,t1.ARPU_abonent_categ_mob_total
    ,t1.ARPU_abonent_categ_mob_no_vas
	,t1.ARPU_abonent_categ_ottoka_mob_total
    ,t1.ARPU_abonent_categ_ottoka_mob_no_vas
	
	,t1.mark_zak_block
	,t1.month_zak_block
	
	,t1.metk_aff_ser
	
	,t1.pritok_mob_total
	,t1.pritok_mob_no_vas
	,t1.pritok_mob_total_all
	,t1.pritok_mob_no_vas_all
	,t1.ottok_mob_total
	,t1.ottok_mob_no_vas
	,t1.ottok_mob_total_all
	,t1.ottok_mob_no_vas_all
	
	,t1.avg_do_ottoka_abonenta_mob_total
	,t1.avg_do_ottoka_abonenta_mob_no_vas
	,t1.rl_po_ottoku_abonenta_mob_total
	,t1.rl_po_ottoku_abonenta_mob_no_vas
	,t1.ottok_last_month_mob_total 
	,t1.ottok_last_month_mob_no_vas 
	
	,t1.proc_status_mob_total
	,t1.proc_status_mob_no_vas
	,t1.categ_proc_status_mob_total
	,t1.categ_proc_status_mob_no_vas
	
	,t1.sum_status_mob_total
	,t1.sum_status_mob_no_vas
	
	,t1.tariff_id_as_is
	,t1.tariff_as_is
	,t1.tp_group_id_as_is
	,t1.tp_group_as_is
	,t1.tp_big_as_is
	,t1.m_categ_id_as_is
	,t1.m_categ_as_is
	,t1.segment_as_is
	,t1.inn_as_is
    ,t1.kontr_name_as_is
	
	,t1.mark_inn_m2m_iot
	,t1.count_abonent
	
	,t1.pritok_mob_total_as_is
	,t1.pritok_mob_no_vas_as_is		
	,t1.pritok_mob_total_all_as_is
	,t1.pritok_mob_no_vas_all_as_is	
	,t1.ottok_mob_total_as_is
	,t1.ottok_mob_no_vas_as_is	
	,t1.ottok_mob_total_all_as_is
	,t1.ottok_mob_no_vas_all_as_is
	
	,t1.fclc_derzava
	,t1.fclc_connect
	,t1.fclc_periodika
	,t1.fclc_trafik
	,t1.fclc_bus_abon
	,t1.fclc_cat_number
	,t1.fclc_instal
	,t1.fclc_other_one_time
	
	,LAG(COALESCE(t1.fclc_derzava,0)) OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period) as fclc_derzava_prev
	,LAG(COALESCE(t1.fclc_connect,0)) OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period) as fclc_connect_prev
	,LAG(COALESCE(t1.fclc_periodika,0)) OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period) as fclc_periodika_prev
	,LAG(COALESCE(t1.fclc_trafik,0)) OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period) as fclc_trafik_prev
	,LAG(COALESCE(t1.fclc_bus_abon,0)) OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period) as fclc_bus_abon_prev
	,LAG(COALESCE(t1.fclc_cat_number,0)) OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period) as fclc_cat_number_prev
	,LAG(COALESCE(t1.fclc_instal,0)) OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period) as fclc_instal_prev
	,LAG(COALESCE(t1.fclc_other_one_time,0)) OVER (PARTITION BY t1.regid, t1.app_n ORDER BY t1.period) as fclc_other_one_time_prev
	
from analyticsb2b_sb.rkahmetov_vr_mob_vrem_28 t1

""").repartition(568).write.mode("overwrite").format("orc").saveAsTable("analyticsb2b_sb.rkahmetov_vr_mob_vrem_29")

In [ ]:
%%time
spark.sql("""
select
	t1.period
	,t1.period_date
	,t1.regid
	,t1.app_n
	,t1.phone_num
	,t1.con_n
	,t1.inn
	,t1.is_con_inn
	,t1.kontr_name
	,t1.abonent_activation_date
	,t1.abonent_activation_month
	,t1.activation_marker
	,t1.region
	,t1.region_txt
	,t1.Reg_Week2
	,t1.TR
	
	,t1.tariff_mgr
	,t1.tariff_id_prev
	,t1.tariff_id
	,t1.tariff_prev
	,t1.tariff
	
	,t1.tp_group_mgr
	,t1.tp_group_id_prev
	,t1.tp_group_id
	,t1.tp_group_prev
	,t1.tp_group
	
	,t1.tp_big_mgr
	,t1.tp_big_prev
	,t1.tp_big
		
	,t1.m_categ_mgr
	,t1.m_categ_id_prev
	,t1.m_categ_id
	,t1.m_categ_prev
	,t1.m_categ
	
	,t1.segment_mgr
	,t1.segment_prev
	,t1.segment
	,t1.smena_segmenta
	
	,t1.fclc_prochie_mob
	,t1.fclc_prochie_vas
	,t1.fclc_golos
	,t1.fclc_gprs
	,t1.fclc_sms
	,t1.fclc_vsr
	,t1.fclc_vz_mg_mn
	,t1.fclc_abon_plata
	,t1.fclc_mob_total
	,t1.fclc_mob_no_vas
	,t1.fclc_total
	,t1.fclc_aff_serv
	
	,t1.fclc_mob_total_prev
	,t1.fclc_mob_no_vas_prev
	,t1.fclc_total_prev
	
	,t1.fclc_mob_total_x
	,t1.fclc_mob_no_vas_x
	,t1.fclc_total_x
	
	,t1.activnost_mob_total
	,t1.activnost_mob_no_vas
	
	,t1.ARPU_abonent_categ_mob_total
    ,t1.ARPU_abonent_categ_mob_no_vas
	,t1.ARPU_abonent_categ_ottoka_mob_total
    ,t1.ARPU_abonent_categ_ottoka_mob_no_vas
	
	,t1.mark_zak_block
	,t1.month_zak_block
	
	,t1.metk_aff_ser
	
	,t1.pritok_mob_total
	,t1.pritok_mob_no_vas
	,t1.pritok_mob_total_all
	,t1.pritok_mob_no_vas_all
	,t1.ottok_mob_total
	,t1.ottok_mob_no_vas
	,t1.ottok_mob_total_all
	,t1.ottok_mob_no_vas_all
	
	,t1.avg_do_ottoka_abonenta_mob_total
	,t1.avg_do_ottoka_abonenta_mob_no_vas
	,t1.rl_po_ottoku_abonenta_mob_total
	,t1.rl_po_ottoku_abonenta_mob_no_vas
	,t1.ottok_last_month_mob_total 
	,t1.ottok_last_month_mob_no_vas 
	
	,t1.proc_status_mob_total
	,t1.proc_status_mob_no_vas
	,t1.categ_proc_status_mob_total
	,t1.categ_proc_status_mob_no_vas
	
	,t1.sum_status_mob_total
	,t1.sum_status_mob_no_vas
	
	,t1.tariff_id_as_is
	,t1.tariff_as_is
	,t1.tp_group_id_as_is
	,t1.tp_group_as_is
	,t1.tp_big_as_is
	,t1.m_categ_id_as_is
	,t1.m_categ_as_is
	,t1.segment_as_is
	,t1.inn_as_is
    ,t1.kontr_name_as_is
	
	,t1.mark_inn_m2m_iot
	,t1.count_abonent
	
	,t1.pritok_mob_total_as_is
	,t1.pritok_mob_no_vas_as_is		
	,t1.pritok_mob_total_all_as_is
	,t1.pritok_mob_no_vas_all_as_is	
	,t1.ottok_mob_total_as_is
	,t1.ottok_mob_no_vas_as_is	
	,t1.ottok_mob_total_all_as_is
	,t1.ottok_mob_no_vas_all_as_is
	
	,t1.fclc_derzava
	,t1.fclc_connect
	,t1.fclc_periodika
	,t1.fclc_trafik
	,t1.fclc_bus_abon
	,t1.fclc_cat_number
	,t1.fclc_instal
	,t1.fclc_other_one_time
	
	,t1.fclc_derzava_prev
	,t1.fclc_connect_prev
	,t1.fclc_periodika_prev
	,t1.fclc_trafik_prev
	,t1.fclc_bus_abon_prev
	,t1.fclc_cat_number_prev
	,t1.fclc_instal_prev
	,t1.fclc_other_one_time_prev
	
	,CASE
		when COALESCE(t1.fclc_mob_total,0) > 0 then 1
		else 0 end as flag_clc_mob_total
	,CASE
		when COALESCE(t1.fclc_mob_no_vas,0) > 0 then 1
		else 0 end as flag_clc_mob_no_vas 
	,CASE
		when COALESCE(t1.fclc_mob_total_prev,0) > 0 then 1
		else 0 end as flag_clc_mob_total_prev
	,CASE
		when COALESCE(t1.fclc_mob_no_vas_prev,0) > 0 then 1
		else 0 end as flag_clc_mob_no_vas_prev
	,CASE
		when COALESCE(t1.fclc_periodika,0) > 0 then 1
		else 0 end as flag_clc_fclc_periodika
	,CASE
		when COALESCE(t1.fclc_periodika_prev,0) > 0 then 1
		else 0 end as flag_clc_fclc_periodika_prev
	,CASE
		when COALESCE(t1.fclc_bus_abon,0) > 0 then 1
		else 0 end as flag_clc_fclc_bus_abon
	,CASE
		when COALESCE(t1.fclc_bus_abon_prev,0) > 0 then 1
		else 0 end as flag_clc_fclc_bus_abon_prev
	,CASE
		when COALESCE(t1.fclc_cat_number,0) > 0 then 1
		else 0 end as flag_clc_fclc_cat_number
	,CASE
		when COALESCE(t1.fclc_cat_number_prev,0) > 0 then 1
		else 0 end as flag_clc_fclc_cat_number_prev
		
from analyticsb2b_sb.rkahmetov_vr_mob_vrem_29 t1

""").repartition(568).write.mode("overwrite").format("orc").saveAsTable("analyticsb2b_sb.rkahmetov_vr_mob_vrem_30")

In [ ]:
%%time
spark.sql("""
SELECT
a.*
,LAG(COALESCE(a.fclc_MNR,0)) OVER (PARTITION BY a.regid, a.app_n ORDER BY a.period) as fclc_MNR_prev
from
(
select
	t1.period
	,t1.period_date
	,t1.regid
	,t1.app_n
	,t1.phone_num
	,t1.con_n
	,t1.inn
	,t1.is_con_inn
	,t1.kontr_name
	,t1.abonent_activation_date
	,t1.abonent_activation_month
	,t1.activation_marker
	,t1.region
	,t1.region_txt
	,t1.Reg_Week2
	,t1.TR
	
	,t1.tariff_mgr
	,t1.tariff_id_prev
	,t1.tariff_id
	,t1.tariff_prev
	,t1.tariff
	
	,t1.tp_group_mgr
	,t1.tp_group_id_prev
	,t1.tp_group_id
	,t1.tp_group_prev
	,t1.tp_group
	
	,t1.tp_big_mgr
	,t1.tp_big_prev
	,t1.tp_big
		
	,t1.m_categ_mgr
	,t1.m_categ_id_prev
	,t1.m_categ_id
	,t1.m_categ_prev
	,t1.m_categ
	
	,t1.segment_mgr
	,t1.segment_prev
	,t1.segment
	,t1.smena_segmenta
	
	,t1.fclc_prochie_mob
	,t1.fclc_prochie_vas
	,t1.fclc_golos
	,t1.fclc_gprs
	,t1.fclc_sms
	,t1.fclc_vsr
	,t1.fclc_vz_mg_mn
	,t1.fclc_abon_plata
	,t1.fclc_mob_total
	,t1.fclc_mob_no_vas
	,t1.fclc_total
	,t1.fclc_aff_serv
	
	,t1.fclc_mob_total_prev
	,t1.fclc_mob_no_vas_prev
	,t1.fclc_total_prev
	
	,t1.fclc_mob_total_x
	,t1.fclc_mob_no_vas_x
	,t1.fclc_total_x
	
	,t1.activnost_mob_total
	,t1.activnost_mob_no_vas
	
	,t1.ARPU_abonent_categ_mob_total
    ,t1.ARPU_abonent_categ_mob_no_vas
	,t1.ARPU_abonent_categ_ottoka_mob_total
    ,t1.ARPU_abonent_categ_ottoka_mob_no_vas
	
	,t1.mark_zak_block
	,t1.month_zak_block
	
	,t1.metk_aff_ser
	
	,t1.pritok_mob_total
	,t1.pritok_mob_no_vas
	,t1.pritok_mob_total_all
	,t1.pritok_mob_no_vas_all
	,t1.ottok_mob_total
	,t1.ottok_mob_no_vas
	,t1.ottok_mob_total_all
	,t1.ottok_mob_no_vas_all
	
	,t1.avg_do_ottoka_abonenta_mob_total
	,t1.avg_do_ottoka_abonenta_mob_no_vas
	,t1.rl_po_ottoku_abonenta_mob_total
	,t1.rl_po_ottoku_abonenta_mob_no_vas
	,t1.ottok_last_month_mob_total 
	,t1.ottok_last_month_mob_no_vas 
	
	,t1.proc_status_mob_total
	,t1.proc_status_mob_no_vas
	,t1.categ_proc_status_mob_total
	,t1.categ_proc_status_mob_no_vas
	
	,t1.sum_status_mob_total
	,t1.sum_status_mob_no_vas
	
	,t1.tariff_id_as_is
	,t1.tariff_as_is
	,t1.tp_group_id_as_is
	,t1.tp_group_as_is
	,t1.tp_big_as_is
	,t1.m_categ_id_as_is
	,t1.m_categ_as_is
	,t1.segment_as_is
	,t1.inn_as_is
    ,t1.kontr_name_as_is
	
	,t1.mark_inn_m2m_iot
	,t1.count_abonent
	
	,t1.pritok_mob_total_as_is
	,t1.pritok_mob_no_vas_as_is		
	,t1.pritok_mob_total_all_as_is
	,t1.pritok_mob_no_vas_all_as_is	
	,t1.ottok_mob_total_as_is
	,t1.ottok_mob_no_vas_as_is	
	,t1.ottok_mob_total_all_as_is
	,t1.ottok_mob_no_vas_all_as_is
	
	,t1.fclc_derzava
	,t1.fclc_connect
	,t1.fclc_periodika
	,t1.fclc_trafik
	,t1.fclc_bus_abon
	,t1.fclc_cat_number
	,t1.fclc_instal
	,t1.fclc_other_one_time
	
	,t1.fclc_derzava_prev
	,t1.fclc_connect_prev
	,t1.fclc_periodika_prev
	,t1.fclc_trafik_prev
	,t1.fclc_bus_abon_prev
	,t1.fclc_cat_number_prev
	,t1.fclc_instal_prev
	,t1.fclc_other_one_time_prev
	
	,t1.flag_clc_mob_total
	,t1.flag_clc_mob_no_vas 
	,t1.flag_clc_mob_total_prev
	,t1.flag_clc_mob_no_vas_prev
	,t1.flag_clc_fclc_periodika
	,t1.flag_clc_fclc_periodika_prev
	,t1.flag_clc_fclc_bus_abon
	,t1.flag_clc_fclc_bus_abon_prev
	,t1.flag_clc_fclc_cat_number
	,t1.flag_clc_fclc_cat_number_prev
	
	,t2.fclc as fclc_MNR
		
from analyticsb2b_sb.rkahmetov_vr_mob_vrem_30 t1
left join analyticsb2b_sb.rkahmetov_mnr_agr t2
	on t1.period = t2.period
	and t1.regid = t2.regid
	and t1.app_n = t2.app_n
) a

""").repartition(568).write.mode("overwrite").format("orc").saveAsTable("analyticsb2b_sb.rkahmetov_mob_analit_end")

In [ ]:
%%time
spark.sql("""
select
	t1.period
	,t1.inn
	,t1.inn_as_is
	,t1.is_con_inn
	,t1.kontr_name
	,t1.kontr_name_as_is
	,t1.region
	,t1.region_txt
	,t1.Reg_Week2
	,t1.TR
	,t1.tariff_id_prev
	,t1.tariff_prev
	,t1.tariff_id
	,t1.tariff
	,t1.tariff_mgr
	,t1.tariff_id_as_is
	,t1.tariff_as_is
	,t1.tp_group_id_prev
	,t1.tp_group_prev
	,t1.tp_group_id
	,t1.tp_group
	,t1.tp_group_mgr
	,t1.tp_group_id_as_is
	,t1.tp_group_as_is
	,t1.tp_big_prev
	,t1.tp_big
	,t1.tp_big_mgr
	,t1.tp_big_as_is
	,t1.m_categ_id_prev
	,t1.m_categ_prev
	,t1.m_categ_id
	,t1.m_categ
	,t1.m_categ_mgr
	,t1.m_categ_id_as_is
	,t1.m_categ_as_is
	,t1.segment_prev
	,t1.segment
	,t1.segment_mgr
	,t1.segment_as_is
	,t1.activation_marker
	,t1.abonent_activation_month
	,t1.ARPU_abonent_categ_mob_total
	,t1.ARPU_abonent_categ_mob_no_vas
	
	,sum(t1.fclc_total) as fclc_total
	,sum(t1.fclc_prochie_mob) as fclc_prochie_mob
	,sum(t1.fclc_gprs) as fclc_gprs
	,sum(t1.fclc_sms) as fclc_sms
	,sum(t1.fclc_vsr) as fclc_vsr
	,sum(t1.fclc_vz_mg_mn) as fclc_vz_mg_mn
	,sum(t1.fclc_abon_plata) as fclc_abon_plata
	,sum(t1.fclc_prochie_vas) as fclc_prochie_vas
	,sum(t1.fclc_mob_total) as fclc_mob_total
	,sum(t1.fclc_mob_no_vas) as fclc_mob_no_vas
	,sum(t1.fclc_total_prev) as fclc_total_prev
	,sum(t1.fclc_mob_total_prev) as fclc_mob_total_prev
	,sum(t1.fclc_mob_no_vas_prev) as fclc_mob_no_vas_prev
	
	,t1.pritok_mob_total
	,t1.pritok_mob_no_vas
	,t1.pritok_mob_total_all
	,t1.pritok_mob_no_vas_all
	,t1.ottok_mob_total
	,t1.ottok_mob_no_vas
	,t1.ottok_mob_total_all
	,t1.ottok_mob_no_vas_all
	,t1.pritok_mob_total_as_is
	,t1.pritok_mob_no_vas_as_is
	,t1.pritok_mob_total_all_as_is
	,t1.pritok_mob_no_vas_all_as_is
	,t1.ottok_mob_total_as_is
	,t1.ottok_mob_no_vas_as_is
	,t1.ottok_mob_total_all_as_is
	,t1.ottok_mob_no_vas_all_as_is
	,t1.ARPU_abonent_categ_ottoka_mob_total
	,t1.ARPU_abonent_categ_ottoka_mob_no_vas
	
	,sum(t1.avg_do_ottoka_abonenta_mob_total) as avg_do_ottoka_abonenta_mob_total
	,sum(t1.avg_do_ottoka_abonenta_mob_no_vas) as avg_do_ottoka_abonenta_mob_no_vas
	,sum(t1.rl_po_ottoku_abonenta_mob_total) as rl_po_ottoku_abonenta_mob_total
	,sum(t1.rl_po_ottoku_abonenta_mob_no_vas) as rl_po_ottoku_abonenta_mob_no_vas
	,sum(t1.sum_status_mob_total) as sum_status_mob_total
	,sum(t1.sum_status_mob_no_vas) as sum_status_mob_no_vas
	
	,t1.categ_proc_status_mob_total
	,t1.categ_proc_status_mob_no_vas
	,t1.ottok_last_month_mob_total
	,t1.ottok_last_month_mob_no_vas
	,t1.mark_zak_block
	,t1.month_zak_block
	,t1.metk_aff_ser as mark_aff_service
	,sum(t1.fclc_aff_serv) as fclc_aff_serv
	,sum(t1.count_abonent) as count_abonent
	
	,sum(t1.fclc_derzava) as fclc_derzava
	,sum(t1.fclc_periodika) as fclc_periodika
	,sum(t1.fclc_trafik) as fclc_trafik
	,sum(t1.fclc_bus_abon) as fclc_bus_abon
	,sum(t1.fclc_cat_number) as fclc_cat_number
	,sum(t1.fclc_connect) as fclc_connect
	,sum(t1.fclc_instal) as fclc_instal
	,sum(t1.fclc_other_one_time) as fclc_other_one_time
	
	,sum(t1.fclc_derzava_prev) as fclc_derzava_prev
	,sum(t1.fclc_periodika_prev) as fclc_periodika_prev
	,sum(t1.fclc_trafik_prev) as fclc_trafik_prev
	,sum(t1.fclc_bus_abon_prev) as fclc_bus_abon_prev
	,sum(t1.fclc_cat_number_prev) as fclc_cat_number_prev
	,sum(t1.fclc_connect_prev) as fclc_connect_prev
	,sum(t1.fclc_instal_prev) as fclc_instal_prev
	,sum(t1.fclc_other_one_time_prev) as fclc_other_one_time_prev
	
	,t1.flag_clc_mob_total
	,t1.flag_clc_mob_no_vas 
	,t1.flag_clc_mob_total_prev
	,t1.flag_clc_mob_no_vas_prev
	,t1.flag_clc_fclc_periodika
	,t1.flag_clc_fclc_periodika_prev
	,t1.flag_clc_fclc_bus_abon
	,t1.flag_clc_fclc_bus_abon_prev
	,t1.flag_clc_fclc_cat_number
	,t1.flag_clc_fclc_cat_number_prev
	
	,sum(t1.fclc_MNR) as fclc_MNR
	,sum(t1.fclc_MNR_prev) as fclc_MNR_prev
	
from analyticsb2b_sb.rkahmetov_mob_analit_end t1
group by 
	t1.period
	,t1.inn
	,t1.inn_as_is
	,t1.is_con_inn
	,t1.kontr_name
	,t1.kontr_name_as_is
	,t1.region
	,t1.region_txt
	,t1.Reg_Week2
	,t1.TR
	,t1.tariff_id_prev
	,t1.tariff_prev
	,t1.tariff_id
	,t1.tariff
	,t1.tariff_mgr
	,t1.tariff_id_as_is
	,t1.tariff_as_is
	,t1.tp_group_id_prev
	,t1.tp_group_prev
	,t1.tp_group_id
	,t1.tp_group
	,t1.tp_group_mgr
	,t1.tp_group_id_as_is
	,t1.tp_group_as_is
	,t1.tp_big_prev
	,t1.tp_big
	,t1.tp_big_mgr
	,t1.tp_big_as_is
	,t1.m_categ_id_prev
	,t1.m_categ_prev
	,t1.m_categ_id
	,t1.m_categ
	,t1.m_categ_mgr
	,t1.m_categ_id_as_is
	,t1.m_categ_as_is
	,t1.segment_prev
	,t1.segment
	,t1.segment_mgr
	,t1.segment_as_is
	,t1.activation_marker
	,t1.abonent_activation_month
	,t1.ARPU_abonent_categ_mob_total
	,t1.ARPU_abonent_categ_mob_no_vas
	,t1.pritok_mob_total
	,t1.pritok_mob_no_vas
	,t1.pritok_mob_total_all
	,t1.pritok_mob_no_vas_all
	,t1.ottok_mob_total
	,t1.ottok_mob_no_vas
	,t1.ottok_mob_total_all
	,t1.ottok_mob_no_vas_all
	,t1.pritok_mob_total_as_is
	,t1.pritok_mob_no_vas_as_is
	,t1.pritok_mob_total_all_as_is
	,t1.pritok_mob_no_vas_all_as_is
	,t1.ottok_mob_total_as_is
	,t1.ottok_mob_no_vas_as_is
	,t1.ottok_mob_total_all_as_is
	,t1.ottok_mob_no_vas_all_as_is
	,t1.ARPU_abonent_categ_ottoka_mob_total
	,t1.ARPU_abonent_categ_ottoka_mob_no_vas
	,t1.categ_proc_status_mob_total
	,t1.categ_proc_status_mob_no_vas
	,t1.ottok_last_month_mob_total
	,t1.ottok_last_month_mob_no_vas
	,t1.mark_zak_block
	,t1.month_zak_block
	,t1.metk_aff_ser
	
	,t1.flag_clc_mob_total
	,t1.flag_clc_mob_no_vas 
	,t1.flag_clc_mob_total_prev
	,t1.flag_clc_mob_no_vas_prev
	,t1.flag_clc_fclc_periodika
	,t1.flag_clc_fclc_periodika_prev
	,t1.flag_clc_fclc_bus_abon
	,t1.flag_clc_fclc_bus_abon_prev
	,t1.flag_clc_fclc_cat_number
	,t1.flag_clc_fclc_cat_number_prev
""").repartition(211).write.mode("overwrite").format("orc").saveAsTable("analyticsb2b_sb.rkahmetov_mob_analit_inn_agr")